# parsing the ['authorizedhighways'] column in the all_permits data for OSOW vehicle permits

## Duplicate created to clean the script while the other notebook works on pulling in the coordinates

- ns

- Originally requested by Stephen Yoon  
    - data provided by Stephen's office
- next step ==   
    - get the coordinates for the intersections

In [1]:
# import modules
import pandas as pd
import warnings
import gcsfs
import re

In [2]:
# the original_mapping is needed to standardize the highway names for the various records 
original_mapping = {
    "5": "I-5", "10": "I-10", "15": "I-15", "8": "I-8", "40": "I-40", "80": "I-80", "105": "I-105", "110": "I-110",
    "205": "I-205", "210": "I-210", "215": "I-215", "280": "I-280", "380": "I-380", "405": "I-405",
    "505": "I-505", "580": "I-580", "605": "I-605", "680": "I-680", "710": "I-710", "805": "I-805",
    "880": "I-880", "980": "I-980", "1": "SR-1", "2": "SR-2", "3": "SR-3", "4": "SR-4", "7": "SR-7",
    "9": "SR-9", "11": "SR-11", "12": "SR-12", "13": "SR-13", "14": "SR-14", 
    #"15": "SR-15",
    "16": "SR-16", "17": "SR-17", "18": "SR-18", "20": "SR-20", "22": "SR-22", "23": "SR-23",
    "24": "SR-24", "25": "SR-25", "26": "SR-26", "27": "SR-27", "28": "SR-28", "29": "SR-29",
    "32": "SR-32", "33": "SR-33", "34": "SR-34", "35": "SR-35", "36": "SR-36", "37": "SR-37",
    "38": "SR-38", "39": "SR-39", "41": "SR-41", "43": "SR-43", "44": "SR-44", "45": "SR-45",
    "46": "SR-46", "seaside highway": "SR-47", "47": "SR-47", "49": "SR-49", "51": "SR-51", "52": "SR-52", "53": "SR-53",
    "54": "SR-54", "55": "SR-55", "56": "SR-56", "57": "SR-57", "58": "SR-58", "59": "SR-59",
    "60": "SR-60", "61": "SR-61", "62": "SR-62", "63": "SR-63", "65": "SR-65", "66": "SR-66",
    "67": "SR-67", "68": "SR-68", "70": "SR-70", "71": "SR-71", "72": "SR-72", "73": "SR-73",
    "74": "SR-74", "75": "SR-75", "76": "SR-76", "77": "SR-77", "78": "SR-78", "79": "SR-79",
    "82": "SR-82", "83": "SR-83", "84": "SR-84", "85": "SR-85", "86": "SR-86", "87": "SR-87",
    "88": "SR-88", "89": "SR-89", "90": "SR-90", "91": "SR-91", "92": "SR-92", "94": "SR-94",
    "96": "SR-96", "98": "SR-98", "imperial highway": "SR-99", "99": "SR-99", "103": "SR-103", "104": "SR-104", "107": "SR-107",
    "108": "SR-108", "109": "SR-109", "110": "SR-110", "111": "SR-111", "112": "SR-112",
    "113": "SR-113", "114": "SR-114", "115": "SR-115", "116": "SR-116", "118": "SR-118",
    "119": "SR-119", "120": "SR-120", "121": "SR-121", "123": "SR-123", "124": "SR-124",
    "125": "SR-125", "126": "SR-126", "127": "SR-127", "128": "SR-128", "129": "SR-129",
    "130": "SR-130", "131": "SR-131", "132": "SR-132", "133": "SR-133", "134": "SR-134",
    "135": "SR-135", "136": "SR-136", "137": "SR-137", "138": "SR-138", "139": "SR-139",
    "140": "SR-140", "142": "SR-142", "144": "SR-144", "145": "SR-145", "146": "SR-146",
    "147": "SR-147", "149": "SR-149", "150": "SR-150", "151": "SR-151", "152": "SR-152",
    "153": "SR-153", "154": "SR-154", "155": "SR-155", "156": "SR-156", "158": "SR-158",
    "160": "SR-160", "161": "SR-161", "162": "SR-162", "163": "SR-163", "164": "SR-164",
    "165": "SR-165", "166": "SR-166", "167": "SR-167", "168": "SR-168", "169": "SR-169",
    "170": "SR-170", "172": "SR-172", "173": "SR-173", "174": "SR-174", "175": "SR-175",
    "177": "SR-177", "178": "SR-178", "180": "SR-180", "182": "SR-182", "183": "SR-183",
    "184": "SR-184", "185": "SR-185", "186": "SR-186", "187": "SR-187", "188": "SR-188",
    "189": "SR-189", "190": "SR-190", "191": "SR-191", "192": "SR-192", "193": "SR-193",
    "197": "SR-197", "198": "SR-198", "200": "SR-200", "201": "SR-201", "202": "SR-202",
    "203": "SR-203", "204": "SR-204", "207": "SR-207", "210": "SR-210", "211": "SR-211",
    "213": "SR-213", "216": "SR-216", "217": "SR-217", "218": "SR-218", "219": "SR-219",
    "220": "SR-220", "221": "SR-221", "222": "SR-222", "223": "SR-223", "227": "SR-227",
    "229": "SR-229", "232": "SR-232", "233": "SR-233", "236": "SR-236", "237": "SR-237",
    "238": "SR-238", "241": "SR-241", "242": "SR-242", "243": "SR-243", "244": "SR-244",
    "245": "SR-245", "246": "SR-246", "247": "SR-247", "253": "SR-253", "254": "SR-254",
    "255": "SR-255", "259": "SR-259", "260": "SR-260", "261": "SR-261", "262": "SR-262",
    "263": "SR-263", "265": "SR-265", "266": "SR-266", "267": "SR-267", "269": "SR-269",
    "270": "SR-270", "271": "SR-271", "273": "SR-273", "275": "SR-275", "281": "SR-281",
    "282": "SR-282", "283": "SR-283", "284": "SR-284", "299": "SR-299", "330": "SR-330",
    "371": "SR-371", "780": "SR-780", "905": "SR-905", "6": "US-6", "50": "US-50",
    "95": "US-95", "97": "US-97", "101": "US-101", "199": "US-199", "395": "US-395"
}

# Generate extended mapping to include leading zeros
road_mapping = {}
for key, value in original_mapping.items():
    road_mapping[key] = value  # Original
    road_mapping[key.zfill(2)] = value  # 2-character zero-padded
    road_mapping[key.zfill(3)] = value  # 3-character zero-padded

#print(road_mapping)

In [3]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/freight/all_permits/"

In [4]:
file_names = ["all_permits_2023_sampleset.xlsx",
              "all_permits_2024_sampleset.xlsx"]

In [5]:
# Step 1
def load_excel_sheets_1(gcs_path, file_names):
    """
    Pull in the first sheet from each Excel file in GCS, add a 'year' column based on the filename,
    and remove records with NaN values in the 'permitnumber' column. Returns a concatenated DataFrame
    with data from all files.

    Parameters:
    gcs_path (str): The Google Cloud Storage path where the files are located.
    file_names (list): A list of Excel file names in the GCS path.

    Returns:
    pd.DataFrame: A single concatenated DataFrame with data from all files, a 'year' column, and
                  records with NaN values in 'permitnumber' removed.
    """
    
    # Create a Google Cloud Storage file system object
    fs = gcsfs.GCSFileSystem()
    
    # List to store all DataFrames
    df_list = []
    
    # Suppress any warnings
    warnings.filterwarnings("ignore")
    
    # Suppress the specific UserWarning
    warnings.filterwarnings(
        "ignore",
        message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project.",
        category=UserWarning,
        module="google.auth._default"
    )
    
    # Define the columns to keep
    columns_to_keep = ['permitnumber', 'year', 'permitvalidfrom', 'permitvalidto', 
                       'loaddescription', 'origin', 'destination', 'authorizedhighways']
    
    # Loop through each file in the file list
    for file in file_names:
        # Extract the year from the filename
        year = file.split('_')[2]  # Assuming the year is the third element when split by '_'
        
        # Open the file and read only the first sheet
        with fs.open(f"{gcs_path}{file}", 'rb') as f:
            df = pd.read_excel(f, sheet_name=0)  # Load only the first sheet
        
        # Clean headers by removing spaces and making characters lowercase
        df.columns = [col.replace(" ", "").lower() for col in df.columns]
        
        # Add 'year' column
        df['year'] = year
        
        # Filter columns and remove rows with NaN in 'permitnumber'
        df = df[columns_to_keep].dropna(subset=['permitnumber'])
        
        # Append to list
        df_list.append(df)
    
    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(df_list, ignore_index=True)
    
    return final_df

# Parsing function to create individual route locations
def parse_routes(route_info):
    segments = []

    # Split the data by "from", "to", or standalone dash patterns
    raw_segments = re.split(r'\s*-\s*from\s+|\s*-\s*to\s+|(?<!\s)-\s*', route_info)

    # Process each segment
    for segment in raw_segments:
        # Split by ' - ' or '-'
        sub_segments = re.split(r'\s*-\s*|\s*-\s*', segment)
        
        # Clean and add sub-segments
        segments.extend([sub.strip() for sub in sub_segments if sub.strip()])

    return segments


# Custom parsing function
def extract_location(text):
    # Stop keywords pattern
    stop_keywords = r"\b(?:dr|drive|rd|ave|way|pkwy|parkway|skyway|road|avenue|blvd|boulevard|st|street|line|lane|ln|hwy|highway)\b"

    # If 'from' exists, process it as before
    if "from" in text.lower():
        match = re.search(r"from\s+(`.*?`|'.*?'|\w+(?:\s+\w+)*)", text, re.IGNORECASE)
        if match:
            location = match.group(1)  # Extract the text after "from"
            # Keep the stop keywords and remove everything after them
            location = re.sub(r"(" + stop_keywords + r").*", r"\1", location, flags=re.IGNORECASE).strip()
            return location
    else:
        # If 'from' doesn't exist, look for a stop keyword and capture location
        match = re.search(r"(`.*?`|'.*?'|\w+(?:\s+\w+)*)\s+(" + stop_keywords + r")", text, re.IGNORECASE)
        if match:
            location = match.group(1)  # Capture location before stop keyword
            return location.strip()

    return None  # If no match is found


# Function to clean each string 
def clean_route(route):
    if not isinstance(route, str):  # Handle non-string entries
        return route
    
    # Define stop keywords regex (including 'exit')
    stop_keywords = r"\b(?:dr|drive|rd|ave|way|pkwy|parkway|skyway|road|avenue|blvd|boulevard|st|street|line|lane|ln|hwy|highway|exit)\b"

    # Extract numeric highway number with optional "rte" or "route" prefixes
    numeric_match = re.match(r"(?:rte|route)?\s*(\d+)", route, flags=re.IGNORECASE)
    if numeric_match:
        return numeric_match.group(1)  # Return the numeric value
    
    # Extract street name up to and including the first stop keyword (but excluding the stop word itself)
    street_match = re.search(rf"(.*?\b{stop_keywords}(?!\s*exit)\b)", route, flags=re.IGNORECASE)
    if street_match:
        return street_match.group(1).strip()  # Return the street name up to the stop keyword (excluding 'exit')
    
    # If neither is found, return the original string
    return route

# A function to parse the ['authorizedhighways'] column to get the route information
def process_route_locations(df, parse_routes, extract_location, clean_route, road_mapping):
    """
    Process a dataframe to format and clean route-related columns.
    
    Parameters:
    df (pandas.DataFrame): The input dataframe to process.
    parse_routes (function): Function to parse the 'authorizedhighways' column into route segments.
    extract_location (function): Function to extract location from a route segment.
    clean_route (function): Function to clean individual route location entries.
    road_mapping (dict): Mapping dictionary for road numbers to their corresponding classes.

    Returns:
    pandas.DataFrame: The processed dataframe with updated route locations and columns.
    """

    # Format the authorized highways field so the text is not all capitalized
    df['authorizedhighways'] = df['authorizedhighways'].str.capitalize()

    # Apply the parsing function to create lists of individual route locations
    df['route_segments'] = df['authorizedhighways'].apply(parse_routes)

    # Determine the maximum number of locations to create the necessary columns
    max_locations = df['route_segments'].apply(len).max()

    # Create new columns for each route location based on the maximum number of locations
    for i in range(max_locations):
        df[f'route_location_{i}'] = df['route_segments'].apply(lambda x: x[i] if i < len(x) else None)

    # Drop the temporary route_segments column
    df.drop(columns=['route_segments'], inplace=True)

    # Add a new column with all values set to "California"
    df.insert(5, "state", "California")  # Index 5 corresponds to the 6th column position

    # Apply title case to the 'origin' and 'destination' columns
    df['origin'] = df['origin'].str.title()
    df['destination'] = df['destination'].str.title()

    # Apply the extract_location function to the column
    df["route_location_start"] = df["route_location_0"].apply(extract_location)

    # Insert the route_location_start column into the 8th position
    df.insert(8, "route_location_start", df.pop("route_location_start"))

    # Drop the [authorizedhighways] column
    #df.drop(columns=['authorizedhighways'], inplace=True)

    # Drop the route_location_0 field
    df = df.drop(columns=['route_location_0'])

    # Identify target columns excluding "route_location_start"
    route_columns = [col for col in df.columns if col.startswith("route_location_") and col != "route_location_start"]

    # Apply the clean_route cleaning function to the target columns (columns that begin with the words "route_location")
    for col in route_columns:
        df[col] = df[col].apply(clean_route)

    # Iterate through each "route_location_" column to remove the word "exit"
    for col in route_columns:
        df[col] = df[col].apply(lambda x: str(x).replace("exit", "").strip() if isinstance(x, str) else x)

    for col in route_columns:
        df[col] = df[col].astype(str)

    # Update the road numbers to their corresponding road class numbers
    for col in route_columns:
        df[col] = df[col].astype(str).map(road_mapping).fillna(df[col])  # Keep original value if no mapping found

    # Create a new field called 'route_location_origin' that identifies the street and city/state
    df['route_location_origin'] = df['route_location_start'] + " " + df['origin'] + ", " + df['state']

    # Move the new column (['route_location_origin_0']) to the 9th position
    columns = list(df.columns)
    columns.insert(9, columns.pop(columns.index('route_location_origin')))
    df = df[columns]
    
    

    # Remove the 'route_location_destination_city' column if it exists
    if 'route_location_start' in df.columns:
        df = df.drop(columns=['route_location_start'])
    
    return df


# A function to create the ['route_intersection_x'] columns
def process_route_intersections(df):
    """
    Process a dataframe to identify and process route location and intersection columns.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe to process.
    
    Returns:
    pandas.DataFrame: A cleaned dataframe with processed route intersections.
    """

    # Identify all columns with "route_location_" prefix
    route_location_cols = [col for col in df.columns if col.startswith("route_location_")]

    # Initialize a counter for the new intersection column names
    intersection_counter = 0

    # Create new columns for intersections
    for i in range(len(route_location_cols) - 1):
        col1 = route_location_cols[i]
        col2 = route_location_cols[i + 1]

        # Name the new intersection column based on the counter
        intersection_col = f"route_intersection_{intersection_counter}"

        # Combine adjacent columns into one field (handle None gracefully)
        df[intersection_col] = df[col1].astype(str) + " and " + df[col2].astype(str)
        df[intersection_col] = df[intersection_col].replace("None and None", None)  # Optional cleanup for all-None rows
        intersection_counter += 1

    # Identify columns that start with "route_intersection_"
    intersection_cols = [col for col in df.columns if col.startswith("route_intersection_")]

    # Iterate over each intersection column
    for col in intersection_cols:
        # Replace values ending with " & None" with None (Null)
        df[col] = df[col].apply(lambda x: None if isinstance(x, str) and x.endswith(" and None") else x)

    # Identify columns that start with "route_intersection_"
    intersection_cols = [col for col in df.columns if col.startswith("route_intersection_")]

    # Iterate over each intersection column to remove leading zeros from numeric values
    for col in intersection_cols:
        # Apply the transformation to each value in the column
        df[col] = df[col].apply(lambda x: ' and '.join([part.lstrip('0') if part.isdigit() else part for part in str(x).split(' and ')]) if isinstance(x, str) else x)

    # Create a list(?) called core_columns to be included in the next iteration of the dataframe
    core_columns = [
        "permitnumber", "year", "permitvalidfrom", "permitvalidto",
        "loaddescription", "state", "origin", "destination", "authorizedhighways", "route_location_origin"
    ]

    # subset_columns combines the core columns with the intersection_cols identified earlier in the script
    subset_columns = core_columns + intersection_cols

    # this next line utilizes the defined subset_columns to create a cleaned up version of the dataframe including only the columns needed for this analysis 
    df = df[subset_columns]

    # Replace None values in intersection_cols with empty strings
    for col in intersection_cols:
        df[col] = df[col].apply(lambda x: "" if x is None else x)
    
    return df

# Function to get the last 'route_intersection_x' field
def get_last_intersection(row):
    # Identify columns that match the pattern 'route_intersection_x'
    intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]
    # Get the last non-null value among these columns
    return row[intersection_columns].dropna().iloc[-1] if intersection_columns else None

def add_route_location_destination_city(df):
    # Function to get the last 'route_intersection_x' value
    def get_last_intersection(row):
        # Identify columns that match the pattern 'route_intersection_x'
        intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]
        # Get the last non-null value among these columns
        return row[intersection_columns].dropna().iloc[-1] if len(intersection_columns) > 0 else None

    # Create the new column
    df['route_location_destination_city'] = df.apply(
        lambda row: f"{get_last_intersection(row)} {row['destination']}, {row['state']}", axis=1
    )

    return df

def create_route_intersection_last(df):
    """
    Create a 'route_intersection_last' column to capture the last non-null value
    from all 'route_intersection_x' columns.

    Parameters:
    df (pandas.DataFrame): The dataframe to process.

    Returns:
    pandas.DataFrame: The dataframe with the new 'route_intersection_last' column.
    """
    # Identify all 'route_intersection_x' columns
    intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]

    if not intersection_columns:
        raise ValueError("No 'route_intersection_' columns found in the dataframe.")

    # Ensure the columns are processed in order
    intersection_columns = sorted(intersection_columns, key=lambda x: int(x.split('_')[-1]))

    # Create 'route_intersection_last' by finding the last non-null value row-wise
    df['route_intersection_last'] = df[intersection_columns].apply(
        lambda row: next((val for val in reversed(row) if pd.notnull(val) and val != ''), None), axis=1
    )

    return df

def create_route_intersection_destination(df):
    """
    Create a new column 'route_intersection_destination' by combining 'route_intersection_last' 
    and 'route_location_destination_city'. Then, clean up the column by removing extra whitespace.
    
    Parameters:
    df (pandas.DataFrame): The input dataframe to process.
    
    Returns:
    pandas.DataFrame: The dataframe with the new 'route_intersection_destination' column and cleaned columns.
    """
    # Create the new 'route_intersection_destination' column
    df['route_intersection_destination'] = df['route_intersection_last'] + " " + df['route_location_destination_city']
    
    # Remove any extra whitespace or empty strings in the resulting combination
    df['route_intersection_destination'] = df['route_intersection_destination'].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # Drop the 'route_location_destination_city' column if it exists
    if 'route_location_destination_city' in df.columns:
        df.drop(columns=['route_location_destination_city'], inplace=True)

    # Drop the 'route_intersection_last' column if it exists
    if 'route_intersection_last' in df.columns:
        df.drop(columns=['route_intersection_last'], inplace=True)

    return df

# A function to clean "route_intersection" columns
def clean_route_intersections(df):
    # Find all columns starting with "route_intersection_"
    intersection_columns = [col for col in df.columns if col.startswith("route_intersection_")]
    
    # Replace variations of "imperial highway" (e.g., "imperial hwy") with "SR-99"
    for col in intersection_columns:
        df[col] = df[col].str.replace(
            r"(?i)\bimperial (highway|hwy)\b", "SR-99", regex=True
        )  # Matches "imperial highway" or "imperial hwy"
    
    return df

def process_workflow(gcs_path, file_names):
    # Load the Excel sheets from GCS and get the initial DataFrame
    df = load_excel_sheets_1(gcs_path, file_names)
    
    # Process route locations
    df = process_route_locations(df, parse_routes, extract_location, clean_route, road_mapping)
       
    # Process route intersections
    df = process_route_intersections(df)
    
    # Add route location destination city
    df = add_route_location_destination_city(df)
    
    # Create route intersection last
    df = create_route_intersection_last(df)
    
    # Create route intersection destination
    df = create_route_intersection_destination(df)

    # this is a new script - intended to help clean up the ["route_locations_x"] before they become intersections
    df = clean_route_intersections(df)
    
    
    # # Replace the last non-null route intersection with destination
    # df = replace_last_non_null_intersection(df)
    
    return df

In [6]:
df = process_workflow(gcs_path, file_names)

/opt/conda/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
df.to_csv("justlooking_subset_v1_od.csv", index=False)

In [8]:
# Find all columns starting with "route_intersection_"
intersection_columns = [col for col in df.columns if col.startswith("route_intersection_")]


def normalize_intersection(intersection_columns):
    """
    Normalizes intersections by ordering highway identifiers numerically.
    If both sides of the intersection are highways (I-, SR-, or US-),
    ensures the smaller-numbered highway appears first.

    Args:
        intersection (str): The intersection string in the format "Location1 and Location2".

    Returns:
        str: The normalized intersection string, or the original string if no changes are needed.
    """

    
    
    if not intersection_columns or pd.isna(intersection_columns):
        return intersection_columns

    parts = [part.strip() for part in intersection_columns.split("and")]
    if len(parts) != 2:
        return intersection_columns  # Return as-is if not exactly two parts

    pattern = r"^(I-|SR-|US-)(\d+)$"  # Pattern to match highway identifiers

    match1 = re.match(pattern, parts[0])
    match2 = re.match(pattern, parts[1])

    if match1 and match2:
        # Extract numeric portions and compare
        num1 = int(match1.group(2))
        num2 = int(match2.group(2))

        if num1 > num2:
            # Swap to ensure the smaller number comes first
            parts = [parts[1], parts[0]]

    return " and ".join(parts)


def process_intersections(df, intersection_columns):
    """
    Processes and normalizes intersections in specified columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to process.

    Returns:
        pd.DataFrame: The updated DataFrame with normalized intersections.
    """
    for col in intersection_columns:
        if col in df.columns:
            df[col] = df[col].apply(normalize_intersection)
    return df

In [9]:
df = process_intersections(df, intersection_columns)

In [10]:
df.head()

,permitnumber,year,permitvalidfrom,permitvalidto,loaddescription,state,origin,destination,authorizedhighways,route_location_origin,...,route_intersection_15,route_intersection_16,route_intersection_17,route_intersection_18,route_intersection_19,route_intersection_20,route_intersection_21,route_intersection_22,route_intersection_23,route_intersection_destination
0,e23-013125,2023,02/15/2023,02/21/2023,75' KELLY BAR,California,Hayward,Antelope,* from clawiter rd s/b on ramp - 092e - 880n -...,"clawiter rd Hayward, California",...,I-80 and antelope rd,,,,,,,,,"I-80 and antelope rd Antelope, California"
1,e23-021610,2023,03/20/2023,03/26/2023,UNLADEN 9 AXLE WITH 2 DECK INSERTS,California,Fontana,Ontario,* from sierra ave w/b on ramp - 015s - 060w - ...,"sierra ave Fontana, California",...,,,,,,,,,,"SR-60 and vineyard ave Ontario, California"
2,e23-022752,2023,03/22/2023,03/28/2023,M95 TRACKED CONVEYOR,California,Dixon,Fresno,* from industrial way - 113n - 080w - 680s - 5...,"industrial way Dixon, California",...,,,,,,,,,,"SR-180 and north floyd ave Fresno, California"
3,e23-036568,2023,05/05/2023,05/11/2023,5 TROWEL MACHINES (END TO END) & MISC LEGAL FR...,California,Elk Grove,Ca/Nv Border,* from grant line rd w/b on ramp - 099n - 051n...,"grant line Elk Grove, California",...,,,,,,,,,,"I-80 and nv line Ca/Nv Border, California"
4,e23-017248,2023,03/02/2023,03/08/2023,150H GRADER,California,Fairfield,Saratoga,* from air base pkwy n/b on ramp - 080w - 680s...,"air base pkwy Fairfield, California",...,,,,,,,,,,"SR-17 and scotts valley dr Saratoga, California"


In [11]:
df.to_csv("justlooking_subset_v2.csv", index=False)

In [12]:
def clean_intersections_1(df):
    """
    Cleans the 'route_intersection_0' column in the DataFrame by replacing specific values.
    
    - Replaces values that start with "az line" and end with "I-40" with 
      "colorado river bridge and I-10, California".
    
    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The updated DataFrame with cleaned intersections.
    """
    # Check if the column exists
    if "route_intersection_0" in df.columns:
        # Debugging: Display the initial state of the column
        print("Before cleaning:")
        print(df["route_intersection_0"].head())
        
        # Define the replacement logic
        def replace_intersection(value):
            if isinstance(value, str) and value.lower().startswith("az line") and value.endswith("I-40"):
                return "colorado river bridge and I-10, California"
            return value

        # Apply the function to clean the column
        df["route_intersection_0"] = df["route_intersection_0"].apply(replace_intersection)
        
        # Debugging: Display the updated state of the column
        print("After cleaning:")
        print(df["route_intersection_0"].head())
    else:
        print("Column 'route_intersection_0' does not exist in the DataFrame.")

    return df

In [13]:
df = clean_intersections_1(df)

Before cleaning:
0       clawiter rd Hayward, California and SR-92
1         sierra ave Fontana, California and I-15
2     industrial way Dixon, California and SR-113
3      grant line Elk Grove, California and SR-99
4    air base pkwy Fairfield, California and I-80
Name: route_intersection_0, dtype: object
After cleaning:
0       clawiter rd Hayward, California and SR-92
1         sierra ave Fontana, California and I-15
2     industrial way Dixon, California and SR-113
3      grant line Elk Grove, California and SR-99
4    air base pkwy Fairfield, California and I-80
Name: route_intersection_0, dtype: object


In [14]:
df.head()

,permitnumber,year,permitvalidfrom,permitvalidto,loaddescription,state,origin,destination,authorizedhighways,route_location_origin,...,route_intersection_15,route_intersection_16,route_intersection_17,route_intersection_18,route_intersection_19,route_intersection_20,route_intersection_21,route_intersection_22,route_intersection_23,route_intersection_destination
0,e23-013125,2023,02/15/2023,02/21/2023,75' KELLY BAR,California,Hayward,Antelope,* from clawiter rd s/b on ramp - 092e - 880n -...,"clawiter rd Hayward, California",...,I-80 and antelope rd,,,,,,,,,"I-80 and antelope rd Antelope, California"
1,e23-021610,2023,03/20/2023,03/26/2023,UNLADEN 9 AXLE WITH 2 DECK INSERTS,California,Fontana,Ontario,* from sierra ave w/b on ramp - 015s - 060w - ...,"sierra ave Fontana, California",...,,,,,,,,,,"SR-60 and vineyard ave Ontario, California"
2,e23-022752,2023,03/22/2023,03/28/2023,M95 TRACKED CONVEYOR,California,Dixon,Fresno,* from industrial way - 113n - 080w - 680s - 5...,"industrial way Dixon, California",...,,,,,,,,,,"SR-180 and north floyd ave Fresno, California"
3,e23-036568,2023,05/05/2023,05/11/2023,5 TROWEL MACHINES (END TO END) & MISC LEGAL FR...,California,Elk Grove,Ca/Nv Border,* from grant line rd w/b on ramp - 099n - 051n...,"grant line Elk Grove, California",...,,,,,,,,,,"I-80 and nv line Ca/Nv Border, California"
4,e23-017248,2023,03/02/2023,03/08/2023,150H GRADER,California,Fairfield,Saratoga,* from air base pkwy n/b on ramp - 080w - 680s...,"air base pkwy Fairfield, California",...,,,,,,,,,,"SR-17 and scotts valley dr Saratoga, California"


In [15]:
df.to_csv("justlooking_subset_v4.csv", index=False)

In [16]:
def update_route_intersection(df):
    # Create a new column with the same values as ['route_intersection_0']
    df['route_intersection_origin'] = df['route_intersection_0']
    
    # Define replacement mappings
    replacements = {
        "colorado river bridge and I-10, California": "I-10 and Arizona Line"
    }
    
    # Pattern-based replacements
    pattern_replacements = [
        (r'^az line.*and I-10$', "I-10 and Arizona Line"),
        (r'^az line.*and I-8$', "I-8 and Arizona Line"),
        (r'^az line.*and SR-62$', "SR-62 and Arizona Line"),
        (r'^or line.*and I-5$', "I-5 and Oregon Line"),
        (r'^or line.*and US-97$', "US-97 and Oregon Line"),
        (r'^or line.*and US-395$', "US-395 and Oregon Line"),
        (r'^or line.*and SR-139$', "SR-139 and Oregon Line"),
        (r'^nv line.*and I-80$', "I-80 and Nevada Line"),
        (r'^nv line.*and SR-15$', "I-15 and Nevada Line"),
        (r'^nv line.*and US-6$', "US-6 and Nevada Line"),
        (r'^nv line.*and US-395$', "US-395 and Nevada Line"),
        (r'^nv line.*and SR-178$', "SR-178 and Nevada Line")
    ]
    
    # Apply direct replacements
    df['route_intersection_origin'] = df['route_intersection_origin'].replace(replacements)
    
    # Apply pattern-based replacements
    for pattern, replacement in pattern_replacements:
        df.loc[df['route_intersection_origin'].str.match(pattern, case=False, na=False), 
               'route_intersection_origin'] = replacement
    
    # Place the new column in the 10th position
    cols = list(df.columns)
    cols.insert(9, cols.pop(cols.index('route_intersection_origin')))
    df = df[cols]
    
    # Remove the original column
    df.drop(columns=['route_intersection_0'], inplace=True)
    
    return df

In [17]:
df = update_route_intersection(df)

In [18]:
df.to_csv("justlooking_subset_v5.csv", index=False)

In [19]:
def update_route_intersection_destination(df):
    # Create a new column with the same values as ['route_intersection_destination']
    df['route_intersection_destination_1'] = df['route_intersection_destination']
    
    # Define replacement mappings
    replacements = {
        "I-40 and az line": "I-40 and Arizona Line",
        "I-10 and az line": "I-10 and Arizona Line",
        "I-8 and az line": "I-8 and Arizona Line",
        "SR-62 and az line": "SR-62 and Arizona Line",
        "I-80 and nv line": "I-80 and Nevada Line",
        "I-5 and or line": "I-5 and Oregon Line",
        "SR-15 and nv line": "SR-15 and Nevada Line",
        "I-15 and nv line": "I-15 and Nevada Line"
    }
    
    # Pattern-based replacements
    pattern_replacements = [
        (r'I-40 and az line', "I-40 and Arizona Line"),
        (r'I-10 and az line', "I-10 and Arizona Line"),
        (r'I-8 and az line', "I-8 and Arizona Line"),
        (r'SR-62 and az line', "SR-62 and Arizona Line"),
        (r'I-80 and nv line', "I-80 and Nevada Line"),
        (r'I-5 and or line', "I-5 and Oregon Line"),
        (r'SR-15 and nv line', "I-15 and Nevada Line"),
        (r'I-15 and nv line', "I-15 and Nevada Line")
    ]
    
    # Apply direct replacements
    df['route_intersection_destination_1'] = df['route_intersection_destination_1'].replace(replacements)
    
    # Apply pattern-based replacements
    for pattern, replacement in pattern_replacements:
        df.loc[df['route_intersection_destination_1'].str.contains(pattern, case=False, na=False), 
               'route_intersection_destination_1'] = replacement
    
    # Drop the original column
    df.drop(columns=['route_intersection_destination'], inplace=True)
    
    # Rename the new column
    df.rename(columns={'route_intersection_destination_1': 'route_intersection_destination'}, inplace=True)
    
    return df

In [20]:
df = update_route_intersection_destination(df)

In [21]:
df.to_csv("justlooking_subset_v6.csv", index=False)

In [22]:
mid_route_intersections = [
    {
        "mid_route_intersections": "I-5 and I-10, California",
        "x_coords": "-118.2193173",
        "y_coords": "34.0395767"
    },
    {
        "mid_route_intersections": "I-5 and I-80, California",
        "x_coords": "-121.5171204", 
        "y_coords": "38.6248864"
    },
    {
        "mid_route_intersections": "I-5 and I-205, California",
        "x_coords": "-121.3212098", 
        "y_coords": "37.7747996"
    },
    {
        "mid_route_intersections": "I-5 and I-405, California",
        "x_coords": "-117.7303127", 
        "y_coords": "33.6389622"
    },
    {
        "mid_route_intersections": "I-5 and I-8, California",
        "x_coords": "-117.2037869", 
        "y_coords": "32.7591082"
    },
    {
        "mid_route_intersections": "I-5 and SR-12, California",
        "x_coords": "-121.3978848", 
        "y_coords": "38.1154351"
    },
    {
        "mid_route_intersections": "I-5 and SR-14, California",
        "x_coords": "-118.507427", 
        "y_coords": "34.3326266"
    },
    {
        "mid_route_intersections": "I-5 and SR-15, California",
        "x_coords": "-117.1211288", 
        "y_coords": "32.6923764"
    },
    {
        "mid_route_intersections": "I-5 and SR-16, California",
        "x_coords": "-120.2791601", 
        "y_coords": "36.3078477"
    },
    {
        "mid_route_intersections": "I-5 and SR-20, California",
        "x_coords": "-122.1498933", 
        "y_coords": "39.1665678"
    },
    {
        "mid_route_intersections": "I-5 and SR-22, California",
        "x_coords": "-117.8769603", 
        "y_coords": "33.7788598"
    },
    {
        "mid_route_intersections": "I-5 and SR-32, California",
        "x_coords": "-122.205501", 
        "y_coords": "39.75032"
    },
    {
        "mid_route_intersections": "I-5 and SR-33, California",
        "x_coords": "-120.3923114", 
        "y_coords": "36.4334225"
    },
    {
        "mid_route_intersections": "I-5 and SR-36, California",
        "x_coords": "-122.2445409", 
        "y_coords": "40.2041737"
    },
    {
        "mid_route_intersections": "I-5 and SR-41, California",
        "x_coords": "-119.9603248", 
        "y_coords": "35.9816544"
    },
    {
        "mid_route_intersections": "I-5 and SR-43, California",
        "x_coords": "-119.2525845", 
        "y_coords": "35.2930485"
    },
    {
        "mid_route_intersections": "I-5 and SR-44, California",
        "x_coords": "-122.3605092", 
        "y_coords": "40.5850561"
    },
    {
        "mid_route_intersections": "I-5 and SR-46, California",
        "x_coords": "-119.6532836", 
        "y_coords": "35.6161199"
    },
    {
        "mid_route_intersections": "I-5 and SR-52, California",
        "x_coords": "-117.2347813", 
        "y_coords": "32.8376004"
    },
    {
        "mid_route_intersections": "I-5 and SR-55, California",
        "x_coords": "-117.83382", 
        "y_coords": "33.7405912"
    },
    {
        "mid_route_intersections": "I-5 and SR-56, California",
        "x_coords": "-117.2408403",
        "y_coords": "32.9301482"
    },
    {
        "mid_route_intersections": "I-5 and SR-57, California",
        "x_coords": "-117.880123", 
        "y_coords": "33.7822912"
    },
    {
        "mid_route_intersections": "I-5 and SR-73, California",
        "x_coords": "-117.6738146", 
        "y_coords": "33.5425909"
    },
    {
        "mid_route_intersections": "I-5 and SR-74, California",
        "x_coords": "-117.6584611", 
        "y_coords": "33.5022528"
    },
    {
        "mid_route_intersections": "I-5 and SR-75, California",
        "x_coords": "-117.1424314", 
        "y_coords": "32.7009545"
    },
    {
        "mid_route_intersections": "I-5 and SR-76, California",
        "x_coords": "-117.3830207", 
        "y_coords": "33.2044458"
    },
    {
        "mid_route_intersections": "I-5 and SR-78, California",
        "x_coords": "-117.3513025", 
        "y_coords": "33.1770414"
    },
    {
        "mid_route_intersections": "I-5 and SR-89, California",
        "x_coords": "-122.304868", 
        "y_coords": "41.286287"
    },
    {
        "mid_route_intersections": "I-5 and SR-91, California",
        "x_coords": "-117.9803962", 
        "y_coords": "33.8569961"
    },
    {
        "mid_route_intersections": "I-5 and SR-94, California",
        "x_coords": "-117.1476619", 
        "y_coords": "32.7143796"
    },
    {
        "mid_route_intersections": "I-5 and SR-118, California",
        "x_coords": "-118.4435609", 
        "y_coords": "34.2667135"
    },
    {
        "mid_route_intersections": "I-5 and SR-119, California",
        "x_coords": "-119.224524", 
        "y_coords": "35.2670658"
    },
    {
        "mid_route_intersections": "I-5 and SR-120, California",
        "x_coords": "-121.3018261", 
        "y_coords": "37.7887748"
    },
    {
        "mid_route_intersections": "I-5 and SR-126, California",
        "x_coords": "-118.6055142", 
        "y_coords": "34.4431988"
    },
    {
        "mid_route_intersections": "I-5 and SR-132, California",
        "x_coords": "-121.3430842", 
        "y_coords": "37.6377555"
    },
    {
        "mid_route_intersections": "I-5 and SR-133, California",
        "x_coords": "-117.7533201", 
        "y_coords": "33.6674393"
    },
    {
        "mid_route_intersections": "I-5 and SR-134, California",
        "x_coords": "-118.2847469", 
        "y_coords": "34.1528621"
    },
    {
        "mid_route_intersections": "I-5 and SR-138, California",
        "x_coords": "-118.7986074", 
        "y_coords": "34.7640468"
    },
    {
        "mid_route_intersections": "I-5 and SR-140, California",
        "x_coords": "-121.0889606", 
        "y_coords": "37.2461183"
    },
    {
        "mid_route_intersections": "I-5 and SR-152, California",
        "x_coords": "-120.9698301", 
        "y_coords": "37.0566575"
    },
    {
        "mid_route_intersections": "I-5 and SR-162, California",
        "x_coords": "-122.2147204", 
        "y_coords": "39.524194"
    },
    {
        "mid_route_intersections": "I-5 and SR-165, California",
        "x_coords": "-120.8404977", 
        "y_coords": "36.9302781"
    },
    {
        "mid_route_inersections": "I-5 and SR-166, California",
        "x_coords": "-118.999163", 
        "y_coords": "35.059316"
    },
    {
        "mid_route_intersections": "I-5 and SR-170, California",
        "x_coords": "-118.4101239", 
        "y_coords": "34.2330844"
    },
    {
        "mid_route_intersections": "I-5 and SR-198, California",
        "x_coords": "-120.2439689", 
        "y_coords": "36.2543345"
    },
    {
        "mid_route_intersections": "I-5 and SR-210, California",
        "x_coords": "-118.4903133", 
        "y_coords": "34.3159777"
    },
    {
        "mid_route_intersections": "I-5 and SR-223, California",
        "x_coords": "-119.1629213", 
        "y_coords": "35.209104"
    },
    {
        "mid_route_intersections": "I-5 and SR-269, California",
        "x_coords": "-120.1020456", 
        "y_coords": "36.0767521"
    },
    {
        "mid_route_intersections": "I-5 and SR-273, California",
        "x_coord": "-122.28333", 
        "y_coord": "40.4272485"
    },
    {
        "mid_route_intersections": "I-5 and SR-299, California",
        "x_coord": "-122.3632521", 
        "y_coord": "40.6121579"
    },
    {
        "mid_route_intersections": "I-5 and SR-905, California",
        "x_coord": "-117.0734143", 
        "y_coord": "32.5651478"
    },
    {
        "mid_route_intersections": "I-5 and US-50, California",
        "x_coord": "-121.5108873", 
        "y_coord": "38.5682793"
    },
    {
        "mid_route_intersections": "I-5 and US-97, California",
        "x_coord": "-122.3825292", 
        "y_coord": "41.41573"
    },
    {
        "mid_route_intersections": "I-10 and I-215, California",
        "x_coord": "-117.2964956", 
        "y_coord": "34.0641924"
    },
    {
        "mid_route_intersections": "I-10 and I-405, California",
        "x_coord": "-118.4335953", 
        "y_coord": "34.0312671"
    },
    {
        "mid_route_intersections": "I-10 and I-605, California",
        "x_coord": "-118.0000932", 
        "y_coord": "34.0650647"
    },
    {
        "mid_route_intersections": "I-10 and I-710, California",
        "x_coord": "-118.1650635", 
        "y_coord": "34.0616746"
    },
    {
        "mid_route_intersections": "I-10 and SR-15, California",
        "x_coord": "-117.5449899", 
        "y_coord": "34.06694"
    },
    {
        "mid_route_intersections": "I-10 and SR-22, California",
        "x_coord": "-118.0562305",
        "y_coord": "33.7555338"
    },
    {
        "mid_route_intersections": "I-10 and SR-57, California",
        "x_coord": "-117.8065739",
        "y_coord": "34.0651133"
    },
    {
        "mid_route_intersections": "I-10 and SR-60, California",
        "x_coord": "-116.9884664",
        "y_coord": "33.9323636"
    },
    {
        "mid_route_intersections": "I-10 and SR-62, California",
        "x_coord": "-116.6043518",
        "y_coord": "33.9142755"
    },
    {
        "mid_route_intersections": "I-10 and SR-78, California",
        "x_coord": "-114.6576732",
        "y_coord": "33.6065263"
    },
    {
        "mid_route_intersections": "I-10 and SR-79, California",
        "x_coord": "-116.9771835",
        "y_coord": "33.9269216"
    },
    {
        "mid_route_intersections": "I-10 and SR-83, California",
        "x_coord": "-117.6511013",
        "y_coord": "34.0873237"
    },
    {
        "mid_route_intersections": "I-10 and SR-86, California",
        "x_coord": "-116.1932279",
        "y_coord": "33.7178667"
    },
    {
        "mid_route_intersections": "I-10 and SR-111, California",
        "x_coord": "-116.2070171",
        "y_coord": "33.7149312"
    },
    {
        "mid_route_intersections": "I-10 and SR-152, California",
        "x_coord": "-121.084865",
        "y_coord": "37.077806"
    },
    {
        "mid_route_intersections": "I-10 and SR-177, California",
        "x_coord": "-115.4002514",
        "y_coord": "33.7104463"
    },
    {
        "mid_route_intersections": "I-10 and SR-243, California",
        "x_coord": "-116.8854203",
        "y_coord": "33.9241006"
    },
    {
        "mid_route_intersections": "I-10 and US-95, California",
        "x_coord": "-114.5706989",
        "y_coord": "33.6072721"
    },
    {
        "mid_route_intersections": "I-40 and US-95, California",
        "x_coord": "-114.5969206",
        "y_coord": "34.8240555"
    },
    {
        "mid_route_intersections": "I-80 and I-505, California",
        "x_coord": "-121.9563057",
        "y_coord": "38.370683"
    },
    {
        "mid_route_intersections": "I-80 and I-580, California",
        "x_coord": "-122.3083543",
        "y_coord": "37.8896701"
    },
    {
        "mid_route_intersections": "I-80 and I-680, California",
        "x_coord": "-122.1379333",
        "y_coord": "38.2163797"
    },
    {
        "mid_route_intersections": "I-80 and I-880, California",
        "x_coord": "-122.2936744",
        "y_coord": "37.8287862"
    },
    {
        "mid_route_intersections": "I-80 and SR-89, California",
        "x_coord": "-120.2077074",
        "y_coord": "39.3230493"
    },
    {
        "mid_route_intersections": "I-80 and SR-113, California",
        "x_coord": "-121.7702521",
        "y_coord": "38.519704"
    },
    {
        "mid_route_intersections": "I-80 and SR-193, California",
        "x_coord": "-121.1257945",
        "y_coord": "38.8780956"
    },
    {
        "mid_route_intersections": "I-80 and SR-780, California",
        "x_coord": "-122.2304448",
        "y_coord": "38.091502"
    },
    {
        "mid_route_intersections": "I-105 and I-405, California",
        "x_coord": "-118.3683925",
        "y_coord": "33.9299777"
    },
    {
        "mid_route_intersections": "I-105 and I-605, California",
        "x_coord": "-118.1048384",
        "y_coord": "33.9139358"
    },
    {
        "mid_route_intersections": "I-105 and I-710, California",
        "x_coord": "-118.1801015",
        "y_coord": "33.9128225"
    },
    {
        "mid_route_intersections": "I-105 and SR-110, California",
        "x_coord": "-118.2807593",
        "y_coord": "33.9286602"
    },
    {
        "mid_route_intersections": "I-215 and SR-259, California",
        "x_coord": "-117.3027148",
        "y_coord": "34.127001"
    },
    {
        "mid_route_intersections": "I-280 and I-380, California",
        "x_coord": "-122.4309517",
        "y_coord": "37.6272684"
    },
    {
        "mid_route_intersections": "I-280 and I-680, California",
        "x_coord": "-121.8589036",
        "y_coord": "37.3352733"
    },
    {
        "mid_route_intersections": "I-280 and I-880, California",
        "x_coord": "-121.9402621",
        "y_coord": "37.3172818"
    },
    {
        "mid_route_intersections": "I-405 and I-605, California",
        "x_coord": "-118.0938338",
        "y_coord": "33.7863509"
    },
    {
        "mid_route_intersections": "I-405 and I-710, California",
        "x_coord": "-118.2074519",
        "y_coord": "33.8263771"
    },
    {
        "mid_route_intersections": "I-680 and SR-780, California",
        "x_coord": "-122.1304573",
        "y_coord": "38.0510052"
    },
    {
        "mid_route_intersections": "I-805 and SR-905, California",
        "x_coord": "-117.0406132",
        "y_coord": "32.5681478"
    },
    {
        "mid_route_intersections": "SR-1 and I-5, California",
        "x_coord": "-117.669851",
        "y_coord": "33.4669366"
    },
    {
        "mid_route_intersections": "SR-1 and I-105, California",
        "x_coord": "-118.3961351",
        "y_coord": "33.9313854"
    },
    {
        "mid_route_intersections": "SR-1 and I-710, California",
        "x_coord": "-118.2078449",
        "y_coord": "33.7898405"
    },
    {
        "mid_route_intersections": "SR-1 and SR-9, California",
        "x_coord": "-122.0297558",
        "y_coord": "36.9844096"
    },
    {
        "mid_route_intersections": "SR-1 and SR-55, California",
        "x_coord": "-117.9293219",
        "y_coord": "33.6212163"
    },
    {
        "mid_route_intersections": "SR-1 and SR-103, California",
        "x_coord": "-118.2249691",
        "y_coord": "33.7897639"
    },
    {
        "mid_route_intersections": "SR-1 and SR-110, California",
        "x_coord": "-118.2820661",
        "y_coord": "33.7905789"
    },
    {
        "mid_route_intersections": "SR-1 and SR-118, California",
        "x_coord": "-119.5144457",
        "y_coord": "34.2516422"
    },
    {
        "mid_route_intersections": "SR-1 and SR-129, California",
        "x_coord": "-121.774179",
        "y_coord": "36.894236"
    },
    {
        "mid_route_intersections": "SR-1 and SR-156, California",
        "x_coord": "-121.7656141",
        "y_coord": "36.7529187"
    },
    {
        "mid_route_intersections": "SR-1 and SR-223, California",
        "x_coord": "-121.7767206",
        "y_coord": "36.762416"
    },
    {
        "mid_route_intersections": "SR-2 and I-405, California",
        "x_coord": "-118.4467267",
        "y_coord": "34.0469528"
    },
    {
        "mid_route_intersections": "SR-2 and SR-134, California",
        "x_coord": "-118.226278",
        "y_coord": "34.1467022"
    },
    {
        "mid_route_intersections": "SR-2 and SR-210, California",
        "x_coord": "-118.2175196",
        "y_coord": "34.2074836"
    },
    {
        "mid_route_intersections": "SR-2 and US-95, California",
        "x_coord": "-114.7829911",
        "y_coord": "32.4839304"
    },
    {
        "mid_route_intersections": "SR-3 and I-5, California",
        "x_coord": "-122.6278309",
        "y_coord": "41.7394717"
    },
    {
        "mid_route_intersections": "SR-4 and SR-160, California",
        "x_coord": "-121.7572058",
        "y_coord": "37.9908358"
    },
    {
        "mid_route_intersections": "SR-4 and SR-242, California",
        "x_coord": "-122.0359094",
        "y_coord": "38.0060299"
    },
    {
        "mid_route_intersections": "SR-7 and SR-8, California",
        "x_coord": "-115.3836718",
        "y_coord": "32.7732264"
    },
    {
        "mid_route_intersections": "SR-7 and SR-65, California",
        "x_coord": "-119.0746319",
        "y_coord": "35.4413814"
    },
    {
        "mid_route_intersections": "SR-8 and I-805, California",
        "x_coord": "-117.1319771",
        "y_coord": "32.7717861"
    },
    {
        "mid_route_intersections": "SR-8 and SR-15, California",
        "x_coord": "-117.1123522",
        "y_coord": "32.7785497"
    },
    {
        "mid_route_intersections": "SR-8 and SR-67, California",
        "x_coord": "-116.9624199",
        "y_coord": "32.8030392"
    },
    {
        "mid_route_intersections": "SR-8 and SR-86, California",
        "x_coord": "-115.5695068",
        "y_coord": "32.7737888"
    },
    {
        "mid_route_intersections": "SR-8 and SR-98, California",
        "x_coord": "-115.0892642",
        "y_coord": "32.7105553"
    },
    {
        "mid_route_intersections": "SR-8 and SR-111, California",
        "x_coord": "-115.5011888",
        "y_coord": "32.7737429"
    },
    {
        "mid_route_intersections": "SR-8 and SR-115, California",
        "x_coord": "-115.2856203",
        "y_coord": "32.7729486"
    },
    {
        "mid_route_intersections": "SR-8 and SR-125, California",
        "x_coord": "-117.003655",
        "y_coord": "32.7776181"
    },
    {
        "mid_route_intersections": "SR-8 and SR-163, California",
        "x_coord": "-117.1633704",
        "y_coord": "32.7624981"
    },
    {
        "mid_route_intersections": "SR-12 and I-680, California",
        "x_coord": "-122.1376249",
        "y_coord": "38.2165412"
    },
    {
        "mid_route_intersections": "SR-12 and SR-99, California",
        "x_coord": "-121.2578895",
        "y_coord": "38.1162481"
    },
    {
        "mid_route_intersections": "SR-12 and US-101, California",
        "x_coord": "-122.7154274",
        "y_coord": "38.4311076"
    },
    {
        "mid_route_intersections": "SR-14 and SR-58, California",
        "x_coord": "-118.1721488",
        "y_coord": "35.0477644"
    },
    {
        "mid_route_intersections": "SR-15 and I-40, California",
        "x_coord": "-117.0112902",
        "y_coord": "34.8858921"
    },
    {
        "mid_route_intersections": "SR-15 and I-215, California",
        "x_coord": "-117.1768221",
        "y_coord": "33.5431454"
    },
    {
        "mid_route_intersections": "SR-15 and I-805, California",
        "x_coord": "-117.1116087",
        "y_coord": "32.7320531"
    },
    {
        "mid_route_intersections": "SR-15 and SR-52, California",
        "x_coord": "-117.1206401",
        "y_coord": "32.8418193"
    },
    {
        "mid_route_intersections": "SR-15 and SR-58, California",
        "x_coord": "-117.0764783",
        "y_coord": "34.870668"
    },
    {
        "mid_route_intersections": "SR-15 and SR-60, California",
        "x_coord": "-117.550282",
        "y_coord": "34.0216929"
    },
    {
        "mid_route_intersections": "SR-15 and SR-74, California",
        "x_coord": "-117.3375886",
        "y_coord": "33.691937"
    },
    {
        "mid_route_intersections": "SR-15 and SR-76, California",
        "x_coord": "-117.1592442",
        "y_coord": "33.3310553"
    },
    {
        "mid_route_intersections": "SR-15 and SR-78, California",
        "x_coord": "-117.1048215",
        "y_coord": "33.1270234"
    },
    {
        "mid_route_intersections": "SR-15 and SR-79, California",
        "x_coord": "-117.1629555",
        "y_coord": "33.5224701"
    },
    {
        "mid_route_intersections": "SR-15 and SR-91, California",
        "x_coord": "-117.5472415",
        "y_coord": "33.881847"
    },
    {
        "mid_route_intersections": "SR-15 and SR-94, California",
        "x_coord": "-117.1178271",
        "y_coord": "32.7162961"
    },
    {
        "mid_route_intersections": "SR-15 and SR-127, California",
        "x_coord": "-116.0732067",
        "y_coord": "35.2633089"
    },
    {
        "mid_route_intersections": "SR-15 and SR-163, California",
        "x_coord": "-117.1136803",
        "y_coord": "32.8581269"
    },
    {
        "mid_route_intersections": "SR-15 and SR-210, California",
        "x_coord": "-117.4959677",
        "y_coord": "34.1362949"
    },
    {
        "mid_route_intersections": "SR-15 and SR-247, California",
        "x_coord": "-117.0229073",
        "y_coord": "34.8859224"
    },
    {
        "mid_route_intersections": "SR-15 and US-395, California",
        "x_coord": "-117.4018882",
        "y_coord": "34.3990487"
    },
    {
        "mid_route_intersections": "SR-16 and I-505, California",
        "x_coord": "-121.9520726",
        "y_coord": "38.6796051"
    },
    {
        "mid_route_intersections": "SR-17 and I-280, California",
        "x_coord": "-121.940286",
        "y_coord": "37.316506"
    },
    {
        "mid_route_intersections": "SR-18 and SR-38, California",
        "x_coord": "-116.8455725",
        "y_coord": "34.2611375"
    },
    {
        "mid_route_intersections": "SR-18 and SR-247, California",
        "x_coord": "-116.9496312",
        "y_coord": "34.4438103"
    },
    {
        "mid_route_intersections": "SR-20 and SR-29, California",
        "x_coord": "-122.9213631",
        "y_coord": "39.1634382"
    },
    {
        "mid_route_intersections": "SR-20 and SR-53, California",
        "x_coord": "-122.6140787",
        "y_coord": "39.0086394"
    },
    {
        "mid_route_intersections": "SR-22 and I-605, California",
        "x_coord": "-118.0912555",
        "y_coord": "33.7847504"
    },
    {
        "mid_route_intersections": "SR-22 and SR-39, California",
        "x_coord": "-117.9923035",
        "y_coord": "33.7710974"
    },
    {
        "mid_route_intersections": "SR-22 and SR-55, California",
        "x_coord": "-117.8333988",
        "y_coord": "33.7767961"
    },
    {
        "mid_route_intersections": "SR-22 and SR-57, California",
        "x_coord": "-117.879924",
        "y_coord": "33.7853374"
    },
    {
        "mid_route_intersections": "SR-24 and I-680, California",
        "x_coord": "-122.071081",
        "y_coord": "37.8985594"
    },
    {
        "mid_route_intersections": "SR-25 and SR-156, California",
        "x_coord": "-121.4272778",
        "y_coord": "36.8954298"
    },
    {
        "mid_route_intersections": "SR-25 and US-101, California",
        "x_coord": "-121.5510535",
        "y_coord": "36.9611334"
    },
    {
        "mid_route_intersections": "SR-26 and SR-99, California",
        "x_coord": "-121.2458016",
        "y_coord": "37.9699259"
    },
    {
        "mid_route_intersections": "SR-27 and SR-118, California",
        "x_coord": "-118.6047846",
        "y_coord": "34.2777048"
    },
    {
        "mid_route_intersections": "SR-27 and US-101, California",
        "x_coord": "-118.6058136",
        "y_coord": "34.1707228"
    },
    {
        "mid_route_intersections": "SR-29 and I-80, California",
        "x_coord": "-122.2317522",
        "y_coord": "38.075832"
    },
    {
        "mid_route_intersections": "SR-29 and SR-37, California",
        "x_coord": "-122.2560807",
        "y_coord": "38.1399243"
    },
    {
        "mid_route_intersections": "SR-29 and SR-221, California",
        "x_coord": "-122.2694572",
        "y_coord": "38.2417237"
    },
    {
        "mid_route_intersections": "SR-33 and US-101, California",
        "x_coord": "-119.3053327",
        "y_coord": "34.2787708"
    },
    {
        "mid_route_intersections": "SR-37 and I-80, California",
        "x_coord": "-122.2168017",
        "y_coord": "38.1454187"
    },
    {
        "mid_route_intersections": "SR-37 and US-101, California",
        "x_coord": "-122.5459499",
        "y_coord": "38.0798641"
    },
    {
        "mid_route_intersections": "SR-39 and I-405, California",
        "x_coord": "-117.9893587",
        "y_coord": "33.7325282"
    },
    {
        "mid_route_intersections": "SR-39 and SR-91, California",
        "x_coord": "-117.9980141",
        "y_coord": "33.8560464"
    },
    {
        "mid_route_intersections": "SR-41 and SR-99, California",
        "x_coord": "-119.7847169",
        "y_coord": "36.7176454"
    },
    {
        "mid_route_intersections": "SR-41 and SR-198, California",
        "x_coord": "-119.8080698",
        "y_coord": "36.2820593"
    },
    {
        "mid_route_intersections": "SR-43 and SR-58, California",
        "x_coord": "-119.2521044",
        "y_coord": "35.3542946"
    },
    {
        "mid_route_intersections": "SR-43 and SR-119, California",
        "x_coord": "-119.2526877",
        "y_coord": "35.2672514"
    },
    {
        "mid_route_intersections": "SR-43 and SR-198, California",
        "x_coord": "-119.6009198",
        "y_coord": "36.3251738"
    },
    {
        "mid_route_intersections": "SR-46 and SR-99, California",
        "x_coord": "-119.2112125",
        "y_coord": "35.6017319"
    },
    {
        "mid_route_intersections": "SR-47 and I-710, California",
        "x_coord": "-118.2385084",
        "y_coord": "33.7583399"
    },
    {
        "mid_route_intersections": "SR-47 and SR-110, California",
        "x_coord": "-118.2907046",
        "y_coord": "33.7492427"
    },
    {
        "mid_route_intersections": "SR-49 and I-80, California",
        "x_coord": "-121.0747006",
        "y_coord": "38.9002061"
    },
    {
        "mid_route_intersections": "SR-52 and I-805, California",
        "x_coord": "-117.1780977",
        "y_coord": "32.8474801"
    },
    {
        "mid_route_intersections": "SR-52 and SR-67, California",
        "x_coord": "-116.962491",
        "y_coord": "32.8322913"
    },
    {
        "mid_route_intersections": "SR-52 and SR-125, California",
        "x_coord": "-117.0031747",
        "y_coord": "32.8364922"
    },
    {
        "mid_route_intersections": "SR-52 and SR-163, California",
        "x_coord": "-117.1335355",
        "y_coord": "32.8411165"
    },
    {
        "mid_route_intersections": "SR-54 and I-805, California",
        "x_coord": "-117.0745078",
        "y_coord": "32.658292"
    },
    {
        "mid_route_intersections": "SR-55 and I-405, California",
        "x_coord": "-117.8722219",
        "y_coord": "33.6873201"
    },
    {
        "mid_route_intersections": "SR-55 and SR-73, California",
        "x_coord": "-117.8862283",
        "y_coord": "33.6733476"
    },
    {
        "mid_route_intersections": "SR-56 and I-805, California",
        "x_coord": "-117.2338534",
        "y_coord": "32.9171369"
    },
    {
        "mid_route_intersections": "SR-57 and SR-71, California",
        "x_coord": "-117.807419",
        "y_coord": "34.0683769"
    },
    {
        "mid_route_intersections": "SR-57 and SR-90, California",
        "x_coord": "-117.8834422",
        "y_coord": "33.9107697"
    },
    {
        "mid_route_intersections": "SR-57 and SR-91, California",
        "x_coord": "-117.8758752",
        "y_coord": "33.8498298"
    },
    {
        "mid_route_intersections": "SR-57 and SR-210, California",
        "x_coord": "-117.8243558",
        "y_coord": "34.1205105"
    },
    {
        "mid_route_intersections": "SR-58 and SR-99, California",
        "x_coord": "-119.0395304",
        "y_coord": "35.3526592"
    },
    {
        "mid_route_intersections": "SR-58 and SR-184, California",
        "x_coord": "-118.9141569",
        "y_coord": "35.3521819"
    },
    {
        "mid_route_intersections": "SR-58 and SR-202, California",
        "x_coord": "-118.4673695",
        "y_coord": "35.1413514"
    },
    {
        "mid_route_intersections": "SR-58 and SR-204, California",
        "x_coord": "-119.0029996",
        "y_coord": "35.3524168"
    },
    {
        "mid_route_intersections": "SR-58 and US-395, California",
        "x_coord": "-117.542581",
        "y_coord": "34.9957715"
    },
    {
        "mid_route_intersections": "SR-59 and SR-152, California",
        "x_coord": "-120.4921135",
        "y_coord": "37.0833327"
    },
    {
        "mid_route_intersections": "SR-60 and I-605, California",
        "x_coord": "-118.0272198",
        "y_coord": "34.0339903"
    },
    {
        "mid_route_intersections": "SR-60 and I-710, California",
        "x_coord": "-118.1700494",
        "y_coord": "34.0356009"
    },
    {
        "mid_route_intersections": "SR-60 and SR-71, California",
        "x_coord": "-117.7492327",
        "y_coord": "34.0232317"
    },
    {
        "mid_route_intersections": "SR-60 and SR-83, California",
        "x_coord": "-117.6509453",
        "y_coord": "34.030361"
    },
    {
        "mid_route_intersections": "SR-61 and I-880, California",
        "x_coord": "-122.1869313",
        "y_coord": "37.7159549"
    },
    {
        "mid_route_intersections": "SR-63 and SR-198, California",
        "x_coord": "-119.3013104",
        "y_coord": "36.3267232"
    },
    {
        "mid_route_intersections": "SR-99 and SR-219, California",
        "x_coord": "-121.0810425",
        "y_coord": "37.7090017"
    },
    {
        "mid_route_intersections": "US-101 and SR-299, California",
        "x_coord": "-124.0830274",
        "y_coord": "40.8967431"
    }
]

In [23]:
# stored as a list, not sure if I like this option - this will be a test run

# a list of locations pulled from the subset of OSOW permit routes
origin_intersections = [
    {
        "origin_intersection": "10th st Gilroy, California and US-101",
        "x_coords": "-121.556489",
        "y_coords": "37.0030168"
    },
    {
        "origin_intersection": "11th ave Hanford, California and SR-198",
        "x_coords": "-119.6550819",
        "y_coords": "36.3213196"
    },
    {
        "origin_intersection": "11th st 656 Wool Creek Drive, San Jose, California and I-280",
        "x_coords": "-121.8712243",
        "y_coords": "37.3284839"
    },
    {
        "origin_intersection": "13th st 180 Niblick Rd, Paso Robles, Ca 93446, California and US-101",
        "x_coords": "-120.6886902",
        "y_coords": "35.6152439"
    },
    {
        "origin_intersection": "165th st Lancaster, California and SR-138",
        "x_coords": "-117.8353998",
        "y_coords": "34.5060998"
    },
    {
        "origin_intersection": "165th st Palmdale, California and SR-138",
        "x_coords": "-117.8353752",
        "y_coords": "34.5060976"
    },
    {
        "origin_intersection": "16th st Merced, California and SR-99",
        "x_coords": "-120.5005912",
        "y_coords": "37.3031708"
    },
    {
        "origin_intersection": "1st ave San Diego, California and I-5",
        "x_coords": "-117.1638923",
        "y_coords": "32.7233218"
    },    
    {
        "origin_intersection": "1st st Beaumont, California and SR-79",
        "x_coords": "-116.9771788",
        "y_coords": "33.9216701"
    },
    {
        "origin_intersection": "20th st Santa Monica, California and I-10",
        "x_coords": "-118.4737752",
        "y_coords": "34.0233192"
    },
    {
        "origin_intersection": "20th st Santa Monica, California and morn10w",
        "x_coords": "-118.4737377",
        "y_coords": "34.0232892"
    },
    {
        "origin_intersection": "29th ave Alameda, California and I-880",
        "x_coords": "-122.2334309",
        "y_coords": "37.7750177"
    },
    {
        "origin_intersection": "2nd st Ripon, California and SR-99",
        "x_coords": "-121.1240604",
        "y_coords": "37.7374009"
    },
    {
        "origin_intersection": "2nd st San Bernadino, California and I-215",
        "x_coords": "-117.3018694",
        "y_coords": "34.1026965"
    },
    {
        "origin_intersection": "3 flags rd Hesperia, California and US-395",
        "x_coords": "-117.3991953",
        "y_coords": "34.4102256"
    },
    {
        "origin_intersection": "30th st San Bernardino, California and SR-18",
        "x_coords": "-117.2786977",
        "y_coords": "34.1459495"
    },
    {
        "origin_intersection": "65th st Sacramento, California and US-50",
        "x_coords": "-121.4277722",
        "y_coords": "38.5505406"
    },
    {
        "origin_intersection": "66th ave Thermal, California and SR-86",
        "x_coords": "-116.0925458",
        "y_coords": "33.5693012"
    },
    {
        "origin_intersection": "6th ave Hamilton City, California and SR-32",
        "x_coords": "-122.0193043",
        "y_coords": "39.7445086"
    },
    {
        "origin_intersection": "7th ave Industry, California and SR-60",
        "x_coords": "-117.9881563",
        "y_coords": "34.0194698"
    },
    {
        "origin_intersection": "7th standard rd 31101 Coberly Rd, Shafter, Ca 93263, California and SR-99",
        "x_coords": "-119.0842878",
        "y_coords": "35.4413718"
    },
    {
        "origin_intersection": "7th standard rd Bakersfield, Ca, California and SR-99",
        "x_coords": "-119.0842995",
        "y_coords": "35.441406"
    },
    {
        "origin_intersection": "7th standard rd Bakersfield, California and I-5",
        "x_coords": "-119.4567879",
        "y_coords": "35.4422626"
    },
    {
        "origin_intersection": "7th standard rd Bakersfield, California and SR-43",
        "x_coords": "-119.2518323",
        "y_coords": "35.4416595"
    },
    {
        "origin_intersection": "7th standard rd Bakersfield, California and SR-99",
        "x_coords": "-119.0842982",
        "y_coords": "35.4413913"
    },
    {
        "origin_intersection": "7th standard rd Shafter, California and I-5",
        "x_coords": "-119.4567183",
        "y_coords": "35.4422256"
    },
    {
        "origin_intersection": "8th st 2205 Belt St. San Diego Ca. 92113, California and I-5",
        "x_coords": "-117.1583397",
        "y_coords": "32.7233137"
    },
    {
        "origin_intersection": "98th ave San Leandro, California and I-880",
        "x_coords": "-122.1885614",
        "y_coords": "37.7316942"
    },
    {
        "origin_intersection": "acacia rd Sutter, California and SR-20",
        "x_coords": "-121.753985",
        "y_coords": "39.1423148"
    },
    {
        "origin_intersection": "academy ave Sanger, California and SR-180",
        "x_coords": "-119.5562908",
        "y_coords": "36.7359975"
    },
    {
        "origin_intersection": "adeline st Oakland, California and I-880",
        "x_coords": "-122.2885645",
        "y_coords": "37.8028911"
    },
    {
        "origin_intersection": "air base parkway Fairfield, California and I-80",
        "x_coords": "-122.0497322",
        "y_coords": "38.2733662"
    },
    {
        "origin_intersection": "air base pkwy Fairfield, California and I-80",
        "x_coords": "-122.0497162",
        "y_coords": "38.273392"
    },
    {
        "origin_intersection": "air expressway Adelanto, California and US-395",
        "x_coords": "-117.4095832",
        "y_coords": "34.5720079"
    },
    {
        "origin_intersection": "air expressway Phelan, California and US-395",
        "x_coords": "-117.4095751",
        "y_coords": "34.5720079"
    },
    {
        "origin_intersection": "airport blvd 10 Harris Place Salinas, Ca, California and US-101",
        "x_coords": "-121.6228739",
        "y_coords": "36.6608137"
    },
    {
        "origin_intersection": "airport blvd 800 S Airport Blvd, San Francisco, Ca 94128, California and US-101",
        "x_coords": "-122.4055361",
        "y_coords": "37.6467612"
    },
    {
        "origin_intersection": "airport blvd Salinas, Ca, California and US-101",
        "x_coords": "-121.6228966",
        "y_coords": "36.6607931"
    },
    {
        "origin_intersection": "airport blvd Thermal, California and SR-86",
        "x_coords": "-116.1329125",
        "y_coords": "33.6420874"
    },
    {
        "origin_intersection": "airport dr Bakersfield, Ca, California and SR-204",
        "x_coords": "-119.0414374",
        "y_coords": "35.4020789"
    },

    
        {
        "origin_intersection": "alabama st Loma Linda, California and I-10",
        "x_coords": "-117.2087662",
        "y_coords": "34.0665861"
    },
    {
        "origin_intersection": "alabama st Redlands, California and I-10",
        "x_coords": "-117.2087914",
        "y_coords": "34.0666322"
    },
    {
        "origin_intersection": "alabama st San Bernardino, California and I-10",
        "x_coords": "-117.2087586",
        "y_coords": "34.066649"
    },
    {
        "origin_intersection": "alameda ave Burbank, California and SR-134",
        "x_coords": "-118.34134",
        "y_coords": "34.1537733"
    },
    {
        "origin_intersection": "alameda st Compton, California and SR-91",
        "x_coords": "-118.2168766",
        "y_coords": "33.8734705"
    },
    {
        "origin_intersection": "alder ave Rialto, California and SR-210",
        "x_coords": "-117.4185059",
        "y_coords": "34.136394"
    },
    {
        "origin_intersection": "alessandro blvd Moreno Valley, California and I-215",
        "x_coords": "-117.2875138",
        "y_coords": "33.9167236"
    },
    {
        "origin_intersection": "alessandro blvd Riverside, California and I-215",
        "x_coords": "-117.2875709",
        "y_coords": "33.9167467"
    },
    {
        "origin_intersection": "alexander ave 5501 East St, Anderson, Ca, 96007, California and SR-273",
        "x_coords": "-122.3149276",
        "y_coords": "40.4596843"
    },
    {
        "origin_intersection": "alicia parkway Mission Viejo, California and I-5",
        "x_coords": "-117.6904914",
        "y_coords": "33.6045494"
    },
    {
        "origin_intersection": "almond drive 22101 Sunset Ave, Los Banos, Ca, California and SR-165",
        "x_coords": "-120.8371258",
        "y_coords": "36.9984151"
    },
    {
        "origin_intersection": "alondra blvd La Mirada, Ca, California and I-605",
        "x_coords": "-118.1044191",
        "y_coords": "33.8875219"
    },
    {
        "origin_intersection": "alondra blvd Long Beach, California and I-710",
        "x_coords": "-118.1888896",
        "y_coords": "33.8890399"
    },
    {
        "origin_intersection": "alondra blvd Paramount, California and I-710",
        "x_coords": "-118.188908",
        "y_coords": "33.8890376"
    },
    {
        "origin_intersection": "alton pkwy Irvine, California and I-5",
        "x_coords": "-117.7439578",
        "y_coords": "33.6556871"
    },
    {
        "origin_intersection": "altus ave Mojave, California and SR-58",
        "x_coords": "-118.1057555",
        "y_coords": "35.0385746"
    },
    {
        "origin_intersection": "american ave Fresno, California and SR-41",
        "x_coords": "-119.7859424",
        "y_coords": "36.6633056"
    },
    {
        "origin_intersection": "anaheim st Wilmington Ca, California and SR-110",
        "x_coords": "-118.2802393",
        "y_coords": "33.778995"
    },
    {
        "origin_intersection": "anaheim st Wilmington, California and SR-110",
        "x_coords": "-118.2802675",
        "y_coords": "33.778996"
    },
    {
        "origin_intersection": "applegate rd Atwater, California and SR-140",
        "x_coords": "-120.6148642",
        "y_coords": "37.3021133"
    },
    {
        "origin_intersection": "arch rd 4800 Logistics Dr, Stockton, Ca, California and SR-99",
        "x_coords": "-121.2229306",
        "y_coords": "37.9048093"
    },
    {
        "origin_intersection": "arch rd Stockton, California and SR-99",
        "x_coords": "-121.2229219",
        "y_coords": "37.9048026"
    },
    {
        "origin_intersection": "archibald ave Ontario, California and SR-60",
        "x_coords": "-117.593243",
        "y_coords": "34.030571"
    },
    {
        "origin_intersection": "arrow highway San Dimas, California and SR-57",
        "x_coords": "-117.8216541",
        "y_coords": "34.1066534"
    },
    {
        "origin_intersection": "arrow hwy Irwindale, California and I-605",
        "x_coords": "-117.9777621",
        "y_coords": "34.1149467"
    },
    {
        "origin_intersection": "artesia blvd La Mirada, Ca, California and SR-91",
        "x_coords": "-118.0601716",
        "y_coords": "33.8731305"
    },

    {
        "origin_intersection": "artesia blvd La Mirada, California and SR-91",
        "x_coords": "-118.0602681",
        "y_coords": "33.8731929"
    },
    {
        "origin_intersection": "ashlan ave Fresno, California and SR-99",
        "x_coords": "-119.8654838",
        "y_coords": "36.7935706"
    },
    {
        "origin_intersection": "atlanta ave Huntington Beach 311 Atlanta, California and SR-39",
        "x_coords": "-117.9883449",
        "y_coords": "33.6576235"
    },
    {
        "origin_intersection": "atlantic ave Long Beach, California and I-405",
        "x_coords": "-118.1850564",
        "y_coords": "33.8153166"
    },
    {
        "origin_intersection": "atlantic blvd Vernon, California and I-710",
        "x_coords": "-118.176805",
        "y_coords": "33.9975235"
    },
    {
        "origin_intersection": "austin rd Ripon, California and SR-99",
        "x_coords": "-121.1793133",
        "y_coords": "37.7746714"
    },
    {
        "origin_intersection": "auto center dr Corona, California and SR-91",
        "x_coords": "-117.6131214",
        "y_coords": "33.8806952"
    },
    {
        "origin_intersection": "auto mall parkway Freemont, California and I-680",
        "x_coords": "-121.9674544",
        "y_coords": "37.5073591"
    },
    {
        "origin_intersection": "avalon blvd Wilmington, Ca, California and SR-1",
        "x_coords": "-118.2633289",
        "y_coords": "33.7911538"
    },
    {
        "origin_intersection": "avalon blvd Wilmington, California and SR-1",
        "x_coords": "-118.263334",
        "y_coords": "33.791156"
    },
    {
        "origin_intersection": "ave 15341 Rd 28 1/2 Madra, Ca, California and SR-99",
        "x_coords": "-120.028455",
        "y_coords": "36.9381609"
    },
    {
        "origin_intersection": "ave 15341 Rd 28 1/2, Madera, Ca 93638, California and SR-99",
        "x_coords": "-120.0284329",
        "y_coords": "36.9381576"
    },
    {
        "origin_intersection": "ave 2300 W Pecan Ave, Madera, Ca, 93637, California and SR-99",
        "x_coords": "-120.056125",
        "y_coords": "36.9382979"
    },
    {
        "origin_intersection": "ave 3250 E Ave M, Palmdale, Ca 93550, California and SR-14",
        "x_coords": "-118.1533258",
        "y_coords": "34.6457081"
    },
    {
        "origin_intersection": "ave 3520 East Ave M, Palmdale, Ca, California and SR-14",
        "x_coords": "-118.1532543",
        "y_coords": "34.6456641"
    },
    {
        "origin_intersection": "azalea ave Mckinleyville Ca, California and SR-200",
        "x_coords": "-124.0795898",
        "y_coords": "40.9159302"
    },
    {
        "origin_intersection": "azusa ave Azusa, California and SR-210",
        "x_coords": "-117.9076149",
        "y_coords": "34.1206108"
    },
    {
        "origin_intersection": "backus rd Rosamond, California and SR-14",
        "x_coords": "-118.1611412",
        "y_coords": "34.9518835"
    },
    {
        "origin_intersection": "bacon island rd Stockton, California and SR-4",
        "x_coords": "-121.4357393",
        "y_coords": "37.9219836"
    },
    {
        "origin_intersection": "bailey rd Mountain Pass, California and I-15",
        "x_coords": "-115.5286965",
        "y_coords": "35.4681819"
    },
    {
        "origin_intersection": "bailey rd Pittsburg, California and SR-4",
        "x_coords": "-121.9419457",
        "y_coords": "38.0188672"
    },
    {
        "origin_intersection": "baker st Lake Elsinore, California and SR-74",
        "x_coords": "-117.3504332",
        "y_coords": "33.6913211"
    },
    {
        "origin_intersection": "balboa ave 4200 Ruffin Rd, San Diego Ca. 92123, California and I-15",
        "x_coords": "-117.1165199",
        "y_coords": "32.8228885"
    },
    {
        "origin_intersection": "balboa ave 4200 Ruffin Rd, San Diego, Ca, California and I-15",
        "x_coords": "-117.1165279",
        "y_coords": "32.8228671"
    },
    {
        "origin_intersection": "balboa ave San Diego, Ca, California and I-15",
        "x_coords": "-117.1165495",
        "y_coords": "32.8228627"
    },
    {
        "origin_intersection": "balboa ave San Diego, California and I-15",
        "x_coords": "-117.1165528",
        "y_coords": "32.8228883"
    },
    {
        "origin_intersection": "bandini blvd Commerce, California and Interstate-710",
        "x_coords": "-118.1762518",
        "y_coords": "33.9986325"
    },
    {
        "origin_intersection": "bardsdale ave Fillmore, California and SR-23",
        "x_coords": "-118.9198303",
        "y_coords": "34.3715693"
    },
    {
        "origin_intersection": "barham dr San Marcos, California and SR-78",
        "x_coords": "-117.1450296",
        "y_coords": "33.1389814"
    },
    {
        "origin_intersection": "barlow rd Lake Isabella, California and SR-155",
        "x_coords": "-118.4797942",
        "y_coords": "35.6298253"
    },
    {
        "origin_intersection": "baseline rd Azusa, California and SR-210",
        "x_coords": "-117.892687",
        "y_coords": "34.1214304"
    },
    {
        "origin_intersection": "bay marina dr National City, California and I-5",
        "x_coords": "-117.108049",
        "y_coords": "32.6600782"
    },
    {
        "origin_intersection": "bay marina wy e San Diego, California and I-5",
        "x_coords": "-117.108001",
        "y_coords": "32.660086"
    },
    {
        "origin_intersection": "bayshore rd Benicia, California and I-680",
        "x_coords": "-122.1308938",
        "y_coords": "38.0643755"
    },
    {
        "origin_intersection": "bear creek rd Boulder Creek, California and SR-9",
        "x_coords": "-122.1230114",
        "y_coords": "37.1284841"
    },
    {
        "origin_intersection": "bear valley rd Victorville, California and I-15",
        "x_coords": "-117.3461908",
        "y_coords": "34.4703646"
    },
    {
        "origin_intersection": "beech ave Bakersfield, California and SR-43",
        "x_coords": "-119.2609486",
        "y_coords": "35.4588035"
    },
    {
        "origin_intersection": "belcher ave Merced, California and SR-59",
        "x_coords": "-120.5043192",
        "y_coords": "37.3391884"
    },
    {
        "origin_intersection": "bellflower blvd 9606 Firestone Blvd, Downey, Ca, 90241, California and I-105",
        "x_coords": "-118.1254009",
        "y_coords": "33.9131829"
    },
    {
        "origin_intersection": "bellflower blvd Downey, California and I-105",
        "x_coords": "-118.1254195",
        "y_coords": "33.9131336"
    },
    {
        "origin_intersection": "ben hur rd Mariposa, California and SR-49",
        "x_coords": "-119.9473606",
        "y_coords": "37.4625423"
    },
    {
        "origin_intersection": "bernal rd San Jose, California and US-101",
        "x_coords": "-121.7727698",
        "y_coords": "37.2440546"
    },
    {
        "origin_intersection": "betteravia rd Santa Maria, California and US-101",
        "x_coords": "-120.4359852",
        "y_coords": "34.9207724"
    },
    {
        "origin_intersection": "bicentennial way Santa Rosa, California and US-101",
        "x_coords": "-122.7294317",
        "y_coords": "38.4705781"
    },
    {
        "origin_intersection": "big bend rd Oroville, California and SR-70",
        "x_coords": "-121.502653",
        "y_coords": "39.7249756"
    },
    {
        "origin_intersection": "binghampton rd Dixon, California and SR-113",
        "x_coords": "-121.8231345",
        "y_coords": "38.3512577"
    },
    {
        "origin_intersection": "blaine st Riverside, California and I-215",
        "x_coords": "-117.3428273",
        "y_coords": "33.9830037"
    },
    {
        "origin_intersection": "bloomfield rd Gilroy, California and SR-152",
        "x_coords": "-121.4909086",
        "y_coords": "36.9886104"
    },    
    {
        "origin_intersection": "blue oaks blvd Roseville, California and SR-65",
        "x_coords": "-121.2973669",
        "y_coords": "38.7956229"
    },
    {
        "origin_intersection": "bob hope dr Rancho Mirage, California and I-10",
        "x_coords": "-116.4091025",
        "y_coords": "33.8200992"
    },
    {
        "origin_intersection": "bob hope dr Thousand Palms, California and I-10",
        "x_coords": "-116.4090647",
        "y_coords": "33.8201008"
    },
    {
        "origin_intersection": "bolsa chica rd Westminster, California and SR-22",
        "x_coords": "-118.037736",
        "y_coords": "33.7745364"
    },
    {
        "origin_intersection": "borax rd Boron, California and SR-58",
        "x_coords": "-117.7032708",
        "y_coords": "35.0067881"
    },
    {
        "origin_intersection": "boron ave Boron, California and SR-58",
        "x_coords": "-117.649825",
        "y_coords": "35.0102766"
    },
    {
        "origin_intersection": "boundry pl Los Angeles, California and SR-1",
        "x_coords": "-118.3959874",
        "y_coords": "33.8771174"
    },
    {
        "origin_intersection": "bowman rd Ridgcrest, California and US-395",
        "x_coords": "-117.7627123",
        "y_coords": "35.6076139"
    },
    {
        "origin_intersection": "bradley ave El Cajon, California and SR-67",
        "x_coords": "-116.9606034",
        "y_coords": "32.8191264"
    },
    {
        "origin_intersection": "bradley rd E Garrison San Miguel, California and US-101",
        "x_coords": "-120.7633853",
        "y_coords": "35.8456176"
    },
    {
        "origin_intersection": "bradley rd San Miguel, California and US-101",
        "x_coords": "-120.7633927",
        "y_coords": "35.8456156"
    },
    {
        "origin_intersection": "bradshaw rd Sacramento, California and US-50",
        "x_coords": "-121.3363403",
        "y_coords": "38.5659396"
    },
    {
        "origin_intersection": "britannia blvd 8580 Avenida Costa Norte, San Diego, Ca, 9215, California and SR-905",
        "x_coords": "-116.9798163",
        "y_coords": "32.5642925"
    },
    {
        "origin_intersection": "britannia blvd Otay Mesa, California and SR-905",
        "x_coords": "-116.9798445",
        "y_coords": "32.5642929"
    },
    {
        "origin_intersection": "britannia blvd San Diego, California and SR-905",
        "x_coords": "-116.9798173",
        "y_coords": "32.5642479"
    },
    {
        "origin_intersection": "broadway ave Salida, California and SR-99",
        "x_coords": "-121.0810494",
        "y_coords": "37.7090101"
    },
    {
        "origin_intersection": "brookhurst st Garden Grove, California and SR-22",
        "x_coords": "-117.9550344",
        "y_coords": "33.7658995"
    },
    {
        "origin_intersection": "brundage ln Bakersfield, California and SR-204",
        "x_coords": "-119.0029914",
        "y_coords": "35.353851"
    },
    {
        "origin_intersection": "bundy canyon rd Lake Elsinore, California and I-15",
        "x_coords": "-117.2729209",
        "y_coords": "33.6268791"
    },
    {
        "origin_intersection": "burbank blvd Burbank, California and I-5",
        "x_coords": "-118.3181909",
        "y_coords": "34.1854299"
    },
    {
        "origin_intersection": "butterfield ranch rd Chino, California and SR-71",
        "x_coords": "-117.6594816",
        "y_coords": "33.9307001"
    },
    {
        "origin_intersection": "butts canyon Middletown, California and SR-29",
        "x_coords": "-122.6070166",
        "y_coords": "38.756877"
    },
    {
        "origin_intersection": "cajalco rd San Jacinto, California and Interstate-15",
        "x_coords": "-117.5187075",
        "y_coords": "33.8213802"
    },
    {
        "origin_intersection": "calaveras rd Sunol, California and I-680",
        "x_coords": "-121.8708277",
        "y_coords": "37.5888109"
    },
    {
        "origin_intersection": "calico rd Yermo, California and I-15",
        "x_coords": "-116.8381714",
        "y_coords": "34.9092626"
    },
    {
        "origin_intersection": "california ave Bakersfield, California and SR-99",
        "x_coords": "-119.0418394",
        "y_coords": "35.3683196"
    },

    {
        "origin_intersection": "California SR-62 and Arizona Line",
        "x_coords": "-114.2992694",
        "y_coords": "34.1596547"
    },
    {
        "origin_intersection": "california st Highland, California and I-10",
        "x_coords": "-117.2261635",
        "y_coords": "34.066607"
    },
    {
        "origin_intersection": "camarillo springs rd Camarillo, California and US-101",
        "x_coords": "-118.9836261",
        "y_coords": "34.2052546"
    },
    {
        "origin_intersection": "camino del norte e Rancho Bernardo, California and I-15",
        "x_coords": "-117.0832461",
        "y_coords": "32.9978623"
    },
    {
        "origin_intersection": "camino del norte e San Deigo, California and I-15",
        "x_coords": "-117.0832798",
        "y_coords": "32.9977763"
    },
    {
        "origin_intersection": "camino del norte e San Diego, California and I-15",
        "x_coords": "-117.0832704",
        "y_coords": "32.9978246"
    },
    {
        "origin_intersection": "camino del norte w San Deigo, California and I-15",
        "x_coords": "-117.0832377",
        "y_coords": "32.9979181"
    },
    {
        "origin_intersection": "camino del norte w San Diego, California and I-15",
        "x_coords": "-117.0832784",
        "y_coords": "32.9978044"
    },
    {
        "origin_intersection": "camino del rio way San Diego, California and I-8",
        "x_coords": "-117.1329791",
        "y_coords": "32.7731875"
    },
    {
        "origin_intersection": "camino del rio west e San Diego, California and I-5",
        "x_coords": "-117.2036847",
        "y_coords": "32.7582733"
    },
    {
        "origin_intersection": "camino viejo rd Montecito, California and SR-192",
        "x_coords": "-119.6459656",
        "y_coords": "34.4337125"
    },
    {
        "origin_intersection": "camp roberts main gate e Camp Roberts, California and US-101",
        "x_coords": "-120.7427329",
        "y_coords": "35.7992084"
    },
    {
        "origin_intersection": "camp roberts main gate e San Miguel, California and US-101",
        "x_coords": "-120.7427034",
        "y_coords": "35.7992301"
    },
    {
        "origin_intersection": "campus ave 145 N 8 Ave, Upland, Ca 91786, California and SR-210",
        "x_coords": "-117.6357005",
        "y_coords": "34.1348342"
    },
    {
        "origin_intersection": "capitol ave San Jose, California and I-680",
        "x_coords": "-121.8757076",
        "y_coords": "37.3992302"
    },
    {
        "origin_intersection": "capitol expressway e San Jose, California and US-101",
        "x_coords": "-121.816731",
        "y_coords": "37.3031229"
    },
    {
        "origin_intersection": "caribbean dr Sunnyvale, California and SR-237",
        "x_coords": "-121.9986754",
        "y_coords": "37.4091555"
    },
    {
        "origin_intersection": "carpenter rd Ceres, California and SR-132",
        "x_coords": "-121.0304216",
        "y_coords": "37.6386838"
    },
    {
        "origin_intersection": "carson st Long Beach, California and I-605",
        "x_coords": "-118.0840924",
        "y_coords": "33.8314182"
    },
    {
        "origin_intersection": "carson st Torrance, Ca, California and SR-110",
        "x_coords": "-118.2873768",
        "y_coords": "33.8316061"
    },
    {
        "origin_intersection": "caspian ave 1500 W 16Th St., Long Beach Ca, California and SR-1",
        "x_coords": "-118.212178",
        "y_coords": "33.7898694"
    },
    {
        "origin_intersection": "caterpillar rd Redding, California and SR-273",
        "x_coords": "-122.3709119",
        "y_coords": "40.625427"
    },
    {
        "origin_intersection": "cattleman rd San Ardo, California and US-101",
        "x_coords": "-120.9222551",
        "y_coords": "36.0129714"
    },
    {
        "origin_intersection": "central ave Compton, California and SR-91",
        "x_coords": "-118.2487037",
        "y_coords": "33.8733992"
    },
    {
        "origin_intersection": "central ave Lake Elsinore, Ca., California and SR-74",
        "x_coords": "-117.3339359",
        "y_coords": "33.6953794"
    },
    {
        "origin_intersection": "central ave Lake Elsinore//18759 Collier Ave (Their Yard), California and SR-74",
        "x_coords": "-117.333915",
        "y_coords": "33.6953924"
    },
    {
        "origin_intersection": "central ave Lompoc, California and SR-1",
        "x_coords": "-120.4576499",
        "y_coords": "34.6607815"
    },
    {
        "origin_intersection": "central ave Lynwood, California and I-105",
        "x_coords": "-118.2542859",
        "y_coords": "33.9280816"
    },
    {
        "origin_intersection": "central ave Montclair, California and SR-60",
        "x_coords": "-117.6894456",
        "y_coords": "34.0302475"
    },
    {
        "origin_intersection": "central ave Sanger, California and SR-99",
        "x_coords": "-119.7414454",
        "y_coords": "36.6779128"
    },
    {
        "origin_intersection": "centre city pkwy Escondido, California and I-15",
        "x_coords": "-117.1024826",
        "y_coords": "33.1587803"
    },
    {
        "origin_intersection": "chadbourne rd Fairfield, California and I-80",
        "x_coords": "-122.0867809",
        "y_coords": "38.2423706"
    },
    {
        "origin_intersection": "chamberlain way Adelanto, California and US-395",
        "x_coords": "-117.4145304",
        "y_coords": "34.5864152"
    },
    {
        "origin_intersection": "charles howard park Ione, California and SR-124",
        "x_coords": "-120.9331742",
        "y_coords": "38.3458936"
    },
    {
        "origin_intersection": "charter way 4777 Carpenter Rd, Stockton, California and I-5",
        "x_coords": "-121.1979358",
        "y_coords": "37.9302341"
    },
    {
        "origin_intersection": "charter way Stockton, California and I-5",
        "x_coords": "-121.297544",
        "y_coords": "37.9367855"
    },
    {
        "origin_intersection": "cherry acres rd Georgetown, California and SR-193",
        "x_coords": "-121.0063223",
        "y_coords": "38.8881278"
    },
    {
        "origin_intersection": "cherry ave Fontana, California and I-10",
        "x_coords": "-117.4889708",
        "y_coords": "34.0658013"
    },
    {
        "origin_intersection": "cherry ave Irvine, California and I-10",
        "x_coords": "-117.4889489",
        "y_coords": "34.0658824"
    },
    {
        "origin_intersection": "cherry ave Long Beach, California and I-405",
        "x_coords": "-118.1676224",
        "y_coords": "33.8135054"
    },
    {
        "origin_intersection": "cherry ave Paramount, California and SR-91",
        "x_coords": "-118.1686745",
        "y_coords": "33.8760479"
    },
    {
        "origin_intersection": "chestnut ave 4752 E Carmen Ave, Fresno, Ca 93703, California and SR-180",
        "x_coords": "-119.7362727",
        "y_coords": "36.7547377"
    },
    {
        "origin_intersection": "chestnut ave Fresno, California and SR-99",
        "x_coords": "-119.7367457",
        "y_coords": "36.6740391"
    },
    {
        "origin_intersection": "chilco st Menlo Park, California and SR-84",
        "x_coords": "-122.1671504",
        "y_coords": "37.4831967"
    },
    {
        "origin_intersection": "childs ave Planada, California and SR-99",
        "x_coords": "-120.4565955",
        "y_coords": "37.2884364"
    },
    {
        "origin_intersection": "china lake blvd China Lake, California and SR-178",
        "x_coords": "-117.6699097",
        "y_coords": "35.6224975"
    },
    {
        "origin_intersection": "chiquito canyon rd Castaic, California and SR-126",
        "x_coords": "-118.6567713",
        "y_coords": "34.4197776"
    },
    {
        "origin_intersection": "churchill downs ave Woodland, California and SR-113",
        "x_coords": "-121.7652055",
        "y_coords": "38.6988228"
    },
    {
        "origin_intersection": "citrus ave Fontana, California and I-10",
        "x_coords": "-117.4536487",
        "y_coords": "34.0669208"
    },
    {
        "origin_intersection": "clawiter rd Hayward, California and SR-92",
        "x_coords": "-122.1209303",
        "y_coords": "37.628221"
    },
    {
        "origin_intersection": "clear creek rd Redding, California and SR-273",
        "x_coords": "-122.3813824",
        "y_coords": "40.5167013"
    },
    {
        "origin_intersection": "cleveland ave Madera, California and SR-99",
        "x_coords": "-120.0763285",
        "y_coords": "36.9746009"
    },
    {
        "origin_intersection": "clinton ave Fresno, California and SR-99",
        "x_coords": "-119.8387299",
        "y_coords": "36.7718895"
    },
    {
        "origin_intersection": "clinton keith rd Menifee, California and I-15",
        "x_coords": "-117.2450365",
        "y_coords": "33.595753"
    },
    {
        "origin_intersection": "clinton keith rd Murrieta, California and I-15",
        "x_coords": "-117.2449998",
        "y_coords": "33.5958026"
    },
    {
        "origin_intersection": "clinton keith rd Wildomar, California and I-15",
        "x_coords": "-117.2449629",
        "y_coords": "33.5958159"
    },
    {
        "origin_intersection": "cluff ave Lodi, California and SR-12",
        "x_coords": "-121.2525771",
        "y_coords": "38.1376264"
    },
    {
        "origin_intersection": "cochrane rd Morgan Hill, California and US-101",
        "x_coords": "-121.6525358",
        "y_coords": "37.152952"
    },
    {
        "origin_intersection": "coffee rd Bakersfield, California and SR-58",
        "x_coords": "-119.0920771",
        "y_coords": "35.3669728"
    },
    {
        "origin_intersection": "cold springs rd Santa Barbara, California and SR-192",
        "x_coords": "-119.6592604",
        "y_coords": "34.4406779"
    },
    
    
    {
        "origin_intersection": "cole rd 2270 Portico Blvd, Calexico Ca,92231, California and SR-111",
        "x_coords": "-115.4992758",
        "y_coords": "32.6939774"
    },
    {
        "origin_intersection": "college ave San Diego, California and I-8",
        "x_coords": "-117.0663648",
        "y_coords": "32.7794056"
    },
    {
        "origin_intersection": "columbia ave 850 Marlborough Ave, Riverside, Ca, California and I-215",
        "x_coords": "-117.3519576",
        "y_coords": "34.001321"
    },
    {
        "origin_intersection": "comanche dr 11412 Malaga Road, Bakersfield, Ca, California and SR-58",
        "x_coords": "-118.8432297",
        "y_coords": "35.3383223"
    },
    {
        "origin_intersection": "comconex rd Manteca, California and SR-120",
        "x_coords": "-121.1682347",
        "y_coords": "37.7975491"
    },
    {
        "origin_intersection": "commercial vehicle border entrance Otay Mesa / Mex Border, California and SR-905",
        "x_coords": "-116.9390526",
        "y_coords": "32.5517706"
    },
    {
        "origin_intersection": "commercial vehicle border entrance Otay Mesa, California and SR-905",
        "x_coords": "-116.939074",
        "y_coords": "32.5518611"
    },
    {
        "origin_intersection": "commercial vehicle border entrance Us/Mex Border At Hwy 905, California and SR-905",
        "x_coords": "-116.939074",
        "y_coords": "32.5517887"
    },
    {
        "origin_intersection": "concannon blvd Livermore, California and SR-84",
        "x_coords": "-121.8055097",
        "y_coords": "37.6608752"
    },
    {
        "origin_intersection": "concord ave Concord, California and SR-242",
        "x_coords": "-122.0433329",
        "y_coords": "37.9790071"
    },
    {
        "origin_intersection": "convoy st Miramar, California and SR-52",
        "x_coords": "-117.1521141",
        "y_coords": "32.8377258"
    },
    {
        "origin_intersection": "cook st Indian Wells, California and I-10",
        "x_coords": "-116.3546927",
        "y_coords": "33.7860299"
    },
    {
        "origin_intersection": "copus rd Bakersfield, California and SR-99",
        "x_coords": "-118.9795056",
        "y_coords": "35.0945112"
    },
    {
        "origin_intersection": "cornelia ave Fresno, California and SR-180",
        "x_coords": "-119.8807549",
        "y_coords": "36.7354706"
    },
    {
        "origin_intersection": "corral hollow rd Tracy, California and I-580",
        "x_coords": "-121.4572418",
        "y_coords": "37.6722352"
    },
    {
        "origin_intersection": "cosumnes river blvd Sacramento, California and I-5",
        "x_coords": "-121.4970381",
        "y_coords": "38.4665695"
    },
    {
        "origin_intersection": "cotner ave Los Angeles, California and SR-2",
        "x_coords": "-118.4460927",
        "y_coords": "34.0471089"
    },
    {
        "origin_intersection": "cottonwood st Burney, California and SR-299",
        "x_coords": "-121.626168",
        "y_coords": "40.9186845"
    },
    {
        "origin_intersection": "country village rd 13470 Philadelphia Ave Fontana, Ca 92337, California and SR-60",
        "x_coords": "-117.5066704",
        "y_coords": "34.0187786"
    },
    {
        "origin_intersection": "country village rd Fontana, California and SR-60",
        "x_coords": "-117.5066959",
        "y_coords": "34.0188109"
    },
    {
        "origin_intersection": "covell blvd Woodland, California and SR-113",
        "x_coords": "-121.7678071",
        "y_coords": "38.5608649"
    },
    {
        "origin_intersection": "crenshaw blvd 1 Rocket Rd Hawthorne Ca, California and I-105",
        "x_coords": "-118.3265032",
        "y_coords": "33.9252328"
    },
    {
        "origin_intersection": "crenshaw blvd Hawthorne, California and I-105",
        "x_coords": "-118.3265221",
        "y_coords": "33.9252017"
    },
    {
        "origin_intersection": "crenshaw blvd Inglewood, California and I-105",
        "x_coords": "-118.3264711",
        "y_coords": "33.9252084"
    },
    {
        "origin_intersection": "crenshaw blvd Space X (Hawthorne), California and I-105",
        "x_coords": "-118.3264996",
        "y_coords": "33.925163"
    },
    {
        "origin_intersection": "crucero rd Amboy, California and I-40",
        "x_coords": "-115.7561961",
        "y_coords": "34.5554313"
    },
    {
        "origin_intersection": "culver blvd Playa Del Rey, California and SR-90",
        "x_coords": "-118.4284592",
        "y_coords": "33.9815233"
    },
    {
        "origin_intersection": "cushing pkwy Fremont, California and I-880",
        "x_coords": "-121.9486408",
        "y_coords": "37.4905609"
    },
    {
        "origin_intersection": "dale evans parkway Apple Valley, California and I-15",
        "x_coords": "-117.2179753",
        "y_coords": "34.6442563"
    },
    {
        "origin_intersection": "day creek blvd Rancho Cucamonga, California and SR-210",
        "x_coords": "-117.5367542",
        "y_coords": "34.1364661"
    },
    {
        "origin_intersection": "deer creek rd Malibu, California and SR-1",
        "x_coords": "-118.9855282",
        "y_coords": "34.0621798"
    },
    {
        "origin_intersection": "del amo blvd Long Beach, California and I-710",
        "x_coords": "-118.2055055",
        "y_coords": "33.8466178"
    },
    {
        "origin_intersection": "del amo blvd Rancho Dominguez, California and I-710",
        "x_coords": "-118.205491",
        "y_coords": "33.846629"
    },
    {
        "origin_intersection": "del amo blvd Torrance, Ca, California and SR-107",
        "x_coords": "-118.3537604",
        "y_coords": "33.8485862"
    },
    {
        "origin_intersection": "del mar heights rd Del Mar, California and I-5",
        "x_coords": "-117.2437013",
        "y_coords": "32.9499944"
    },
    {
        "origin_intersection": "del norte blvd Oxnard, California and US-101",
        "x_coords": "-119.1268334",
        "y_coords": "34.2220852"
    },
    {
        "origin_intersection": "deschutes rd Palo Cedro, California and SR-44",
        "x_coords": "-122.2384558",
        "y_coords": "40.5572782"
    },
    {
        "origin_intersection": "devore rd San Bernardino, California and I-215",
        "x_coords": "-117.4026201",
        "y_coords": "34.2223796"
    },
    {
        "origin_intersection": "diamond bar blvd Diamond Bar, California and SR-60",
        "x_coords": "-117.811632",
        "y_coords": "34.0232992"
    },
    {
        "origin_intersection": "dillon rd Niland, California and I-10",
        "x_coords": "-116.1727427",
        "y_coords": "33.7150602"
    },
    {
        "origin_intersection": "dolan rd Moss Landing, California and SR-1",
        "x_coords": "-121.7833393",
        "y_coords": "36.8003919"
    },
    {
        "origin_intersection": "dover dr Newport Beach/2200 West Coast Hwy, California and SR-1",
        "x_coords": "-117.9072028",
        "y_coords": "33.6162325"
    },
    {
        "origin_intersection": "dudley st Pomona, California and I-10",
        "x_coords": "-117.7783432",
        "y_coords": "34.0728838"
    },
    {
        "origin_intersection": "dunaweal ln Calistoga, California and SR-29",
        "x_coords": "-122.5580051",
        "y_coords": "38.5649729"
    },
    {
        "origin_intersection": "durfee ave 3301 Gilman Rd, El Monte, Ca, 91732, California and I-10",
        "x_coords": "-118.011089",
        "y_coords": "34.0644099"
    },
    {
        "origin_intersection": "durham dayton hwy Durham, California and SR-99",
        "x_coords": "-121.7161582",
        "y_coords": "39.6422508"
    },
    {
        "origin_intersection": "dyer rd Irvine, California and SR-55",
        "x_coords": "-117.8517618",
        "y_coords": "33.708568"
    },
    {
        "origin_intersection": "dyer rd Santa Ana, California and SR-55",
        "x_coords": "-117.8517935",
        "y_coords": "33.7085733"
    },
    {
        "origin_intersection": "e 223rd st Carson, California and I-405",
        "x_coords": "-118.2252936",
        "y_coords": "33.8243046"
    },
    {
        "origin_intersection": "e 223rd st Long Beach, California and I-405",
        "x_coords": "-118.2252881",
        "y_coords": "33.8243313"
    },
    {
        "origin_intersection": "e 223rd st Wilmington, California and I-405",
        "x_coords": "-118.2252758",
        "y_coords": "33.8243201"
    },
    {
        "origin_intersection": "e bidwell st Folsom, California and US-50",
        "x_coords": "-121.1149141",
        "y_coords": "38.6432434"
    },
    {
        "origin_intersection": "e orange ave Chula Vista, California and I-805",
        "x_coords": "-117.0363137",
        "y_coords": "32.6058014"
    },
    {
        "origin_intersection": "e orange ave San Diego, California and I-805",
        "x_coords": "-117.0362703",
        "y_coords": "32.6058164"
    },
    {
        "origin_intersection": "east anaheim st Long Beach, Ca, California and SR-1",
        "x_coords": "-118.1326939",
        "y_coords": "33.782692"
    },
    {
        "origin_intersection": "east dunne ave Morgan Hill, California and US-101",
        "x_coords": "-121.6335869",
        "y_coords": "37.1315306"
    },
    {
        "origin_intersection": "east i st Long Beach, California and SR-103",
        "x_coords": "-118.2264892",
        "y_coords": "33.783499"
    },
    {
        "origin_intersection": "east i st Los Angeles, California and SR-103",
        "x_coords": "-118.2264739",
        "y_coords": "33.7834808"
    },
    {
        "origin_intersection": "east st Woodland, California and I-5",
        "x_coords": "-121.7651982",
        "y_coords": "38.6965145"
    },
    {
        "origin_intersection": "east vineyard ave Oxnard, California and SR-232",
        "x_coords": "-119.164082",
        "y_coords": "34.2440809"
    },
    {
        "origin_intersection": "east vineyard ave Port Hueneme, California and SR-232",
        "x_coords": "-119.1640988",
        "y_coords": "34.2440546"
    },
    {
        "origin_intersection": "east vineyard ave Port Huneme, California and SR-232",
        "x_coords": "-119.1640585",
        "y_coords": "34.2441011"
    },
    {
        "origin_intersection": "edison highway Bakersfield, California and SR-184",
        "x_coords": "-118.9141906",
        "y_coords": "35.3582703"
    },
    {
        "origin_intersection": "eight mile rd Stockton, California and SR-88",
        "x_coords": "-121.1887117",
        "y_coords": "38.0579491"
    },
    {
        "origin_intersection": "eight mile rd Stockton, California and SR-99",
        "x_coords": "-121.2593986",
        "y_coords": "38.0569109"
    },
    {
        "origin_intersection": "el dorado st Stockton, California and SR-4",
        "x_coords": "-121.2860513",
        "y_coords": "37.9389336"
    },
    {
        "origin_intersection": "el segundo blvd 324 W El Segundo Blvd, El Segundo, California and SR-1",
        "x_coords": "-118.3959832",
        "y_coords": "33.9163618"
    },
    {
        "origin_intersection": "el segundo blvd El Segundo, California and I-405",
        "x_coords": "-118.3702474",
        "y_coords": "33.9163854"
    },
    {
        "origin_intersection": "el segundo blvd Gardena, California and SR-110",
        "x_coords": "-118.2858589",
        "y_coords": "33.9164193"
    },
    {
        "origin_intersection": "el segundo blvd Space X (Hawthorne), California and SR-110",
        "x_coords": "-118.2858193",
        "y_coords": "33.9164251"
    },
    {
        "origin_intersection": "el toro rd Lake Forest, California and I-5",
        "x_coords": "-117.7076566",
        "y_coords": "33.6147478"
    },
    {
        "origin_intersection": "elm st San Diego, California and I-5",
        "x_coords": "-117.1638653",
        "y_coords": "32.7240504"
    },
    {
        "origin_intersection": "emery rd Tecate, California and SR-94",
        "x_coords": "-116.6387403",
        "y_coords": "32.6017424"
    },
    {
        "origin_intersection": "empire ave Burbank, California and I-5",
        "x_coords": "-118.3273658",
        "y_coords": "34.1917217"
    },
    {
        "origin_intersection": "enrico fermi dr Otay Mesa, California and SR-11",
        "x_coords": "-116.9279141",
        "y_coords": "32.5637075"
    },
    {
        "origin_intersection": "enrico fermi dr San Diego, California and SR-11",
        "x_coords": "-116.9279396",
        "y_coords": "32.5637868"
    },
    {
        "origin_intersection": "enterprise blvd West Sacramento, California and I-80",
        "x_coords": "-121.5773814",
        "y_coords": "38.5742546"
    },
    {
        "origin_intersection": "enterprise blvd West Sacramento, California and I-80",
        "x_coords": "-121.577433",
        "y_coords": "38.574226"
    },
    {
        "origin_intersection": "esplanade Chico, California and SR-99",
        "x_coords": "-121.8987238",
        "y_coords": "39.7932504"
    },
    {
        "origin_intersection": "ethanac rd Perris, California and I-215",
        "x_coords": "-117.189457",
        "y_coords": "33.7428047"
    },
    {
        "origin_intersection": "etiwanda ave 10606 Commerce Way, Fontana Ca 92337, California and I-10",
        "x_coords": "-117.5240979",
        "y_coords": "34.0675402"
    },
    {
        "origin_intersection": "etiwanda ave Fontana, California and I-10",
        "x_coords": "-117.5240951",
        "y_coords": "34.0675097"
    },
    {
        "origin_intersection": "eucalyptus ave Ontario, California and SR-83",
        "x_coords": "-117.6507908",
        "y_coords": "33.9902829"
    },
    {
        "origin_intersection": "eureka rd Roseville, California and I-80",
        "x_coords": "-121.2616087",
        "y_coords": "38.758116"
    },
    {
        "origin_intersection": "fairfax rd Bakersfield, California and SR-58",
        "x_coords": "-118.9319631",
        "y_coords": "35.3522921"
    },
    {
        "origin_intersection": "fairview rd Dunneville, Ca, California and SR-156",
        "x_coords": "-121.3881984",
        "y_coords": "36.9330081"
    },
    {
        "origin_intersection": "fallon rd Dublin, California and I-580",
        "x_coords": "-121.8495148",
        "y_coords": "37.7012329"
    },
    {
        "origin_intersection": "farmington rd Boron, California and US-395",
        "x_coords": "-117.5513935",
        "y_coords": "35.0276363"
    },
    {
        "origin_intersection": "farmington rd Byron, California and US-395",
        "x_coords": "-117.5514573",
        "y_coords": "35.0276533"
    },
    {
        "origin_intersection": "ferry st 614 Terminal Way, San Pedro, Ca, California and SR-47",
        "x_coords": "-118.2627421",
        "y_coords": "33.7483255"
    },
    {
        "origin_intersection": "ferry st Long Beach, California and SR-47",
        "x_coords": "-118.2627364",
        "y_coords": "33.7483448"
    },
    {
        "origin_intersection": "ferry st San Pedro, California and SR-47",
        "x_coords": "-118.2627156",
        "y_coords": "33.7483375"
    },
    {
        "origin_intersection": "ferry st Terminal Island, California and SR-47",
        "x_coords": "-118.2627256",
        "y_coords": "33.7483291"
    },
    {
        "origin_intersection": "fiji way Marina Del Rey, California and SR-1",
        "x_coords": "-118.4365758",
        "y_coords": "33.9785202"
    },
    {
        "origin_intersection": "firebaugh ave Exeter, California and SR-65",
        "x_coords": "-119.1361222",
        "y_coords": "36.2898734"
    },
    {
        "origin_intersection": "firestone blvd South Gate, Ca, California and I-710",
        "x_coords": "-118.170415",
        "y_coords": "33.9497344"
    },
    {
        "origin_intersection": "firestone blvd South Gate, California and I-710",
        "x_coords": "-118.1704901",
        "y_coords": "33.9497655"
    },
    {
        "origin_intersection": "first st King City, California and US-101",
        "x_coords": "-121.1125818",
        "y_coords": "36.201312"
    },
    {
        "origin_intersection": "flightpath wy 34948 Gazelle, Bakersfield, California and SR-65",
        "x_coords": "-119.0747082",
        "y_coords": "35.4475398"
    },
    {
        "origin_intersection": "florence ave Bell Gardens, California and I-710",
        "x_coords": "-118.1678138",
        "y_coords": "33.9681684"
    },
    {
        "origin_intersection": "florence ave Commerce, California and I-710",
        "x_coords": "-118.167796",
        "y_coords": "33.9681699"
    },
    {
        "origin_intersection": "florin rd Sacramento, California and I-5",
        "x_coords": "-121.516552",
        "y_coords": "38.4955274"
    },
    {
        "origin_intersection": "foothill blvd Los Angeles, California and SR-210",
        "x_coords": "-118.38077",
        "y_coords": "34.2749634"
    },
    {
        "origin_intersection": "foothill expressway e Gilroy, California and I-280",
        "x_coords": "-122.0684023",
        "y_coords": "37.3343649"
    },
    {
        "origin_intersection": "forest gate rd Campo, California and SR-94",
        "x_coords": "-116.4741423",
        "y_coords": "32.6083469"
    },
    {
        "origin_intersection": "forest st Gilroy, California and SR-152",
        "x_coords": "-121.5717185",
        "y_coords": "37.0204604"
    },
    {
        "origin_intersection": "fourth st Fontana, California and I-15",
        "x_coords": "-117.5448535",
        "y_coords": "34.0772223"
    },
    {
        "origin_intersection": "fowler ave Clovis, California and SR-180",
        "x_coords": "-119.68196",
        "y_coords": "36.7530438"
    },
    {
        "origin_intersection": "fred d haight rd Crescent City, California and US-101",
        "x_coords": "-124.1460153",
        "y_coords": "41.9299906"
    },
    {
        "origin_intersection": "fred d haight rd Smith River, California and US-101",
        "x_coords": "-124.1459645",
        "y_coords": "41.9299917"
    },
    {
        "origin_intersection": "fremont blvd Fremont, California and I-880",
        "x_coords": "-121.9486033",
        "y_coords": "37.4926678"
    },
    {
        "origin_intersection": "french camp rd Stockton, California and I-5",
        "x_coords": "-121.2838361",
        "y_coords": "37.8970728"
    },
    {
        "origin_intersection": "fresno ave Stockton, California and SR-4",
        "x_coords": "-121.3121237",
        "y_coords": "37.9328874"
    },
    {
        "origin_intersection": "ft irwin rd Ft Irwin, California and I-15",
        "x_coords": "-116.9262171",
        "y_coords": "34.9017105"
    },
    {
        "origin_intersection": "gaffey st San Pedro, Ca, California and SR-110",
        "x_coords": "-118.292322",
        "y_coords": "33.746302"
    },
    {
        "origin_intersection": "garden grove blvd Garden Grove, California and SR-39",
        "x_coords": "-117.9927875",
        "y_coords": "33.7737747"
    },
    {
        "origin_intersection": "garden grove blvd Santa Ana, California and SR-22",
        "x_coords": "-117.9048509",
        "y_coords": "33.7744299"
    },
    {
        "origin_intersection": "garvey ave El Monte, California and I-10",
        "x_coords": "-118.0087404",
        "y_coords": "34.0640098"
    },
    {
        "origin_intersection": "gas point rd Anderson, California and I-5",
        "x_coords": "-122.2852483",
        "y_coords": "40.3893153"
    },
    {
        "origin_intersection": "gateway dr 15341 Road 28 1/2 Madera Ca, California and SR-145",
        "x_coords": "-120.0605321",
        "y_coords": "36.9612309"
    },
    {
        "origin_intersection": "genesee ave San Diego, California and I-5",
        "x_coords": "-117.2277622",
        "y_coords": "32.8871991"
    },
    {
        "origin_intersection": "georgia pacific way Oroville, California and SR-70",
        "x_coords": "-121.575675",
        "y_coords": "39.4773978"
    },
    {
        "origin_intersection": "gephart rd Boron, California and SR-58",
        "x_coords": "-117.73917",
        "y_coords": "35.0068083"
    },
    {
        "origin_intersection": "gish rd San Jose @ 1370 Vander Way, California and I-880",
        "x_coords": "-121.8996812",
        "y_coords": "37.367993"
    },
    {
        "origin_intersection": "glenoaks blvd Pacoima, California and SR-118",
        "x_coords": "-118.4187249",
        "y_coords": "34.2803352"
    },
    {
        "origin_intersection": "glenwood ave Turlock, California and SR-165",
        "x_coords": "-120.8490038",
        "y_coords": "37.4755642"
    },
    {
        "origin_intersection": "goffs rd Fenner, California and I-40",
        "x_coords": "-115.1902987",
        "y_coords": "34.8121345"
    },
    {
        "origin_intersection": "goldlake rd Bassets, California and SR-49",
        "x_coords": "-120.5913583",
        "y_coords": "39.6170808"
    },
    {
        "origin_intersection": "golf center pkwy Indio, California and I-10",
        "x_coords": "-116.2015041",
        "y_coords": "33.7260409"
    },
    {
        "origin_intersection": "golf center pkwy La Quinta, California and I-10",
        "x_coords": "-116.2014726",
        "y_coords": "33.7260697"
    },
    {
        "origin_intersection": "golf course dr Rohnert Park, California and US-101",
        "x_coords": "-122.7127461",
        "y_coords": "38.3632196"
    },
    {
        "origin_intersection": "gonzaga rd 31426 Gonzaga Rd, Gustine, Ca, 95322, California and SR-33",
        "x_coords": "-121.0162911",
        "y_coords": "37.0562603"
    },
    {
        "origin_intersection": "goose valley rd Burney, California and SR-299",
        "x_coords": "-121.6945159",
        "y_coords": "40.8724826"
    },
    {
        "origin_intersection": "gopher canyon rd Escondido, California and I-15",
        "x_coords": "-117.1559222",
        "y_coords": "33.2528842"
    },
    {
        "origin_intersection": "graham hill rd Felton, California and SR-9",
        "x_coords": "-122.0732654",
        "y_coords": "37.0530334"
    },
    {
        "origin_intersection": "grand ave Chino, California and SR-71",
        "x_coords": "-117.7238956",
        "y_coords": "34.0006841"
    },
    {
        "origin_intersection": "grand ave Santa Ana, California and I-5",
        "x_coords": "-117.8518881",
        "y_coords": "33.7548908"
    },
    {
        "origin_intersection": "grangeville blvd Hanford, California and SR-43",
        "x_coords": "-119.6009522",
        "y_coords": "36.3425023"
    },
    {
        "origin_intersection": "granite creek rd Santa Cruz, California and SR-17",
        "x_coords": "-122.0045804",
        "y_coords": "37.0611504"
    },
    {
        "origin_intersection": "grant line Elk Grove, California and SR-99",
        "x_coords": "-121.3620431",
        "y_coords": "38.376388"
    },
    {
        "origin_intersection": "grant line Slough House, California and SR-16",
        "x_coords": "-121.2233552",
        "y_coords": "38.5019262"
    },
    {
        "origin_intersection": "guernsey ave Malibu, California and SR-1",
        "x_coords": "-118.8392761",
        "y_coords": "34.0281237"
    },    
    {
        "origin_intersection": "hallock dr Santa Paula, California and SR-126",
        "x_coords": "-119.0414922",
        "y_coords": "34.3601798"
    },
    {
        "origin_intersection": "hamilton ave San Jose, California and SR-17",
        "x_coords": "-121.9383855",
        "y_coords": "37.2943275"
    },
    {
        "origin_intersection": "hammer ln Stockton, California and I-5",
        "x_coords": "-121.3588483",
        "y_coords": "38.0213285"
    },
    {
        "origin_intersection": "hammer ln Stockton, California and SR-99",
        "x_coords": "-121.2582073",
        "y_coords": "38.0211699"
    },
    {
        "origin_intersection": "hanford armona rd Hanford, California and SR-43",
        "x_coords": "-119.6007975",
        "y_coords": "36.3134898"
    },
    {
        "origin_intersection": "harbor ave Long Beach, California and SR-1",
        "x_coords": "-118.2105041",
        "y_coords": "33.7898182"
    },
    {
        "origin_intersection": "harbor ave Wilmington, California and SR-1",
        "x_coords": "-118.2820907",
        "y_coords": "33.7905448"
    },
    {
        "origin_intersection": "harbor blvd Anaheim, California and SR-91",
        "x_coords": "-117.9242661",
        "y_coords": "33.8538604"
    },
    {
        "origin_intersection": "harbor blvd Santa Ana, California and SR-22",
        "x_coords": "-117.9203848",
        "y_coords": "33.7677492"
    },
    {
        "origin_intersection": "harbor blvd West Sacramento, California and US-50",
        "x_coords": "-121.548666",
        "y_coords": "38.575016"
    },
    {
        "origin_intersection": "harbor blvd West Sacremento, California and US-50",
        "x_coords": "-121.5486808",
        "y_coords": "38.5750369"
    },
    {
        "origin_intersection": "harbor blvd Wilmington, California and SR-47",
        "x_coords": "-118.2810472",
        "y_coords": "33.7495371"
    },
    {
        "origin_intersection": "harbor dr Oceanside, California and I-5",
        "x_coords": "-117.387299",
        "y_coords": "33.2104665"
    },
    {
        "origin_intersection": "harbor scenic dr 1525 Pier F Ave, Long Beach, Ca 90802, California and I-710",
        "x_coords": "-118.2072704",
        "y_coords": "33.765434"
    },
    {
        "origin_intersection": "harbor scenic dr 50 W Pier D St, Long Beach, Ca 90802, California and I-710",
        "x_coords": "-118.2072788",
        "y_coords": "33.7654289"
    },
    {
        "origin_intersection": "harbor scenic dr Long Beach, California and I-710",
        "x_coords": "-118.2072891",
        "y_coords": "33.7654643"
    },
    {
        "origin_intersection": "hargrave st Banning, California and I-10",
        "x_coords": "-116.8679559",
        "y_coords": "33.9242618"
    },
    {
        "origin_intersection": "hargrave st Indio, California and I-10",
        "x_coords": "-116.2014566",
        "y_coords": "33.7259527"
    },
    {
        "origin_intersection": "harkins slough rd 200 Westridge Dr, Watsonville, Ca, 95076, California and SR-1",
        "x_coords": "-121.7869164",
        "y_coords": "36.9143895"
    },
    {
        "origin_intersection": "harley knox blvd Perris, California and I-215",
        "x_coords": "-117.2596682",
        "y_coords": "33.8625315"
    },
    {
        "origin_intersection": "harris spring rd Bartel, California and SR-89",
        "x_coords": "-121.8240751",
        "y_coords": "41.2526815"
    },
    {
        "origin_intersection": "harrison st San Francisco, California and I-80",
        "x_coords": "-122.394108",
        "y_coords": "37.7851782"
    },
    {
        "origin_intersection": "harry bridges blvd 100 S Broad Ave, Wilimington, Ca, California and SR-110",
        "x_coords": "-118.2799581",
        "y_coords": "33.7718881"
    },
    {
        "origin_intersection": "harry bridges blvd Los Angeles City, California and SR-110",
        "x_coords": "-118.2799313",
        "y_coords": "33.7718786"
    },
    {
        "origin_intersection": "harry bridges blvd Wilmington, California and SR-110",
        "x_coords": "-118.2798941",
        "y_coords": "33.771873"
    },
    {
        "origin_intersection": "haugen Cabazon, Ca, California and lehmann way",
        "x_coords": "-116.689487",
        "y_coords": "33.923735"
    },
    {
        "origin_intersection": "haugen White Water, California and lehmann way",
        "x_coords": "-116.689487",
        "y_coords": "33.9237528"
    },
    {
        "origin_intersection": "haugen Whitewater, Ca, California and lehmann way",
        "x_coords": "-116.6894763",
        "y_coords": "33.9237439"
    },
    {
        "origin_intersection": "haven ave Ontario, California and SR-60",
        "x_coords": "-117.5757365",
        "y_coords": "34.0289473"
    },
    {
        "origin_intersection": "hazel ave Sacramento, California and US-50",
        "x_coords": "-121.217062",
        "y_coords": "38.6308408"
    },
    {
        "origin_intersection": "heil ave Huntington Beach, California and SR-39",
        "x_coords": "-117.9892078",
        "y_coords": "33.7229396"
    },
    {
        "origin_intersection": "henderson ave 770 N Plano St, Porterville, Ca, 93257, California and SR-65",
        "x_coords": "-119.0393052",
        "y_coords": "36.0804544"
    },
    {
        "origin_intersection": "henry station rd Ukiah, California and US-101",
        "x_coords": "-123.150448",
        "y_coords": "39.0444782"
    },
    {
        "origin_intersection": "hesperian blvd Hayward, California and SR-92",
        "x_coords": "-122.0999744",
        "y_coords": "37.6362844"
    },
    {
        "origin_intersection": "hidden valley pkwy Corona, California and I-15",
        "x_coords": "-117.5598826",
        "y_coords": "33.8987324"
    },
    {
        "origin_intersection": "highland springs ave Banning, California and I-10",
        "x_coords": "-116.9468823",
        "y_coords": "33.9268786"
    },
    {
        "origin_intersection": "holsclaw rd Gilroy (1590 Pacheco Pass Hwy), California and SR-152",
        "x_coords": "-121.5319341",
        "y_coords": "37.0018669"
    },
    {
        "origin_intersection": "hooker creek rd Cottonwood, California and I-5",
        "x_coords": "-122.2789809",
        "y_coords": "40.2930224"
    },
    {
        "origin_intersection": "humphrey rd Sutter, California and SR-20",
        "x_coords": "-121.7261331",
        "y_coords": "39.1418871"
    },
    {
        "origin_intersection": "huntington dr Arcadia, California and SR-210",
        "x_coords": "-118.0165691",
        "y_coords": "34.1402038"
    },
    {
        "origin_intersection": "husted rd Williams, California and I-5",
        "x_coords": "-122.1328074",
        "y_coords": "39.128814"
    },
    {
        "origin_intersection": "I-10 and Arizona Line",
        "x_coords": "-114.5309445",
        "y_coords": "33.6047557"
    },
    {
        "origin_intersection": "I-5 and Oregon Line",
        "x_coords": "-122.6154688",
        "y_coords": "42.0053774"
    },
    {
        "origin_intersection": "I-80 and Nevada Line",
        "x_coords": "-120.0029073",
        "y_coords": "39.4681518"
    },
    {
        "origin_intersection": "ice house rd Pollock Pines, California and US-50",
        "x_coords": "-120.447524",
        "y_coords": "38.7692568"
    },
    {
        "origin_intersection": "imjin parkway Marina, California and SR-1",
        "x_coords": "-121.8130107",
        "y_coords": "36.6694461"
    },
    {
        "origin_intersection": "indian canyon dr N Palm Springs, California and I-10",
        "x_coords": "-116.5454081",
        "y_coords": "33.9046795"
    },
    {
        "origin_intersection": "indian truck trail w Corona, California and Interstate-15",
        "x_coords": "-117.4524405",
        "y_coords": "33.7487194"
    },
    {
        "origin_intersection": "industrial parkway Hayward, California and I-880",
        "x_coords": "-122.0743806",
        "y_coords": "37.6175391"
    },
    {
        "origin_intersection": "industrial way 401 West Channel Road, Benicia, Ca, California and I-680",
        "x_coords": "-122.1280441",
        "y_coords": "38.0680464"
    },
    {
        "origin_intersection": "industrial way Dixon, California and SR-113",
        "x_coords": "-121.8224247",
        "y_coords": "38.4589893"
    },
    {
        "origin_intersection": "ingram creek rd Westley, California and I-5",
        "x_coords": "-121.2676778",
        "y_coords": "37.5400667"
    },
    {
        "origin_intersection": "inland dr Stockton, California and SR-4",
        "x_coords": "-121.3970174",
        "y_coords": "37.9268939"
    },
    {
        "origin_intersection": "intake blvd 8580 20Th Ave E, Blythe, Ca, California and I-10",
        "x_coords": "-114.5707078",
        "y_coords": "33.607429"
    },
    {
        "origin_intersection": "iowa ave Grand Terrace, California and I-215",
        "x_coords": "-117.3353761",
        "y_coords": "34.0239462"
    },
    {
        "origin_intersection": "irwin st San Rafael, California and US-101",
        "x_coords": "-122.5208091",
        "y_coords": "37.9717083"
    },
    {
        "origin_intersection": "irwindale ave Azusa, California and SR-210",
        "x_coords": "-117.9336401",
        "y_coords": "34.1297842"
    },
    {
        "origin_intersection": "irwindale ave Irwindale, California and SR-210",
        "x_coords": "-117.933656",
        "y_coords": "34.1298128"
    },
    {
        "origin_intersection": "jack tone road Manteca, California and SR-120",
        "x_coords": "-121.1431748",
        "y_coords": "37.7976494"
    },
    {
        "origin_intersection": "jamboree rd Irvine, California and I-405",
        "x_coords": "-117.8435888",
        "y_coords": "33.676787"
    },
    {
        "origin_intersection": "jamboree rd Newport Beach, California and SR-1",
        "x_coords": "-117.8906674",
        "y_coords": "33.6127508"
    },
    {
        "origin_intersection": "james rd Bakersfield, California and SR-65",
        "x_coords": "-119.0746082",
        "y_coords": "35.4739056"
    },
    {
        "origin_intersection": "jaye st Porterville, California and SR-190",
        "x_coords": "-119.0262713",
        "y_coords": "36.0515475"
    },
    {
        "origin_intersection": "jensen ave Fresno, California and SR-41",
        "x_coords": "-119.7849696",
        "y_coords": "36.7068539"
    },
    {
        "origin_intersection": "jensen ave Fresno, California and SR-99",
        "x_coords": "-119.7740558",
        "y_coords": "36.7068583"
    },
    {
        "origin_intersection": "jensen rd Watsonville, California and SR-1",
        "x_coords": "-121.7711038",
        "y_coords": "36.8538033"
    },
    {
        "origin_intersection": "john s gibson blvd San Pedro, California and SR-110",
        "x_coords": "-118.2897514",
        "y_coords": "33.7564446"
    },
    {
        "origin_intersection": "john tyler dr Malibu, California and SR-1",
        "x_coords": "-118.709939",
        "y_coords": "34.0342392"
    },
    {
        "origin_intersection": "jurupa st 4265 E Guasti Rd, Ontario, Ca, California and I-15",
        "x_coords": "-117.5499979",
        "y_coords": "34.0480703"
    },
    {
        "origin_intersection": "jurupa st Fontana, California and I-15",
        "x_coords": "-117.5499682",
        "y_coords": "34.0480767"
    },
    {
        "origin_intersection": "jurupa st Riverside, California and I-15",
        "x_coords": "-117.54995",
        "y_coords": "34.0480423"
    },
    {
        "origin_intersection": "kaiser rd Desert Center, California and SR-177",
        "x_coords": "-115.3999784",
        "y_coords": "33.7142898"
    },
    {
        "origin_intersection": "kamm ave 3 Rocks, California and SR-33",
        "x_coords": "-120.3890144",
        "y_coords": "36.5305468"
    },
    {
        "origin_intersection": "kanan dume rd Malibu, California and SR-1",
        "x_coords": "-118.8025789",
        "y_coords": "34.0215078"
    },
    {
        "origin_intersection": "kansas ave Hanford, Ca, California and SR-43",
        "x_coords": "-119.6005787",
        "y_coords": "36.2109243"
    },
    {
        "origin_intersection": "kansas ave Hanford, California and SR-43",
        "x_coords": "-119.6005895",
        "y_coords": "36.2109163"
    },
    {
        "origin_intersection": "kansas ave Modesto, Ca, California and SR-99",
        "x_coords": "-121.0164524",
        "y_coords": "37.6459577"
    },
    {
        "origin_intersection": "kansas ave Tulare, California and SR-41",
        "x_coords": "-119.8161211",
        "y_coords": "36.2114873"
    },
    {
        "origin_intersection": "kansas ave Tulare, California and SR-43",
        "x_coords": "-119.6005885",
        "y_coords": "36.2109254"
    },
    {
        "origin_intersection": "kasson rd Tracy, California and I-5",
        "x_coords": "-121.3437159",
        "y_coords": "37.7482243"
    },
    {
        "origin_intersection": "katella ave Anaheim, California and california SR-57",
        "x_coords": "-117.8801778",
        "y_coords": "33.8060883"
    },
    
        {
        "origin_intersection": "katella ave Orange, California and SR-55",
        "x_coords": "-117.8319623",
        "y_coords": "33.8102214"
    },
    {
        "origin_intersection": "katella ave Orange, California and SR-57",
        "x_coords": "-117.8802152",
        "y_coords": "33.806085"
    },
    {
        "origin_intersection": "kearny villa rd San Diego, California and SR-163",
        "x_coords": "-117.1189789",
        "y_coords": "32.8549611"
    },
    {
        "origin_intersection": "kelbaker rd Amboy, California and I-40",
        "x_coords": "-115.6784654",
        "y_coords": "34.720967"
    },
    {
        "origin_intersection": "kent ave Hanford, California and SR-43",
        "x_coords": "-119.6007943",
        "y_coords": "36.2259353"
    },
    {
        "origin_intersection": "kettleman lane 1333 E Kettleman Lane Lodi, Ca, California and SR-99",
        "x_coords": "-121.2578997",
        "y_coords": "38.1162861"
    },
    {
        "origin_intersection": "kimball ave Chino, California and SR-83",
        "x_coords": "-117.6507011",
        "y_coords": "33.9685065"
    },
    {
        "origin_intersection": "knott ave Buena Park, California and SR-91",
        "x_coords": "-118.0112094",
        "y_coords": "33.8559432"
    },
    {
        "origin_intersection": "koster rd Vernalis, California and SR-132",
        "x_coords": "-121.3243486",
        "y_coords": "37.6379886"
    },
    {
        "origin_intersection": "kugelman st Lower Lake, California and SR-53",
        "x_coords": "-122.6122512",
        "y_coords": "38.9140501"
    },
    {
        "origin_intersection": "la cadena dr Colton (2650 S La Cadena Dr), California and I-215",
        "x_coords": "-117.3355855",
        "y_coords": "34.0264105"
    },
    {
        "origin_intersection": "la media rd 8580 Avenida Costa Norte, San Diego, Ca, 9215, California and SR-905",
        "x_coords": "-116.962429",
        "y_coords": "32.5642602"
    },
    {
        "origin_intersection": "la media rd Otay Mesa, California and SR-905",
        "x_coords": "-116.9624209",
        "y_coords": "32.5641918"
    },
    {
        "origin_intersection": "la media rd San Diego, California and SR-905",
        "x_coords": "-116.9624318",
        "y_coords": "32.5641419"
    },
    {
        "origin_intersection": "la sierra ave Riverside, California and SR-91",
        "x_coords": "-117.4734469",
        "y_coords": "33.9000835"
    },
    {
        "origin_intersection": "la tijera blvd Los Angeles, California and I-405",
        "x_coords": "-118.3780261",
        "y_coords": "33.9718346"
    },
    {
        "origin_intersection": "lake forrest dr Lake Forest, California and SR-241",
        "x_coords": "-117.6606537",
        "y_coords": "33.6717515"
    },
    {
        "origin_intersection": "lake mendocino dr Ukiah, California and US-101",
        "x_coords": "-123.2105729",
        "y_coords": "39.1913595"
    },
    {
        "origin_intersection": "lake st Corona, California and I-15",
        "x_coords": "-117.3219074",
        "y_coords": "33.6638013"
    },
    {
        "origin_intersection": "lakewood blvd Downey, California and I-105",
        "x_coords": "-118.1406395",
        "y_coords": "33.9130928"
    },
    {
        "origin_intersection": "lakewood blvd Lakewood, California and I-405",
        "x_coords": "-118.1426148",
        "y_coords": "33.8057665"
    },
    {
        "origin_intersection": "lakewood blvd Long Beach, California and I-405",
        "x_coords": "-118.1426367",
        "y_coords": "33.8057279"
    },
    {
        "origin_intersection": "lakewood blvd Long Beach, California and SR-91",
        "x_coords": "-118.1425911",
        "y_coords": "33.8771215"
    },
    {
        "origin_intersection": "lakewood blvd Paramount, California and I-105",
        "x_coords": "-118.1406635",
        "y_coords": "33.9130823"
    },
    {
        "origin_intersection": "lander ave Turlock, California and SR-99",
        "x_coords": "-120.8491641",
        "y_coords": "37.4735877"
    },
    {
        "origin_intersection": "las posas rd 265 Durley Ave, Camarillo, Ca, California and US-101",
        "x_coords": "-119.0698288",
        "y_coords": "34.2192054"
    },
    {
        "origin_intersection": "las posas rd Camarillo, California and US-101",
        "x_coords": "-119.069803",
        "y_coords": "34.219096"
    },
    {
        "origin_intersection": "las posas rd Oxnard, California and SR-1",
        "x_coords": "-119.0853706",
        "y_coords": "34.1167816"
    },
    {
        "origin_intersection": "las posas rd Point Magu, California and SR-1",
        "x_coords": "-119.0852648",
        "y_coords": "34.1168109"
    },
    {
        "origin_intersection": "lassen st Chatsworth, Ca 91311, California and SR-27",
        "x_coords": "-118.6060628",
        "y_coords": "34.2499545"
    },
    {
        "origin_intersection": "lathrop st Redwood City, California and SR-82",
        "x_coords": "-122.2242724",
        "y_coords": "37.4782834"
    },
    {
        "origin_intersection": "latona rd Anderson, California and SR-273",
        "x_coords": "-122.3463119",
        "y_coords": "40.4811502"
    },
    {
        "origin_intersection": "laval rd Lebec, California and I-5",
        "x_coords": "-118.9452943",
        "y_coords": "34.9863285"
    },
    {
        "origin_intersection": "laval rd Wheeler Ridge, California and I-5",
        "x_coords": "-118.9423592",
        "y_coords": "34.9910838"
    },
    {
        "origin_intersection": "le grand rd Merced, California and SR-99",
        "x_coords": "-120.3750833",
        "y_coords": "37.2256858"
    },
    {
        "origin_intersection": "leavesley rd Gilroy, California and US-101",
        "x_coords": "-121.5666727",
        "y_coords": "37.0223884"
    },
    {
        "origin_intersection": "lee rd 1450 W Beach St, Watsonville, Ca, 95076, California and SR-129",
        "x_coords": "-121.7742239",
        "y_coords": "36.8941705"
    },
    {
        "origin_intersection": "lemon ave Walnut, California and SR-60",
        "x_coords": "-117.85497",
        "y_coords": "33.998771"
    },
    {
        "origin_intersection": "lemoore naval air base Lemoore, California and SR-198",
        "x_coords": "-119.8960744",
        "y_coords": "36.2554875"
    },
    {
        "origin_intersection": "lemoore naval air station Lemoore, California and SR-198",
        "x_coords": "-119.8960831",
        "y_coords": "36.2554978"
    },
    {
        "origin_intersection": "lerdo highway Shafter, California and SR-65",
        "x_coords": "-119.0766122",
        "y_coords": "35.4994389"
    },
    {
        "origin_intersection": "leucadia blvd Carlsbad, California and I-5",
        "x_coords": "-117.2922728",
        "y_coords": "33.0651019"
    },
    {
        "origin_intersection": "lewis rd Port Hueneme, California and SR-34",
        "x_coords": "-119.0345702",
        "y_coords": "34.2054185"
    },
    {
        "origin_intersection": "limonite ave Corona, California and I-15",
        "x_coords": "-117.548846",
        "y_coords": "33.9752044"
    },
    {
        "origin_intersection": "live oak ave Irwindale, California and I-605",
        "x_coords": "-117.9778661",
        "y_coords": "34.1148935"
    },
    {
        "origin_intersection": "live oak canyon rd Yucaipa, California and I-10",
        "x_coords": "-117.0960234",
        "y_coords": "34.0167011"
    },
    {
        "origin_intersection": "lone hill ave Glendora, California and SR-210",
        "x_coords": "-117.8289431",
        "y_coords": "34.120962"
    },
    {
        "origin_intersection": "long beach blvd Coast (Compton), California and I-710",
        "x_coords": "-118.1993128",
        "y_coords": "33.8625548"
    },
    {
        "origin_intersection": "los osos valley rd San Luis Obispo, California and US-101",
        "x_coords": "-120.6822281",
        "y_coords": "35.2446076"
    },
    {
        "origin_intersection": "lost hills rd Los Hills, California and SR-46",
        "x_coords": "-119.6898",
        "y_coords": "35.6160851"
    },
    {
        "origin_intersection": "louise ave Lathrop, California and I-5",
        "x_coords": "-121.2956193",
        "y_coords": "37.8117961"
    },
    {
        "origin_intersection": "loveridge rd Pittsburg, California and SR-4",
        "x_coords": "-121.8671777",
        "y_coords": "38.0118278"
    },
    {
        "origin_intersection": "lower azusa rd Irwindale, California and I-605",
        "x_coords": "-117.9915453",
        "y_coords": "34.0918884"
    },
    {
        "origin_intersection": "lower honcut rd Marysville, California and SR-70",
        "x_coords": "-121.5962101",
        "y_coords": "39.3209181"
    },
    {
        "origin_intersection": "lydick loop Imperial, California and SR-86",
        "x_coords": "-115.5701176",
        "y_coords": "32.8944909"
    },
    {
        "origin_intersection": "macarthur dr Tracy, California and I-205",
        "x_coords": "-121.4147387",
        "y_coords": "37.7642352"
    },
    {
        "origin_intersection": "madera rd Simi Valley, California and SR-118",
        "x_coords": "-118.7928523",
        "y_coords": "34.2839285"
    },
    {
        "origin_intersection": "madison ave Rancho Cordova, California and I-80",
        "x_coords": "-121.3599876",
        "y_coords": "38.6609873"
    },
    {
        "origin_intersection": "madison ave Sacramento, California and I-80",
        "x_coords": "-121.3600374",
        "y_coords": "38.6609808"
    },
    {
        "origin_intersection": "magnolia ave 10611 Prospect Ave, Santee, Ca 92071, California and SR-52",
        "x_coords": "-116.9640515",
        "y_coords": "32.8324568"
    },
    {
        "origin_intersection": "magnolia ave Corona, California and I-15",
        "x_coords": "-117.5453449",
        "y_coords": "33.8743324"
    },
    {
        "origin_intersection": "magnolia ave Long Beach, California and SR-1",
        "x_coords": "-118.1981054",
        "y_coords": "33.7898519"
    },
    {
        "origin_intersection": "magnolia ave Ramona, California and SR-78",
        "x_coords": "-116.8448516",
        "y_coords": "33.051134"
    },
    {
        "origin_intersection": "magnolia ave Santee, California and SR-52",
        "x_coords": "-116.9640668",
        "y_coords": "32.8324524"
    },
    {
        "origin_intersection": "magnolia blvd Los Angeles, California and SR-170",
        "x_coords": "-118.3827007",
        "y_coords": "34.164898"
    },
    {
        "origin_intersection": "magnolia dr Napa, California and SR-221",
        "x_coords": "-122.2738476",
        "y_coords": "38.27833"
    },
    {
        "origin_intersection": "main st Barstow, California and I-40",
        "x_coords": "-117.0029719",
        "y_coords": "34.8934297"
    },
    {
        "origin_intersection": "main st Brawley, California and SR-86",
        "x_coords": "-115.5416411",
        "y_coords": "32.9786112"
    },
    {
        "origin_intersection": "main st Chula Vista, California and I-805",
        "x_coords": "-117.0363062",
        "y_coords": "32.5948855"
    },
    {
        "origin_intersection": "main st Colton (2650 S La Cadena Dr), California and SR-60",
        "x_coords": "-117.3647787",
        "y_coords": "33.996473"
    },
    {
        "origin_intersection": "main st Colton (2650 S La Cadena), California and SR-60",
        "x_coords": "-117.3355362",
        "y_coords": "34.0263972"
    },
    {
        "origin_intersection": "main st Hesperia, California and I-15",
        "x_coords": "-117.3803217",
        "y_coords": "34.4267609"
    },
    {
        "origin_intersection": "main st Ripon, California and SR-99",
        "x_coords": "-121.121221",
        "y_coords": "37.7396973"
    },
    {
        "origin_intersection": "main st Riverside, California and SR-60",
        "x_coords": "-117.3647711",
        "y_coords": "33.9964646"
    },
    {
        "origin_intersection": "malaga rd Arvin, California and SR-223",
        "x_coords": "-118.8601993",
        "y_coords": "35.2089111"
    },
    {
        "origin_intersection": "malaga rd Bakersfield, California and SR-223",
        "x_coords": "-118.8602076",
        "y_coords": "35.2089268"
    },
    {
        "origin_intersection": "malibu canyon rd Malibu, California and SR-1",
        "x_coords": "-118.7033712",
        "y_coords": "34.0347274"
    },
    {
        "origin_intersection": "malibu canyon rd Santa Monica //24255 Pch, California and SR-1",
        "x_coords": "-118.7033568",
        "y_coords": "34.0347562"
    },
    {
        "origin_intersection": "manning ave 30750 Manning Ave, Cantua Creek, Ca, California and SR-33",
        "x_coords": "-120.3885484",
        "y_coords": "36.6033822"
    },
    {
        "origin_intersection": "manning ave Fresno, California and SR-41",
        "x_coords": "-119.7860086",
        "y_coords": "36.6052625"
    },
    {
        "origin_intersection": "manuel campos pkwy Fairfield, California and I-80",
        "x_coords": "-122.0340596",
        "y_coords": "38.2952251"
    },
    {
        "origin_intersection": "maple st Corona, California and SR-91",
        "x_coords": "-117.6036729",
        "y_coords": "33.8801296"
    },
    {
        "origin_intersection": "mapleview st Lakeside (10168 Channel Rd), California and SR-67",
        "x_coords": "-116.9230021",
        "y_coords": "32.8628335"
    },
    {
        "origin_intersection": "mapleview st Lakeside, California and SR-67",
        "x_coords": "-116.922966",
        "y_coords": "32.8628399"
    },
    {
        "origin_intersection": "marengo ave Los Angeles, California and I-10",
        "x_coords": "-118.20264",
        "y_coords": "34.0545416"
    },
    {
        "origin_intersection": "marina blvd San Leandro, California and I-880",
        "x_coords": "-122.1666759",
        "y_coords": "37.7105481"
    },
    {
        "origin_intersection": "mariposa rd Pleasation, California and SR-99",
        "x_coords": "-121.9230351",
        "y_coords": "37.7002758"
    },
    {
        "origin_intersection": "mariposa rd Stockton, California and SR-99",
        "x_coords": "-121.2341646",
        "y_coords": "37.9333815"
    },
    {
        "origin_intersection": "martin rd Westmorland, California and SR-86",
        "x_coords": "-115.6301303",
        "y_coords": "33.0373425"
    },
    {
        "origin_intersection": "masten ave San Martin, California and US-101",
        "x_coords": "-121.5846017",
        "y_coords": "37.0593521"
    },
    {
        "origin_intersection": "mc cabe rd Santa Nella, California and SR-33",
        "x_coords": "-121.0156258",
        "y_coords": "37.1152178"
    },
    {
        "origin_intersection": "mcbean parkway Santa Clarita, California and I-5",
        "x_coords": "-118.5712671",
        "y_coords": "34.3941033"
    },
    {
        "origin_intersection": "mcgowan parkway 3736 Rancho Rd Marysville Ca 95901, California and SR-70",
        "x_coords": "-121.5413611",
        "y_coords": "39.0773001"
    },
    {
        "origin_intersection": "mckinley ave Fresno, California and SR-168",
        "x_coords": "-119.7493253",
        "y_coords": "36.7650946"
    },
    {
        "origin_intersection": "mckinley st Corona, California and SR-91",
        "x_coords": "-117.5178657",
        "y_coords": "33.8863027"
    },
    {
        "origin_intersection": "meadowview rd Sacramento, California and I-5",
        "x_coords": "-121.5105984",
        "y_coords": "38.4811838"
    },
    {
        "origin_intersection": "menlo ave San Jacinto, California and SR-79",
        "x_coords": "-116.9587294",
        "y_coords": "33.7585005"
    },
    {
        "origin_intersection": "merced ave 17047 Zachary Ave, Shafter, Ca, 93263, California and SR-99",
        "x_coords": "-119.1896406",
        "y_coords": "35.5297745"
    },
    {
        "origin_intersection": "merced ave Shafter, California and SR-43",
        "x_coords": "-119.3060549",
        "y_coords": "35.5290319"
    },
    {
        "origin_intersection": "meridian rd Lucerne Valley, California and SR-18",
        "x_coords": "-116.928304",
        "y_coords": "34.4277149"
    },
    {
        "origin_intersection": "merle haggard dr Bakersfield, California and SR-65",
        "x_coords": "-119.084131",
        "y_coords": "35.441415"
    },
    {
        "origin_intersection": "merle haggard dr Bakersfield, California and SR-99",
        "x_coords": "-119.0841116",
        "y_coords": "35.4413919"
    },
    {
        "origin_intersection": "mettler frontage rd Bakersfield, California and SR-99",
        "x_coords": "-118.9746106",
        "y_coords": "35.0761544"
    },
    {
        "origin_intersection": "mexico border Calexico, California and SR-7",
        "x_coords": "-115.3877795",
        "y_coords": "32.6740132"
    },
    {
        "origin_intersection": "mexico border Mex , California and SR-7",
        "x_coords": "-115.3878005",
        "y_coords": "32.6739913"
    },
    {
        "origin_intersection": "mexico border United States / Mexican Border - Calexico, California and SR-7",
        "x_coords": "-115.3878034",
        "y_coords": "32.6740478"
    },
    {
        "origin_intersection": "mexico border Us/Mex Border At Hwy 188, California and SR-188",
        "x_coords": "-116.6274985",
        "y_coords": "32.5763774"
    },
    {
        "origin_intersection": "mexico border Us/Mx Border 7, California and SR-7",
        "x_coords": "-115.3877667",
        "y_coords": "32.6740805"
    },
    {
        "origin_intersection": "michigan bar rd Ione, Ca, California and SR-104",
        "x_coords": "-120.9846814",
        "y_coords": "38.3784832"
    },
    {
        "origin_intersection": "midway rd Vacaville, California and I-80",
        "x_coords": "-121.8973448",
        "y_coords": "38.4162897"
    },
    {
        "origin_intersection": "mira mesa blvd San Diego, California and I-15",
        "x_coords": "-117.11608",
        "y_coords": "32.9170683"
    },
    {
        "origin_intersection": "miramar way 7130 Miramar Rd, San Diego, Ca, California and I-15",
        "x_coords": "-117.1160574",
        "y_coords": "32.9170856"
    },
    {
        "origin_intersection": "mission blvd Mira Loma, California and SR-60",
        "x_coords": "-117.5276532",
        "y_coords": "34.0179692"
    },
    {
        "origin_intersection": "mission blvd Ontario, California and SR-83",
        "x_coords": "-117.6509576",
        "y_coords": "34.0549537"
    },
    {
        "origin_intersection": "mission blvd Pomona, California and SR-71",
        "x_coords": "-117.7812753",
        "y_coords": "34.0534706"
    },
    {
        "origin_intersection": "mission blvd Riverside, California and SR-60",
        "x_coords": "-117.4431734",
        "y_coords": "34.0119082"
    },
    {
        "origin_intersection": "mission center rd San Diego, California and I-8",
        "x_coords": "-117.1536781",
        "y_coords": "32.7654438"
    },
    {
        "origin_intersection": "mission rd Fallbrook, California and SR-13",
        "x_coords": "-117.1746897",
        "y_coords": "33.3871064"
    },
    {
        "origin_intersection": "missouri flat rd Placerville, California and SR-49",
        "x_coords": "-120.8234984",
        "y_coords": "38.6928844"
    },
    {
        "origin_intersection": "mitchell rd Ceres, California and SR-99",
        "x_coords": "-120.9387649",
        "y_coords": "37.5761141"
    },
    {
        "origin_intersection": "mohawk st Bakerfield, California and SR-58",
        "x_coords": "-119.065919",
        "y_coords": "35.3714645"
    },
    {
        "origin_intersection": "mohawk st Bakersfield, California and SR-58",
        "x_coords": "-119.0659511",
        "y_coords": "35.3714996"
    },
    {
        "origin_intersection": "mojave rd Twentynine Palms, California and SR-62",
        "x_coords": "-115.98394",
        "y_coords": "34.1363501"
    },
    {
        "origin_intersection": "mono st Mojave, California and SR-14",
        "x_coords": "-118.1752807",
        "y_coords": "35.0574168"
    },
    {
        "origin_intersection": "monroe st Indio, California and I-10",
        "x_coords": "-116.2338803",
        "y_coords": "33.7394539"
    },
    {
        "origin_intersection": "monte diablo ave Stockton, California and I-5",
        "x_coords": "-121.3294699",
        "y_coords": "37.9571454"
    },
    {
        "origin_intersection": "monte vista ave Upland, California and I-10",
        "x_coords": "-117.6982708",
        "y_coords": "34.0819863"
    },
    {
        "origin_intersection": "monterey st Gilroy, California and US-101",
        "x_coords": "-121.5523757",
        "y_coords": "36.9660838"
    },
    {
        "origin_intersection": "moorpark ave San Jose, California and I-280",
        "x_coords": "-121.9515091",
        "y_coords": "37.3161108"
    },
    {
        "origin_intersection": "mossdale rd Lathrop, California and I-5",
        "x_coords": "-121.3146548",
        "y_coords": "37.7790268"
    },
    {
        "origin_intersection": "mountain ave Upland, California and I-10",
        "x_coords": "-117.6701597",
        "y_coords": "34.0873146"
    },
    {
        "origin_intersection": "mountain ave Upland, California and SR-210",
        "x_coords": "-117.6703825",
        "y_coords": "34.1346591"
    },
    {
        "origin_intersection": "mountain house pkwy Byron, California and I-205",
        "x_coords": "-121.5316411",
        "y_coords": "37.7424816"
    },
    {
        "origin_intersection": "mountain view ave Loma Linda, California and I-10",
        "x_coords": "-117.2436706",
        "y_coords": "34.0666015"
    },
    {
        "origin_intersection": "mountain view ave Redlands, Ca, California and I-10",
        "x_coords": "-117.2436695",
        "y_coords": "34.066591"
    },
    {
        "origin_intersection": "mountain view st Weaverville, California and SR-299",
        "x_coords": "-122.9328565",
        "y_coords": "40.7255188"
    },
    {
        "origin_intersection": "mowry ave Newark, Ca, California and I-880",
        "x_coords": "-122.0022341",
        "y_coords": "37.5297842"
    },
    {
        "origin_intersection": "mowry ave Newark, California and I-880",
        "x_coords": "-122.0022073",
        "y_coords": "37.5297715"
    },
    {
        "origin_intersection": "mt Bakersfield, California and SR-184",
        "x_coords": "-118.9143765",
        "y_coords": "35.2815483"
    },
    {
        "origin_intersection": "mt vernon ave Bakersfield, California and SR-58",
        "x_coords": "-118.9675336",
        "y_coords": "35.3530071"
    },
    {
        "origin_intersection": "mt whitney ave 9790 W Mt Whitney Ave Riverdale,Ca, California and SR-41",
        "x_coords": "-119.8011736",
        "y_coords": "36.4310471"
    },
    {
        "origin_intersection": "mt whitney ave Laton, California and SR-41",
        "x_coords": "-119.801185",
        "y_coords": "36.4310475"
    },
    {
        "origin_intersection": "mt whitney ave Lemoore, California and SR-145",
        "x_coords": "-120.1030683",
        "y_coords": "36.42962"
    },
    {
        "origin_intersection": "murrieta hot sprgs rd Murrieta, California and I-215",
        "x_coords": "-117.1808721",
        "y_coords": "33.5562286"
    },
    {
        "origin_intersection": "n Mare Island, California and SR-37",
        "x_coords": "-122.288406",
        "y_coords": "38.1175187"
    },
    {
        "origin_intersection": "nardo st Ventura, California and SR-118",
        "x_coords": "-119.1476032",
        "y_coords": "34.2820215"
    },
    {
        "origin_intersection": "navy way Long Beach, California and SR-47",
        "x_coords": "-118.2517717",
        "y_coords": "33.7539546"
    },
    {
        "origin_intersection": "navy way Los Angeles, California and SR-47",
        "x_coords": "-118.251753",
        "y_coords": "33.7539273"
    },
    {
        "origin_intersection": "navy way San Pedro, California and SR-47",
        "x_coords": "-118.251791",
        "y_coords": "33.7539799"
    },
    {
        "origin_intersection": "navy way Terminal Island, California and SR-47",
        "x_coords": "-118.2518092",
        "y_coords": "33.7539299"
    },
    {
        "origin_intersection": "new dock st Long Beach, California and SR-47",
        "x_coords": "-118.2389904",
        "y_coords": "33.7611334"
    },
    {
        "origin_intersection": "new dock st San Pedro, California and SR-47",
        "x_coords": "-118.238961",
        "y_coords": "33.7611341"
    },
    {
        "origin_intersection": "newhall ranch rd Santa Clarita, California and I-5",
        "x_coords": "-118.6055243",
        "y_coords": "34.4432429"
    },
    {
        "origin_intersection": "newhall ranch rd Santa Clarita, California and SR-126",
        "x_coords": "-118.6054363",
        "y_coords": "34.443236"
    },
    {
        "origin_intersection": "newport blvd Newport Beach, California and SR-55",
        "x_coords": "-117.9293269",
        "y_coords": "33.6212238"
    },
    {
        "origin_intersection": "nobel dr San Diego, California and I-805",
        "x_coords": "-117.1948666",
        "y_coords": "32.8697125"
    },
    {
        "origin_intersection": "nordahl rd Escondido, California and SR-78",
        "x_coords": "-117.1214182",
        "y_coords": "33.1320916"
    },
    {
        "origin_intersection": "normandie ave Torrance, California and I-405",
        "x_coords": "-118.2992247",
        "y_coords": "33.8598254"
    },
    {
        "origin_intersection": "north 1st st Alviso, California and SR-237",
        "x_coords": "-121.9593857",
        "y_coords": "37.4191897"
    },
    {
        "origin_intersection": "north ave Fresno, California and SR-41",
        "x_coords": "-119.7859356",
        "y_coords": "36.692269"
    },
    {
        "origin_intersection": "north ave Fresno, California and SR-99",
        "x_coords": "-119.7588265",
        "y_coords": "36.692398"
    },
    {
        "origin_intersection": "north grand ave Porterville, California and SR-65",
        "x_coords": "-119.0401084",
        "y_coords": "36.0982515"
    },    
    {
        "origin_intersection": "nutwood ave Placentia, California and SR-57",
        "x_coords": "-117.8803825",
        "y_coords": "33.8781206"
    },
    {
        "origin_intersection": "nv line Az Line, California and Interstate-15",
        "x_coords": "-115.3917269",
        "y_coords": "35.6063444"
    },
    {
        "origin_intersection": "nv line Ca State Line, California and I-15",
        "x_coords": "-115.3915848",
        "y_coords": "35.6063396"
    },
    {
        "origin_intersection": "nv line Ca, California and US-95",
        "x_coords": "-114.8492049",
        "y_coords": "35.1753909"
    },
    {
        "origin_intersection": "nv line I-15/Nv Line, California and I-15",
        "x_coords": "-115.3914511",
        "y_coords": "35.6067516"
    },
    {
        "origin_intersection": "nv line Nevada @ Hwy 15, California and I-15",
        "x_coords": "-115.3915498",
        "y_coords": "35.6065427"
    },
    {
        "origin_intersection": "nv line Nevada Line 15, California and I-15",
        "x_coords": "-115.3915418",
        "y_coords": "35.6063937"
    },
    {
        "origin_intersection": "nv line Nevada Line, California and I-15",
        "x_coords": "-115.3916165",
        "y_coords": "35.6064302"
    },
    {
        "origin_intersection": "nv line Nevada State Line, California and I-15",
        "x_coords": "-115.3915464",
        "y_coords": "35.6066874"
    },
    {
        "origin_intersection": "nv line Nevada State Line, California and US-95",
        "x_coords": "-114.849156",
        "y_coords": "35.1752649"
    },
    {
        "origin_intersection": "nv line Nevada, California and I-15",
        "x_coords": "-115.3916369",
        "y_coords": "35.6063268"
    },
    {
        "origin_intersection": "nv line Nv Ca Line 15, California and Interstate-15",
        "x_coords": "-115.3916585",
        "y_coords": "35.6062144"
    },
    {
        "origin_intersection": "nv line Nv Line @ I-15, California and I-15",
        "x_coords": "-115.3916506",
        "y_coords": "35.6065029"
    },
    {
        "origin_intersection": "nv line Nv Line 15, California and I-15",
        "x_coords": "-115.3915211",
        "y_coords": "35.6068041"
    },
    {
        "origin_intersection": "nv line Nv Line At I-15, California and I-15",
        "x_coords": "-115.3914978",
        "y_coords": "35.6066239"
    },
    {
        "origin_intersection": "nv line Nv Line, California and I-15",
        "x_coords": "-115.3915133",
        "y_coords": "35.6064828"
    },
    {
        "origin_intersection": "nv line Nv Line, California and US-95",
        "x_coords": "-114.84917",
        "y_coords": "35.1751943"
    },
    {
        "origin_intersection": "nv line Nv Lne 15, California and I-15",
        "x_coords": "-115.391705",
        "y_coords": "35.6062931"
    },
    {
        "origin_intersection": "nv line Nv On I-15, California and I-15",
        "x_coords": "-115.3916952",
        "y_coords": "35.6063956"
    },
    {
        "origin_intersection": "nv line Nv State Line Us-95, California and US-95",
        "x_coords": "-114.8491909",
        "y_coords": "35.1752957"
    },
    {
        "origin_intersection": "nv line Nv State Line, California and I-15",
        "x_coords": "-115.3916123",
        "y_coords": "35.6066222"
    },
    {
        "origin_intersection": "nv line Nv, California and I-15",
        "x_coords": "-115.3916285",
        "y_coords": "35.606256"
    },
    {
        "origin_intersection": "nv line Nv/Ca Border I-15, California and I-15",
        "x_coords": "-115.3917629",
        "y_coords": "35.6062454"
    },
    {
        "origin_intersection": "o st 612 N Flint Ave Wilmington Ca, California and SR-1",
        "x_coords": "-118.2510048",
        "y_coords": "33.7908953"
    },
    {
        "origin_intersection": "o st Wilmington, California and SR-1",
        "x_coords": "-118.2380996",
        "y_coords": "33.7914528"
    },
    {
        "origin_intersection": "oak ave Greenfield, California and US-101",
        "x_coords": "-121.2381811",
        "y_coords": "36.3237725"
    },
    {
        "origin_intersection": "oakland rd San Jose, California and US-101",
        "x_coords": "-121.8915195",
        "y_coords": "37.3630495"
    },
    {
        "origin_intersection": "ocean blvd Long Beach, California and I-710",
        "x_coords": "-118.2042798",
        "y_coords": "33.7672372"
    },
    {
        "origin_intersection": "old bayshore highway San Jose, California and US-101",
        "x_coords": "-121.9079847",
        "y_coords": "37.3689582"
    },
    {
        "origin_intersection": "old creek rd Ventura, California and SR-33",
        "x_coords": "-119.3033675",
        "y_coords": "34.3826751"
    },
    {
        "origin_intersection": "old river rd Maricopa, California and I-5",
        "x_coords": "-119.1106669",
        "y_coords": "35.1614953"
    },
    {
        "origin_intersection": "olema Bolinas, California and bolinas rd",
        "x_coords": "-122.6996505",
        "y_coords": "37.9347986"
    },
    {
        "origin_intersection": "olive dr Bakersfield, California and SR-99",
        "x_coords": "-119.054955",
        "y_coords": "35.4123088"
    },
    {
        "origin_intersection": "ophir rd Oroville, California and SR-70",
        "x_coords": "-121.5789703",
        "y_coords": "39.4639701"
    },
    {
        "origin_intersection": "orange show rd Redlands, Ca, California and I-215",
        "x_coords": "-117.2964146",
        "y_coords": "34.0781084"
    },
    {
        "origin_intersection": "orange show rd San Bernardino, California and I-215",
        "x_coords": "-117.2963925",
        "y_coords": "34.0781251"
    },
    {
        "origin_intersection": "orange st Redlands, California and SR-38",
        "x_coords": "-117.1824716",
        "y_coords": "34.0700392"
    },
    {
        "origin_intersection": "orangethorpe ave Fullerton, California and SR-57",
        "x_coords": "-117.8772589",
        "y_coords": "33.8613337"
    },
    {
        "origin_intersection": "orangewood ave Orange, Ca, California and SR-57",
        "x_coords": "-117.8778978",
        "y_coords": "33.7957159"
    },
    {
        "origin_intersection": "orangewood ave Orange, California and SR-57",
        "x_coords": "-117.8779192",
        "y_coords": "33.7956357"
    },
    {
        "origin_intersection": "orangewood ave Stanton, California and SR-39",
        "x_coords": "-117.993034",
        "y_coords": "33.795656"
    },
    {
        "origin_intersection": "palm ave Imperial Beach,Ca, California and I-5",
        "x_coords": "-117.0873847",
        "y_coords": "32.5838256"
    },
    {
        "origin_intersection": "palm ave Santa Paula, California and SR-126",
        "x_coords": "-119.0685342",
        "y_coords": "34.3455078"
    },
    {
        "origin_intersection": "palm st Exeter, Ca 93221, California and SR-65",
        "x_coords": "-119.1359707",
        "y_coords": "36.2994334"
    },
    {
        "origin_intersection": "palmdale blvd Palmdale, California and SR-14",
        "x_coords": "-118.1337749",
        "y_coords": "34.5813224"
    },
    {
        "origin_intersection": "palomar rd 28711 Mclaughlin Rd, Romoland, Ca 92585, California and SR-74",
        "x_coords": "-117.1629893",
        "y_coords": "33.7430176"
    },
    {
        "origin_intersection": "palomar rd Romoland, California and SR-74",
        "x_coords": "-117.1629927",
        "y_coords": "33.7430306"
    },
    {
        "origin_intersection": "paradise cove rd Malibu, California and SR-1",
        "x_coords": "-118.7867033",
        "y_coords": "34.0240839"
    },
    {
        "origin_intersection": "paramount blvd Long Beach, California and SR-91",
        "x_coords": "-118.1600117",
        "y_coords": "33.8761144"
    },
    {
        "origin_intersection": "park moabi rd Needles, California and I-40",
        "x_coords": "-114.5108987",
        "y_coords": "34.7175463"
    },
    {
        "origin_intersection": "parrots ferry rd Columbia, California and SR-4",
        "x_coords": "-120.4717103",
        "y_coords": "38.0951644"
    },
    {
        "origin_intersection": "paxton st Los Angeles, California and SR-210",
        "x_coords": "-118.406983",
        "y_coords": "34.2875554"
    },
    {
        "origin_intersection": "peck rd Industry, California and SR-60",
        "x_coords": "-118.0383828",
        "y_coords": "34.0374049"
    },
    {
        "origin_intersection": "pedley rd Riverside, California and SR-60",
        "x_coords": "-117.4804165",
        "y_coords": "34.0191377"
    },
    {
        "origin_intersection": "pentz rd Oroville, California and SR-70",
        "x_coords": "-121.5595019",
        "y_coords": "39.6497904"
    },
    {
        "origin_intersection": "pepper ave Colton, California and I-10",
        "x_coords": "-117.3530978",
        "y_coords": "34.0690199"
    },
    {
        "origin_intersection": "pepper ave Rialto, California and I-10",
        "x_coords": "-117.3530859",
        "y_coords": "34.0689718"
    },
    {
        "origin_intersection": "pepper ave Rialto, California and SR-66",
        "x_coords": "-117.3529226",
        "y_coords": "34.1067571"
    },
    {
        "origin_intersection": "pershing ave Stockton, California and I-5",
        "x_coords": "-121.3123276",
        "y_coords": "37.9547932"
    },
    {
        "origin_intersection": "pershing ave Tracy, California and I-5",
        "x_coords": "-121.3190349",
        "y_coords": "37.7763507"
    },
    {
        "origin_intersection": "phelan rd Phelan, California and US-395",
        "x_coords": "-117.400033",
        "y_coords": "34.4267239"
    },
    {
        "origin_intersection": "phelps ave Coalinga, California and SR-33",
        "x_coords": "-120.3544177",
        "y_coords": "36.1542105"
    },
    {
        "origin_intersection": "pickerell ave Corcoran, California and SR-43",
        "x_coords": "-119.5542716",
        "y_coords": "36.1068553"
    },
    {
        "origin_intersection": "pico ave 1525 Pier F Ave, Long Beach, Ca 90802, California and I-710",
        "x_coords": "-118.2095568",
        "y_coords": "33.7671199"
    },
    {
        "origin_intersection": "pico ave Carson, Ca, California and I-710",
        "x_coords": "-118.2095633",
        "y_coords": "33.7671443"
    },
    {
        "origin_intersection": "pico ave Long Beach, Ca, California and I-710",
        "x_coords": "-118.2095606",
        "y_coords": "33.7671324"
    },
    {
        "origin_intersection": "pico ave Long Beach, California and I-710",
        "x_coords": "-118.2095563",
        "y_coords": "33.7671424"
    },
    {
        "origin_intersection": "pico ave Wilmington, California and I-710",
        "x_coords": "-118.2095563",
        "y_coords": "33.7671287"
    },
    {
        "origin_intersection": "pico canyon rd Castaic, California and I-5",
        "x_coords": "-118.5688414",
        "y_coords": "34.3801479"
    },
    {
        "origin_intersection": "pierson blvd Desert Hot Springs, California and SR-62",
        "x_coords": "-116.5906103",
        "y_coords": "33.9612897"
    },
    {
        "origin_intersection": "placentia ave Perris, California and I-215",
        "x_coords": "-117.2412497",
        "y_coords": "33.8230715"
    },
    {
        "origin_intersection": "placer hills rd Auburn, California and I-80",
        "x_coords": "-121.0177504",
        "y_coords": "38.9697102"
    },
    {
        "origin_intersection": "placerville dr Placerville, California and US-50",
        "x_coords": "-120.8164468",
        "y_coords": "38.7302568"
    },
    {
        "origin_intersection": "plaza dr Visalia, California and SR-198",
        "x_coords": "-119.3900015",
        "y_coords": "36.327281"
    },
    {
        "origin_intersection": "plaza rd Santa Nella, California and SR-33",
        "x_coords": "-121.0157043",
        "y_coords": "37.1119103"
    },
    {
        "origin_intersection": "pleasant valley rd 1000 Howard Rd, Camarillo, California and US-101",
        "x_coords": "-119.0698014",
        "y_coords": "34.2191588"
    },
    {
        "origin_intersection": "poe rd Brawley, California and SR-86",
        "x_coords": "-115.7527299",
        "y_coords": "33.081509"
    },
    {
        "origin_intersection": "poinsettia ln Carlsbad, California and I-5",
        "x_coords": "-117.3105525",
        "y_coords": "33.1034129"
    },
    {
        "origin_intersection": "port of stockton expressway Stockton, California and SR-4",
        "x_coords": "-121.3580132",
        "y_coords": "37.9268856"
    },
    {
        "origin_intersection": "produce ave 800 S Airport Blvd, San Francisco, Ca 94128, California and US-101",
        "x_coords": "-122.4073709",
        "y_coords": "37.6487921"
    },
    {
        "origin_intersection": "produce ave South San Francisco, California and US-101",
        "x_coords": "-122.4073738",
        "y_coords": "37.6487809"
    },
    {
        "origin_intersection": "pyrite st Jurupa Valley, California and SR-60",
        "x_coords": "-117.4623716",
        "y_coords": "34.0162158"
    },
    {
        "origin_intersection": "railroad ave Concord, California and SR-4",
        "x_coords": "-121.8895447",
        "y_coords": "38.0168522"
    },
    {
        "origin_intersection": "railroad ave Pittsburg, California and SR-4",
        "x_coords": "-121.8895123",
        "y_coords": "38.0169287"
    },
    {
        "origin_intersection": "ramona blvd El Monte, California and I-605",
        "x_coords": "-117.9980831",
        "y_coords": "34.0776363"
    },
    {
        "origin_intersection": "ramona expwy w Perris, Ca, California and I-215",
        "x_coords": "-117.2508646",
        "y_coords": "33.843808"
    },
    {
        "origin_intersection": "ramona expwy w Perris, California and I-215",
        "x_coords": "-117.2508186",
        "y_coords": "33.8438277"
    },
    {
        "origin_intersection": "ramona expwy w Riverside, California and I-215",
        "x_coords": "-117.2509056",
        "y_coords": "33.8438144"
    },
    {
        "origin_intersection": "rancho bernardo rd San Diego, California and I-15",
        "x_coords": "-117.0787994",
        "y_coords": "33.0231089"
    },
    {
        "origin_intersection": "rancho del oro dr Oceanside, California and SR-76",
        "x_coords": "-117.3182969",
        "y_coords": "33.2278625"
    },
    {
        "origin_intersection": "rancho rd Adelanto, California and US-395",
        "x_coords": "-117.4052586",
        "y_coords": "34.5592162"
    },
    {
        "origin_intersection": "randolph ave Brea, California and SR-90",
        "x_coords": "-117.8898728",
        "y_coords": "33.9125886"
    },
    {
        "origin_intersection": "reclamation rd Robins, California and SR-113",
        "x_coords": "-121.7074812",
        "y_coords": "38.8710666"
    },
    {
        "origin_intersection": "redlands blvd Moreno Valley, California and SR-60",
        "x_coords": "-117.1567435",
        "y_coords": "33.9392887"
    },
    {
        "origin_intersection": "reseda blvd Tarzana, California and US-101",
        "x_coords": "-118.5360242",
        "y_coords": "34.1732402"
    },
    {
        "origin_intersection": "reserve rd 56533 Highway 58, Mckittrick, California and SR-33",
        "x_coords": "-119.6230239",
        "y_coords": "35.3054532"
    },
    {
        "origin_intersection": "reservoir st Pomona, Ca, California and SR-60",
        "x_coords": "-117.7314685",
        "y_coords": "34.0250053"
    },
    {
        "origin_intersection": "reservoir st Pomona, California and SR-60",
        "x_coords": "-117.731484",
        "y_coords": "34.0249701"
    },
    {
        "origin_intersection": "rice ave Oxnard, California and US-101",
        "x_coords": "-119.1422375",
        "y_coords": "34.2225573"
    },
    {
        "origin_intersection": "richards blvd Sacramento I-5 & Richards Blvd, California and I-5",
        "x_coords": "-121.5039055",
        "y_coords": "38.5970054"
    },
    {
        "origin_intersection": "richards blvd Sacramento, California and I-5",
        "x_coords": "-121.5039401",
        "y_coords": "38.596964"
    },
    {
        "origin_intersection": "richmond parkway Richmond, California and I-80",
        "x_coords": "-122.3146394",
        "y_coords": "37.9867923"
    },
    {
        "origin_intersection": "riverford rd Lakeside, California and SR-67",
        "x_coords": "-116.9470591",
        "y_coords": "32.8548762"
    },
    {
        "origin_intersection": "riverside ave San Bernardino, California and SR-210",
        "x_coords": "-117.3704431",
        "y_coords": "34.1358772"
    },
    {
        "origin_intersection": "riverside dr Ontario, California and SR-83",
        "x_coords": "-117.6508",
        "y_coords": "34.0193248"
    },
    {
        "origin_intersection": "rosamond bl e Mojave, California and SR-14",
        "x_coords": "-118.1716743",
        "y_coords": "34.8642622"
    },
    {
        "origin_intersection": "rosamond blvd Rosamond, California and SR-14",
        "x_coords": "-118.1717299",
        "y_coords": "34.8642482"
    },
    {
        "origin_intersection": "roscoe blvd Sun Valley, California and SR-170",
        "x_coords": "-118.4101671",
        "y_coords": "34.2216531"
    },
    {
        "origin_intersection": "rose ave 1090 Pacific Ave, Oxnard, Ca, California and US-101",
        "x_coords": "-119.1587662",
        "y_coords": "34.225148"
    },
    {
        "origin_intersection": "rose ave Oxnard, California and US-101",
        "x_coords": "-119.1587302",
        "y_coords": "34.2251366"
    },
    {
        "origin_intersection": "rose hills rd City Of Industry, California and I-605",
        "x_coords": "-118.0528417",
        "y_coords": "34.0179783"
    },
    {
        "origin_intersection": "rose hills rd Industry, California and I-605",
        "x_coords": "-118.0528234",
        "y_coords": "34.0179692"
    },
    {
        "origin_intersection": "rose hills rd Whittier, Ca, California and Interstate-605",
        "x_coords": "-118.0527884",
        "y_coords": "34.0179502"
    },
    {
        "origin_intersection": "rose hills rd Whittier, California and I-605",
        "x_coords": "-118.0528075",
        "y_coords": "34.0179587"
    },
    {
        "origin_intersection": "rosecrans ave Gardena, California and SR-110",
        "x_coords": "-118.2866718",
        "y_coords": "33.9018812"
    },
    {
        "origin_intersection": "rosecrans ave Lawndale, California and SR-1",
        "x_coords": "-118.3961858",
        "y_coords": "33.9018679"
    },
    {
        "origin_intersection": "rosecrans ave Los Angeles, Ca, California and SR-110",
        "x_coords": "-118.2865201",
        "y_coords": "33.9018682"
    },
    {
        "origin_intersection": "rosecrans ave Santa Fe Springs, California and I-605",
        "x_coords": "-118.1052331",
        "y_coords": "33.9020686"
    },
    {
        "origin_intersection": "rosedale hwy Bakersfield, California and SR-43",
        "x_coords": "-119.2520383",
        "y_coords": "35.3834705"
    },
    {
        "origin_intersection": "roselle st San Diego, California and I-5",
        "x_coords": "-117.2243447",
        "y_coords": "32.9012072"
    },
    {
        "origin_intersection": "roxford st Sylmar, California and I-5",
        "x_coords": "-118.4796637",
        "y_coords": "34.3033018"
    },
    {
        "origin_intersection": "rubidoux blvd Riverside, California and SR-60",
        "x_coords": "-117.4003438",
        "y_coords": "34.0045799"
    },
    {
        "origin_intersection": "s rice ave Pt Hueneme, California and SR-1",
        "x_coords": "-119.1379382",
        "y_coords": "34.1611431"
    },
    {
        "origin_intersection": "sabodan st Bakersfield, California and SR-166",
        "x_coords": "-118.975866",
        "y_coords": "35.0593794"
    },
    {
        "origin_intersection": "san bruno ave 800 S Airport Blvd, San Francisco, Ca, California and US-101",
        "x_coords": "-122.4036043",
        "y_coords": "37.6357536"
    },
    {
        "origin_intersection": "san bruno ave San Francisco, Ca, California and US-101",
        "x_coords": "-122.4036068",
        "y_coords": "37.6357424"
    },
    {
        "origin_intersection": "san bruno ave San Francisco, California and US-101",
        "x_coords": "-122.407851",
        "y_coords": "37.7359247"
    },
    {
        "origin_intersection": "san gorgonio ave Banning - 300 W Lincoln St / Ca-243, California and SR-243",
        "x_coords": "-116.8767107",
        "y_coords": "33.9217359"
    },
    {
        "origin_intersection": "san gorgonio ave Banning, California and SR-243",
        "x_coords": "-116.8767098",
        "y_coords": "33.9217292"
    },
    {
        "origin_intersection": "san luis dam service rd Gustine, California and SR-152",
        "x_coords": "-121.1016663",
        "y_coords": "37.0837923"
    },
    {
        "origin_intersection": "san marcos blvd San Marcos, California and SR-78",
        "x_coords": "-117.1763765",
        "y_coords": "33.1378439"
    },
    {
        "origin_intersection": "san martin ave San Martin, California and US-101",
        "x_coords": "-121.5994173",
        "y_coords": "37.0888663"
    },
    {
        "origin_intersection": "san tomas expwy e Santa Clara, California and US-101",
        "x_coords": "-121.963906",
        "y_coords": "37.3819301"
    },
    {
        "origin_intersection": "sanborn rd 1300 Abbott St, Salinas, Ca, California and US-101",
        "x_coords": "-121.6281507",
        "y_coords": "36.6666323"
    },
    {
        "origin_intersection": "sand canyon ave Irvine, California and I-5",
        "x_coords": "-117.7581685",
        "y_coords": "33.6777738"
    },
    {
        "origin_intersection": "sanderson ave Hemet, California and SR-79",
        "x_coords": "-117.0035215",
        "y_coords": "33.8232615"
    },
    {
        "origin_intersection": "sanderson ave Perris, California and SR-79",
        "x_coords": "-117.0034997",
        "y_coords": "33.8232307"
    },
    {
        "origin_intersection": "sanderson ave San Jacinto, California and SR-79",
        "x_coords": "-117.0034827",
        "y_coords": "33.8232643"
    },
    {
        "origin_intersection": "sandquist rd China Lake, California and SR-178",
        "x_coords": "-117.6698632",
        "y_coords": "35.6516335"
    },
    {
        "origin_intersection": "santa anita ave Arcadia, California and SR-210",
        "x_coords": "-118.0315916",
        "y_coords": "34.1478089"
    },
    {
        "origin_intersection": "santa fe ave 1520 W 11Th St, Long Beach, Ca 90813, California and SR-1",
        "x_coords": "-118.2154939",
        "y_coords": "33.7898588"
    },
    {
        "origin_intersection": "santa fe ave 1520 W 11Th St, Long Beach, Ca, 90813, California and SR-1",
        "x_coords": "-118.215469",
        "y_coords": "33.7898621"
    },
    {
        "origin_intersection": "santa fe ave 1525 Pier F Ave, Long Beach, California and SR-1",
        "x_coords": "-118.2141583",
        "y_coords": "33.7479375"
    },
    {
        "origin_intersection": "santa fe ave 1525 Pier F Avenue Berth 206, Long Beach, Ca,, California and SR-1",
        "x_coords": "-118.2142173",
        "y_coords": "33.747933"
    },
    {
        "origin_intersection": "santa fe ave 50 West Pier D St Long Beach, Ca, California and SR-1",
        "x_coords": "-118.2087404",
        "y_coords": "33.7697912"
    },
    {
        "origin_intersection": "santa fe ave Compton, California and SR-91",
        "x_coords": "-118.2143743",
        "y_coords": "33.8735071"
    },
    {
        "origin_intersection": "santa fe ave Corcoran, California and SR-43",
        "x_coords": "-119.5664963",
        "y_coords": "36.1118146"
    },
    {
        "origin_intersection": "santa fe ave Long Beach, Ca, California and SR-1",
        "x_coords": "-118.215468",
        "y_coords": "33.7898422"
    },
    {
        "origin_intersection": "santa fe ave Long Beach, California and SR-1",
        "x_coords": "-118.2154462",
        "y_coords": "33.7898649"
    },
    {
        "origin_intersection": "santa fe ave Riverside, California and SR-1",
        "x_coords": "-118.2154653",
        "y_coords": "33.7898846"
    },
    {
        "origin_intersection": "santa fe ave Terminal Island, California and SR-1",
        "x_coords": "-118.215487",
        "y_coords": "33.7898411"
    },
    {
        "origin_intersection": "santa fe materials rock pit Waterford, California and SR-132",
        "x_coords": "-120.6658576",
        "y_coords": "37.6461831"
    },
    {
        "origin_intersection": "santa fe springs rd Whittier, California and SR-72",
        "x_coords": "-118.0420986",
        "y_coords": "33.9683249"
    },
    {
        "origin_intersection": "santa lucia cyn rd Lompoc, California and SR-1",
        "x_coords": "-120.4891254",
        "y_coords": "34.7131282"
    },
    {
        "origin_intersection": "santa lucia cyn rd Vandenberg Afb, Lompoc, California and SR-1",
        "x_coords": "-120.4891412",
        "y_coords": "34.713164"
    },
    {
        "origin_intersection": "santa paula st Santa Paula, California and SR-150",
        "x_coords": "-119.061545",
        "y_coords": "34.3573482"
    },
    {
        "origin_intersection": "santa rita rd Pleasanton, California and I-580",
        "x_coords": "-121.8720862",
        "y_coords": "37.7014922"
    },
    {
        "origin_intersection": "santa rita rd Pleasation, California and I-580",
        "x_coords": "-121.8721018",
        "y_coords": "37.7014487"
    },
    {
        "origin_intersection": "santa rosa ave Santa Rosa, California and US-101",
        "x_coords": "-122.7135469",
        "y_coords": "38.421178"
    },
    {
        "origin_intersection": "scott creek rd Fremont, California and I-680",
        "x_coords": "-121.9056526",
        "y_coords": "37.4663765"
    },
    {
        "origin_intersection": "scott rd Menifee, California and I-215",
        "x_coords": "-117.1712034",
        "y_coords": "33.6417255"
    },
    {
        "origin_intersection": "scott rd Winchester, California and I-215",
        "x_coords": "-117.1712277",
        "y_coords": "33.6417396"
    },
    {
        "origin_intersection": "scripps poway pkwy Lakeside, California and SR-67",
        "x_coords": "-116.9691209",
        "y_coords": "32.9548842"
    },
    {
        "origin_intersection": "scripps poway pkwy Poway, California and I-15",
        "x_coords": "-117.1113461",
        "y_coords": "32.9367317"
    },
    {
        "origin_intersection": "sea world dr San Diego, Ca, California and I-5",
        "x_coords": "-117.2066449",
        "y_coords": "32.7699523"
    },
    {
        "origin_intersection": "seamas ave Sacramento, California and I-5",
        "x_coords": "-121.521308",
        "y_coords": "38.5255752"
    },
    {
        "origin_intersection": "seaside fwy Long Beach, Ca, California and I-710",
        "x_coords": "-118.2323959",
        "y_coords": "33.7606061"
    },
    {
        "origin_intersection": "seaside fwy San Pedro, Ca, California and I-710",
        "x_coords": "-118.2442873",
        "y_coords": "33.7567633"
    },
    {
        "origin_intersection": "seaward ave Ventura, California and US-101",
        "x_coords": "-119.2691798",
        "y_coords": "34.2650018"
    },
    {
        "origin_intersection": "second st Norco, California and I-15",
        "x_coords": "-117.558661",
        "y_coords": "33.9097217"
    },
    {
        "origin_intersection": "sepulveda blvd 22707 S Wilmington Ave. Carson, Ca 90745, California and SR-110",
        "x_coords": "-118.2876172",
        "y_coords": "33.8090345"
    },
    {
        "origin_intersection": "shadow mtn rd Oro Grande, California and US-395",
        "x_coords": "-117.4623464",
        "y_coords": "34.7267638"
    },
    {
        "origin_intersection": "shaw ave Fresno, California and SR-99",
        "x_coords": "-119.886808",
        "y_coords": "36.8080188"
    },
    {
        "origin_intersection": "shell rd Ventura, California and SR-33",
        "x_coords": "-119.2942879",
        "y_coords": "34.3157102"
    },
    {
        "origin_intersection": "sherman way Burbank, California and SR-170",
        "x_coords": "-118.4036499",
        "y_coords": "34.2011852"
    },
    {
        "origin_intersection": "sherman way Van Nuys, Ca, California and SR-170",
        "x_coords": "-118.4036628",
        "y_coords": "34.2011939"
    },
    {
        "origin_intersection": "siempre viva rd Otay Mesa, California and SR-905",
        "x_coords": "-116.9398661",
        "y_coords": "32.5553926"
    },
    {
        "origin_intersection": "sierra ave Bloomington, California and I-10",
        "x_coords": "-117.4356762",
        "y_coords": "34.0673372"
    },
    {
        "origin_intersection": "sierra ave Fontana, California and I-15",
        "x_coords": "-117.4368008",
        "y_coords": "34.1816271"
    },
    {
        "origin_intersection": "sierra college blvd Rocklin, California and I-80",
        "x_coords": "-121.2054475",
        "y_coords": "38.8029659"
    },
    {
        "origin_intersection": "sierra hwy Acton, California and SR-14",
        "x_coords": "-118.2264334",
        "y_coords": "34.5001833"
    },
    {
        "origin_intersection": "sierra hwy Palmdale, California and SR-14",
        "x_coords": "-118.226525",
        "y_coords": "34.500223"
    },
    {
        "origin_intersection": "silver queen rd Mojave, California and SR-14",
        "x_coords": "-118.1597816",
        "y_coords": "34.9953941"
    },
    {
        "origin_intersection": "sir francis drake bl Olema, California and SR-1",
        "x_coords": "-122.7877964",
        "y_coords": "38.0404989"
    },
    {
        "origin_intersection": "sir francis drake blvd San Quentin, California and I-580",
        "x_coords": "-122.4907128",
        "y_coords": "37.9460084"
    },
    {
        "origin_intersection": "slaughter house cyn rd 12570 Hwy 67 Lakeside Ca. 92040, California and SR-67",
        "x_coords": "-116.9429071",
        "y_coords": "32.9068163"
    },
    {
        "origin_intersection": "slaughter house cyn rd Lakeside, California and SR-67",
        "x_coords": "-116.9428738",
        "y_coords": "32.9068224"
    },
    {
        "origin_intersection": "slauson ave Certified Steel (Huntington Park), California and SR-110",
        "x_coords": "-118.2805185",
        "y_coords": "33.9891897"
    },
    {
        "origin_intersection": "slauson ave Long Angeles, California and SR-110",
        "x_coords": "-118.2804992",
        "y_coords": "33.9891891"
    },
    {
        "origin_intersection": "slauson ave Los Angeles, California and SR-110",
        "x_coords": "-118.2805069",
        "y_coords": "33.9891805"
    },
    {
        "origin_intersection": "slauson ave Santa Fe Springs, California and I-605",
        "x_coords": "-118.0833697",
        "y_coords": "33.9672303"
    },
    {
        "origin_intersection": "slauson ave Vernon, California and SR-110",
        "x_coords": "-118.2805184",
        "y_coords": "33.9891802"
    },
    {
        "origin_intersection": "snyder rd 2653 E Wright Road, Holtville, California and SR-115",
        "x_coords": "-115.3370796",
        "y_coords": "32.811103"
    },
    {
        "origin_intersection": "solano ave Vallejo, California and SR-29",
        "x_coords": "-122.2511509",
        "y_coords": "38.094492"
    },
    {
        "origin_intersection": "south bonnyview rd Redding, California and I-5",
        "x_coords": "-122.3510291",
        "y_coords": "40.5383633"
    },
    {
        "origin_intersection": "south main st Oregon - 5, California and I-5",
        "x_coords": "-122.6154065",
        "y_coords": "42.0052519"
    },
    {
        "origin_intersection": "south main st Red Bluff, California and I-5",
        "x_coords": "-122.2233521",
        "y_coords": "40.1573883"
    },
    {
        "origin_intersection": "south union ave Bakersfield, California and SR-204",
        "x_coords": "-119.002996",
        "y_coords": "35.3534872"
    },
    {
        "origin_intersection": "south watt ave Sacramento, California and SR-16",
        "x_coords": "-121.3709716",
        "y_coords": "38.5387852"
    },
    {
        "origin_intersection": "southfront rd Livermore, California and I-580",
        "x_coords": "-121.7220813",
        "y_coords": "37.7065792"
    },
    {
        "origin_intersection": "spring st Long Beach, California and I-405",
        "x_coords": "-118.1618878",
        "y_coords": "33.8116826"
    },
    {
        "origin_intersection": "spring st Signal Hill, California and I-405",
        "x_coords": "-118.1618487",
        "y_coords": "33.811685"
    },
    {
        "origin_intersection": "spring st Spring Hill, California and I-405",
        "x_coords": "-118.1618115",
        "y_coords": "33.811661"
    },
    {
        "origin_intersection": "SR-139 and Oregon Line",
        "x_coords": "-121.5191292",
        "y_coords": "41.9979742"
    },
    {
        "origin_intersection": "SR-178 and Nevada Line",
        "x_coords": "-116.1132488",
        "y_coords": "36.1661336"
    },
    {
        "origin_intersection": "stanley blvd Pleasanton, California and SR-84",
        "x_coords": "-121.805332",
        "y_coords": "37.6760905"
    },
    {
        "origin_intersection": "stevens creek blvd Santa Clara, California and I-280",
        "x_coords": "-121.9998157",
        "y_coords": "37.3230383"
    },
    {
        "origin_intersection": "stoddard rd Modesto, California and SR-219",
        "x_coords": "-121.0684039",
        "y_coords": "37.71113"
    },
    {
        "origin_intersection": "stoddard wells rd Victorville, California and I-15",
        "x_coords": "-117.2894045",
        "y_coords": "34.554343"
    },
    {
        "origin_intersection": "studebaker rd Long Beach, California and SR-22",
        "x_coords": "-118.1031239",
        "y_coords": "33.7742563"
    },
    {
        "origin_intersection": "studebaker rd Santa Fe Springs, California and I-605",
        "x_coords": "-118.0979974",
        "y_coords": "33.9353873"
    },
    {
        "origin_intersection": "stuhr rd Newman, California and I-5",
        "x_coords": "-121.10998",
        "y_coords": "37.3254566"
    },
    {
        "origin_intersection": "sunland blvd Los Angeles, California and SR-210",
        "x_coords": "-118.3298809",
        "y_coords": "34.2597837"
    },
    {
        "origin_intersection": "sunland blvd Shadow Hills, California and SR-210",
        "x_coords": "-118.329878",
        "y_coords": "34.2597419"
    },
    {
        "origin_intersection": "sunland blvd Signal Hill, California and SR-210",
        "x_coords": "-118.3299981",
        "y_coords": "34.2597811"
    },
    {
        "origin_intersection": "sweetwater cyn rd Malibu, California and SR-1",
        "x_coords": "-118.6740909",
        "y_coords": "34.0385692"
    },
    {
        "origin_intersection": "sweetwater spngs blvd Spring Valley, California and SR-94",
        "x_coords": "-116.9744352",
        "y_coords": "32.7444822"
    },
    {
        "origin_intersection": "sycamore ave Vista, California and SR-78",
        "x_coords": "-117.2146301",
        "y_coords": "33.1670183"
    },
    {
        "origin_intersection": "table mountain blvd 2216 Table Mountain Blvd, Oroville, Ca 95965, California and SR-70",
        "x_coords": "-121.6182019",
        "y_coords": "39.6025278"
    },
    {
        "origin_intersection": "taft ave Orange, California and SR-57",
        "x_coords": "-117.8762961",
        "y_coords": "33.8180817"
    },
    {
        "origin_intersection": "tampa ave Northridge, California and SR-118",
        "x_coords": "-118.5562214",
        "y_coords": "34.2723626"
    },
    {
        "origin_intersection": "tarawa rd Coronado, California and SR-75",
        "x_coords": "-117.1668581",
        "y_coords": "32.6725184"
    },
    {
        "origin_intersection": "tavern rd Alpine, California and I-8",
        "x_coords": "-116.7796368",
        "y_coords": "32.8400159"
    },
    {
        "origin_intersection": "tejon highway Arvin, California and SR-223",
        "x_coords": "-118.8242665",
        "y_coords": "35.2091161"
    },
    {
        "origin_intersection": "telegraph rd Santa Fe Springs, California and I-605",
        "x_coords": "-118.0919494",
        "y_coords": "33.9497531"
    },
    {
        "origin_intersection": "templin highway Castaic, California and I-5",
        "x_coords": "-118.6878985",
        "y_coords": "34.5696583"
    },
    {
        "origin_intersection": "templin highway Gilroy, California and I-5",
        "x_coords": "-118.6878709",
        "y_coords": "34.5696763"
    },
    {
        "origin_intersection": "templin highway Santa Clarita, California and I-5",
        "x_coords": "-118.6879286",
        "y_coords": "34.5696365"
    },
    {
        "origin_intersection": "templin highway Stockton, California and I-5",
        "x_coords": "-118.687907",
        "y_coords": "34.5696379"
    },
    {
        "origin_intersection": "terminal island frwy Long Beach, Ca, California and SR-103",
        "x_coords": "-118.2405064",
        "y_coords": "33.7725219"
    },
    {
        "origin_intersection": "the old rd Santa Clarita, California and I-5",
        "x_coords": "-118.5426152",
        "y_coords": "34.3502947"
    },
    {
        "origin_intersection": "thornton ave Newark, California and SR-84",
        "x_coords": "-122.0207025",
        "y_coords": "37.5491954"
    },
    {
        "origin_intersection": "tillie lewis rd Stockton, California and SR-4",
        "x_coords": "-121.3183578",
        "y_coords": "37.9312393"
    },
    {
        "origin_intersection": "tippecanoe ave San Bernardino, California and I-10",
        "x_coords": "-117.2611955",
        "y_coords": "34.0644737"
    },
    {
        "origin_intersection": "todd rd Santa Rosa, California and US-101",
        "x_coords": "-122.7161848",
        "y_coords": "38.3870017"
    },
    {
        "origin_intersection": "trinity dam blvd Lewiston, California and SR-299",
        "x_coords": "-122.7970137",
        "y_coords": "40.6636788"
    },
    {
        "origin_intersection": "tully rd 656 Wool Creek Drive, San Jose, California and US-101",
        "x_coords": "-121.8315308",
        "y_coords": "37.3184312"
    },
    {
        "origin_intersection": "tully rd San Jose, California and US-101",
        "x_coords": "-121.8315363",
        "y_coords": "37.3184246"
    },
    {
        "origin_intersection": "turner rd Lodi, California and SR-99",
        "x_coords": "-121.261249",
        "y_coords": "38.1457484"
    },
    {
        "origin_intersection": "tustin ave Anaheim, California and SR-91",
        "x_coords": "-117.837976",
        "y_coords": "33.8511023"
    },
    {
        "origin_intersection": "tustin ave Placentia, California and SR-91",
        "x_coords": "-117.8379906",
        "y_coords": "33.8511374"
    },
    {
        "origin_intersection": "twain rd Twain, California and SR-70",
        "x_coords": "-121.0713708",
        "y_coords": "40.0206499"
    },
    {
        "origin_intersection": "twain rd Twain, California and SR-70",
        "x_coords": "-121.0713891",
        "y_coords": "40.0206565"
    },
    {
        "origin_intersection": "twin oaks valley rd Escondido, California and SR-78",
        "x_coords": "-117.1636525",
        "y_coords": "33.1371254"
    },
    {
        "origin_intersection": "twin oaks valley rd San Marcos, California and SR-78",
        "x_coords": "-117.1636226",
        "y_coords": "33.1371268"
    },
    {
        "origin_intersection": "twin view blvd Redding, California and I-5",
        "x_coords": "-122.3666315",
        "y_coords": "40.6229433"
    },
    {
        "origin_intersection": "union st Oakland, California and I-880",
        "x_coords": "-122.2906622",
        "y_coords": "37.8037951"
    },
    {
        "origin_intersection": "university ave Riverside, California and I-215",
        "x_coords": "-117.3353184",
        "y_coords": "33.9756626"
    },
    {
        "origin_intersection": "US-395 and Nevada Line",
        "x_coords": "-120.0008077",
        "y_coords": "39.6732954"
    },
    {
        "origin_intersection": "US-395 and Oregon Line",
        "x_coords": "-120.2980881",
        "y_coords": "41.9925567"
    },
    {
        "origin_intersection": "US-6 and Nevada Line",
        "x_coords": "-118.4354258",
        "y_coords": "37.9011252"
    },
    {
        "origin_intersection": "US-97 and Oregon Line",
        "x_coords": "-121.8892113",
        "y_coords": "42.0032008"
    },
    {
        "origin_intersection": "utica ave Avenal, California and SR-41",
        "x_coords": "-120.0182411",
        "y_coords": "35.9345514"
    },
    {
        "origin_intersection": "valley blvd El Monte, California and I-605",
        "x_coords": "-118.0042484",
        "y_coords": "34.0543521"
    },
    {
        "origin_intersection": "valley blvd Industry, California and I-605",
        "x_coords": "-118.0042632",
        "y_coords": "34.0543704"
    },
    {
        "origin_intersection": "valley blvd Los Angeles, California and I-710",
        "x_coords": "-118.161228",
        "y_coords": "34.0750433"
    },
    {
        "origin_intersection": "valley parkway 1148 Industrial Ave, Escondido, Ca, 92029, California and I-15",
        "x_coords": "-117.099087",
        "y_coords": "33.1154022"
    },
    {
        "origin_intersection": "valley parkway Escondido, California and I-15",
        "x_coords": "-117.0991244",
        "y_coords": "33.1153775"
    },
    {
        "origin_intersection": "valley view st Buena Park, California and SR-91",
        "x_coords": "-118.0286397",
        "y_coords": "33.8567107"
    },
    {
        "origin_intersection": "valley view st Huntington Beach, California and SR-22",
        "x_coords": "-118.0378229",
        "y_coords": "33.774486"
    },
    {
        "origin_intersection": "van buren blvd Riverside, California and I-215",
        "x_coords": "-117.2699668",
        "y_coords": "33.8876694"
    },
    {
        "origin_intersection": "van buren blvd Riverside, California and SR-60",
        "x_coords": "-117.5242834",
        "y_coords": "34.0161269"
    },
    {
        "origin_intersection": "vandergrift blvd Camp Pendleton, California and I-5",
        "x_coords": "-117.3872979",
        "y_coords": "33.2104341"
    },
    {
        "origin_intersection": "vasco rd Livermore, California and I-580",
        "x_coords": "-121.7237955",
        "y_coords": "37.7090292"
    },
    {
        "origin_intersection": "vasco rd Livermore, California and SR-4",
        "x_coords": "-121.7126591",
        "y_coords": "37.8957598"
    },
    {
        "origin_intersection": "vaughn rd Dixon, California and SR-113",
        "x_coords": "-121.8223797",
        "y_coords": "38.4672753"
    },
    {
        "origin_intersection": "ventura blvd Oxnard, California and SR-232",
        "x_coords": "-119.1720352",
        "y_coords": "34.2335382"
    },
    {
        "origin_intersection": "vermont ave Torrance, California and SR-91",
        "x_coords": "-118.2909227",
        "y_coords": "33.872775"
    },
    {
        "origin_intersection": "via rancho pkwy Escondido, California and I-15",
        "x_coords": "-117.0694416",
        "y_coords": "33.0692355"
    },
    {
        "origin_intersection": "via rancho pkwy San Diego, California and I-15",
        "x_coords": "-117.0694659",
        "y_coords": "33.0692335"
    },
    {
        "origin_intersection": "victoria ave 2900 Golf Course Dr, Ventura, Ca 93003, California and US-101",
        "x_coords": "-119.2110278",
        "y_coords": "34.2525627"
    },
    {
        "origin_intersection": "vigilante rd Lakeside, California and SR-67",
        "x_coords": "-116.9416518",
        "y_coords": "32.9060134"
    },
    {
        "origin_intersection": "vincent ave West Covina, California and I-10",
        "x_coords": "-117.9267052",
        "y_coords": "34.0723557"
    },
    {
        "origin_intersection": "vineyard ave Ontario, California and I-10",
        "x_coords": "-117.611182",
        "y_coords": "34.0706198"
    },
    {
        "origin_intersection": "vineyard ave Ontario, California and SR-60",
        "x_coords": "-117.6110027",
        "y_coords": "34.0305216"
    },
    {
        "origin_intersection": "vineyard ave Oxnard, California and US-101",
        "x_coords": "-119.1732823",
        "y_coords": "34.2317713"
    },
    {
        "origin_intersection": "vineyard ave Oxnard, California and US-101",
        "x_coords": "-119.1732609",
        "y_coords": "34.2318245"
    },
    {
        "origin_intersection": "vineyard ave Port Heneme, California and US-101",
        "x_coords": "-119.1732408",
        "y_coords": "34.2318695"
    },
    {
        "origin_intersection": "vineyard ave Port Hueneme, California and US-101",
        "x_coords": "-119.1731911",
        "y_coords": "34.2318913"
    },
    {
        "origin_intersection": "virginia st San Jose, California and I-280",
        "x_coords": "-121.8764459",
        "y_coords": "37.326499"
    },
    {
        "origin_intersection": "virginia st San Jose, California and I-280",
        "x_coords": "-121.8759117",
        "y_coords": "37.3267466"
    },
    {
        "origin_intersection": "vista sorrento pkwy Sorrento Valley, California and I-805",
        "x_coords": "-117.2066529",
        "y_coords": "32.8922046"
    },
    {
        "origin_intersection": "vista village dr Vista, California and SR-78",
        "x_coords": "-117.2500819",
        "y_coords": "33.1948787"
    },
    {
        "origin_intersection": "w 16th st Merced, California and SR-59",
        "x_coords": "-120.5005899",
        "y_coords": "37.3031921"
    },
    {
        "origin_intersection": "walnut ave Greenfield, California and US-101",
        "x_coords": "-121.2438646",
        "y_coords": "36.3293801"
    },
    {
        "origin_intersection": "walters rd Travis Afb, California and SR-12",
        "x_coords": "-121.9897817",
        "y_coords": "38.2402344"
    },
    {
        "origin_intersection": "warner ave Huntington Beach, California and SR-1",
        "x_coords": "-118.0630522",
        "y_coords": "33.7115908"
    },
    {
        "origin_intersection": "warren rd San Jacinto, California and SR-74",
        "x_coords": "-117.0328795",
        "y_coords": "33.7437852"
    },
    {
        "origin_intersection": "wasco way Taft, California and SR-58",
        "x_coords": "-119.4475911",
        "y_coords": "35.3991152"
    },
    {
        "origin_intersection": "washington ave San Leandro, California and SR-238",
        "x_coords": "-122.1388187",
        "y_coords": "37.6911894"
    },
    {
        "origin_intersection": "washington blvd Commerce, California and I-605",
        "x_coords": "-118.080045",
        "y_coords": "33.9739428"
    },
    {
        "origin_intersection": "washington blvd Commerce, California and I-605",
        "x_coords": "-118.0800504",
        "y_coords": "33.9739545"
    },
    {
        "origin_intersection": "washington blvd Commerce, California and I-710",
        "x_coords": "-118.1729415",
        "y_coords": "34.0047455"
    },
    {
        "origin_intersection": "washington blvd Commerce, California and I-710",
        "x_coords": "-118.1729414",
        "y_coords": "34.004766"
    },
    {
        "origin_intersection": "washington st San Diego, California and I-5",
        "x_coords": "-117.1824481",
        "y_coords": "32.7424857"
    },
    {
        "origin_intersection": "watson ave Chester, California and SR-36",
        "x_coords": "-121.2410008",
        "y_coords": "40.2947841"
    },
    {
        "origin_intersection": "watson ave Chester, California and SR-36",
        "x_coords": "-121.241067",
        "y_coords": "40.2948113"
    },
    {
        "origin_intersection": "watt ave Sacramento, California and I-80",
        "x_coords": "-121.3831473",
        "y_coords": "38.645116"
    },
    {
        "origin_intersection": "watt ave Sacramento, California and US-50",
        "x_coords": "-121.3794311",
        "y_coords": "38.5587317"
    },
    {
        "origin_intersection": "weir canyon rd 22833 La Palma Dr., Yorba Linda, Ca 92887, California and SR-91",
        "x_coords": "-117.7446538",
        "y_coords": "33.8704509"
    },
    {
        "origin_intersection": "weirick rd Corona, Ca, California and I-15",
        "x_coords": "-117.5074664",
        "y_coords": "33.8070007"
    },
    {
        "origin_intersection": "weirick rd Corona, California and I-15",
        "x_coords": "-117.5074235",
        "y_coords": "33.8069562"
    },
    {
        "origin_intersection": "wells rd Santa Paula, California and SR-118",
        "x_coords": "-119.1563674",
        "y_coords": "34.2896398"
    },
    {
        "origin_intersection": "west 6th st Corona, California and SR-91",
        "x_coords": "-117.6037251",
        "y_coords": "33.8793648"
    },
    {
        "origin_intersection": "west el camino ave 2850 El Centro Rd, Sacramento, Ca, 95833, California and I-80",
        "x_coords": "-121.5354392",
        "y_coords": "38.6152492"
    },
    {
        "origin_intersection": "west grand ave Oakland, California and I-80",
        "x_coords": "-122.307223",
        "y_coords": "37.8248487"
    },
    {
        "origin_intersection": "west st Woodland, California and I-5",
        "x_coords": "-121.7839318",
        "y_coords": "38.7030896"
    },
    {
        "origin_intersection": "western ave Wasco, California and SR-46",
        "x_coords": "-119.3758829",
        "y_coords": "35.6014926"
    },
    {
        "origin_intersection": "westwood cemetary rd Westwood, California and SR-36",
        "x_coords": "-120.9672937",
        "y_coords": "40.316518"
    },
    {
        "origin_intersection": "whipple ave San Carlos / 270 Industrial Road, California and SR-82",
        "x_coords": "-122.2397907",
        "y_coords": "37.4904467"
    },
    {
        "origin_intersection": "whipple rd Hayward, California and I-880",
        "x_coords": "-122.0666974",
        "y_coords": "37.60674"
    },
    {
        "origin_intersection": "whitley ave Corcoran, California and SR-43",
        "x_coords": "-119.5458491",
        "y_coords": "36.0980346"
    },
    {
        "origin_intersection": "wilbur ave Antioch, California and SR-160",
        "x_coords": "-121.7521099",
        "y_coords": "38.012224"
    },
    {
        "origin_intersection": "wild horse rd King City, California and US-101",
        "x_coords": "-121.0723429",
        "y_coords": "36.1875564"
    },
    {
        "origin_intersection": "wileys well rd Colorado River Substation, Blythe, Ca, California and I-10",
        "x_coords": "-114.9016368",
        "y_coords": "33.6079559"
    },
    {
        "origin_intersection": "will s green rd Colusa, California and SR-20",
        "x_coords": "-122.0230927",
        "y_coords": "39.2023304"
    },
    {
        "origin_intersection": "willow ave Hercules, California and SR-4",
        "x_coords": "-122.256751",
        "y_coords": "38.0136845"
    },
    {
        "origin_intersection": "willow st Long Beach, California and I-405",
        "x_coords": "-118.1893009",
        "y_coords": "33.817465"
    },
    {
        "origin_intersection": "wilmington ave 22707 S Wilmington Ave, Carson, California and I-405",
        "x_coords": "-118.2416831",
        "y_coords": "33.8258265"
    },
    {
        "origin_intersection": "wilmington ave Carson, California and I-405",
        "x_coords": "-118.2416498",
        "y_coords": "33.8258276"
    },
    {
        "origin_intersection": "wilmington ave Compton, California and I-105",
        "x_coords": "-118.2391207",
        "y_coords": "33.928313"
    },
    {
        "origin_intersection": "wilmington ave Compton, California and SR-91",
        "x_coords": "-118.2360606",
        "y_coords": "33.8737087"
    },
    {
        "origin_intersection": "wilshire blvd Los Angeles, California and I-405",
        "x_coords": "-118.4521607",
        "y_coords": "34.0557389"
    },
    {
        "origin_intersection": "wilson ave Colusa, California and SR-20",
        "x_coords": "-122.0371195",
        "y_coords": "39.1966366"
    },
    {
        "origin_intersection": "wilson way Stockton, California and SR-4",
        "x_coords": "-121.2709043",
        "y_coords": "37.9533421"
    },
    {
        "origin_intersection": "winchester rd Temecula, California and I-15",
        "x_coords": "-117.1629897",
        "y_coords": "33.5224564"
    },
    {
        "origin_intersection": "winter gardens blvd Lakeside, California and SR-67",
        "x_coords": "-116.9321172",
        "y_coords": "32.8582497"
    },
    {
        "origin_intersection": "winters st Mcclellan, California and I-80",
        "x_coords": "-121.4114857",
        "y_coords": "38.6373529"
    },
    {
        "origin_intersection": "winters st Sacramento, California and I-80",
        "x_coords": "-121.4114781",
        "y_coords": "38.6373332"
    },
    {
        "origin_intersection": "winton ave Hayward, California and I-880",
        "x_coords": "-122.1014884",
        "y_coords": "37.6564226"
    },
    {
        "origin_intersection": "wonderland blvd Mountain Gate, California and I-5",
        "x_coords": "-122.3390298",
        "y_coords": "40.7064998"
    },
    {
        "origin_intersection": "worth rd Porterville, California and SR-190",
        "x_coords": "-118.9371633",
        "y_coords": "36.0525608"
    },
    {
        "origin_intersection": "yosemite ave Oakdale, California and SR-120",
        "x_coords": "-120.8470007",
        "y_coords": "37.7666409"
    },
    {
        "origin_intersection": "yosemite national park 1 Ahwahnee Drive, Yosemite Valley, Ca, 95389, California and SR-120",
        "x_coords": "-119.7975817",
        "y_coords": "37.7524488"
    },
    {
        "origin_intersection": "yucca mesa rd Yucca Valley, California and SR-62",
        "x_coords": "-116.3697003",
        "y_coords": "34.134896"
    },
    {
        "origin_intersection": "zhone way 5601 Oakport St, Oakland, Ca 94621, California and I-880",
        "x_coords": "-122.2080746",
        "y_coords": "37.7539217"
    },
    {
        "origin_intersection": "zinfandel ln 550 Community Hall Ln, St Helena, Ca 94574, California and SR-29",
        "x_coords": "-122.4426043",
        "y_coords": "38.4825944"
    }    
]

In [25]:
type(origin_intersections)

list

In [26]:
# a list of locations pulled from the subset of OSOW permit routes
destination_locations = [
    {
        "destination_intersection": "albers rd and * Oakdale, California",
        "x_coords": "-120.8399751",
        "y_coords": "37.7448711"
    },
    {
        "destination_intersection": "bolinas rd and bolinas rd Bolinas, California",
        "x_coords": "-122.7000656",
        "y_coords": "37.9356238"
    },
    {
        "destination_intersection": "business 58 east and barstow hwy Mojave, California",
        "x_coords": "-118.1488136",
        "y_coords": "35.0426269"
    },
    {
        "destination_intersection": "cajalco rd and I-215 Perris, California",
        "y_coords": "33.8437802"
    },
    {
        "destination_intersection": "del amo bl. oc over rte 405 and * Long Beach, California",
        "x_coords": "-118.2707909",
        "y_coords": "33.8465039"
    },
    {
        "destination_intersection": "evan hewes hwy and I-80 El Centro, California",
        "x_coords": "-115.9942003",
        "y_coords": "32.7342349"
    },
    {
        "destination_intersection": "galleano ranch road and galleano ranch rd 11991 Landon Dr., Mira Loma, Ca, 91752, California",
        "x_coords": "-117.5503454",
        "y_coords": "34.0022325"
    },
    {
        "destination_intersection": "galleano ranch road and galleano ranch rd Ontario, California",
        "x_coords": "-117.5503371",
        "y_coords": "34.0021773"
    },
    {
        "destination_intersection": "I-10 and alabama st Highland, California",
        "x_coords": "-117.2087283",
        "y_coords": "34.0665637"
    },
    {
        "destination_intersection": "I-10 and archibald ave Ontario, California",
        "x_coords": "-117.59341",
        "y_coords": "34.0673499"
    },
    {
        "destination_intersection": "I-10 and Arizona Line",
        "x_coords": "-114.5311155",
        "y_coords": "33.6047178"
    },
    {
        "destination_intersection": "I-10 and cactus rest area Cactus City, California",
        "x_coords": "-115.9638269",
        "y_coords": "33.6783822"
    },
    {
        "destination_intersection": "I-10 and cedar ave  Bloomington, California",
        "x_coords": "-117.3961789",
        "y_coords": "34.0684486"
    },
    {
        "destination_intersection": "I-10 and cherry ave Fontana, California",
        "x_coords": "-117.4889461",
        "y_coords": "34.0660911"
    },
    {
        "destination_intersection": "I-10 and chiriaco summit Indio, California",
        "x_coords": "-115.7236739",
        "y_coords": "33.6600664"
    },
    {
        "destination_intersection": "I-10 and chiriaco summit n/b off ramp Indio, California",
        "x_coords": "-115.7234453",
        "y_coords": "33.6604769"
    },
    {
        "destination_intersection": "I-10 and citrus ave  Fontana, California",
        "x_coords": "-117.4536429",
        "y_coords": "34.0669222"
    },
    {
        "destination_intersection": "I-10 and cook st Indian Wells, California",
        "x_coords": "-116.3551141",
        "y_coords": "33.785484"
    },
    {
        "destination_intersection": "I-10 and dillon rd Desert Hot Springs, California",
        "x_coords": "-116.1727376",
        "y_coords": "33.7150681"
    },
    {
        "destination_intersection": "I-10 and etiwanda ave  Fontana, Ca, California",
        "x_coords": "-117.5240965",
        "y_coords": "34.0674733"
    },
    {
        "destination_intersection": "I-10 and etiwanda ave Ontario, California",
        "x_coords": "-117.5240856",
        "y_coords": "34.0675478"
    },
    {
        "destination_intersection": "I-10 and etiwanda ave Fontana, California",
        "x_coords": "-117.5241102",
        "y_coords": "34.0675257"
    },
    {
        "destination_intersection": "I-10 and garvey ave 3301 Gilman Rd, El Monte, Ca, 91732, California",
        "x_coords": "-118.0113345",
        "y_coords": "34.063632"
    },
    {
        "destination_intersection": "I-10 and golf center parkway 45600 Citrus Ave, Indio, Ca, California",
        "x_coords": "-116.2015528",
        "y_coords": "33.7260449"
    },
    {
        "destination_intersection": "I-10 and hargrave st Banning, California",
        "x_coords": "-116.8679817",
        "y_coords": "33.9242775"
    },
    {
        "destination_intersection": "I-10 and haven ave Rancho Cucamonga, California",
        "x_coords": "-117.5758004",
        "y_coords": "34.0674599"
    },
    {
        "destination_intersection": "I-10 and indian hill blvd Claremont, California",
        "x_coords": "-117.7192969",
        "y_coords": "34.0817385"
    },
    {
        "destination_intersection": "I-10 and intake blvd Blythe, California",
        "x_coords": "-114.5707033",
        "y_coords": "33.6074328"
    },
    {
        "destination_intersection": "I-10 and live oak canyon rd Yucaipa, California",
        "x_coords": "-117.0960153",
        "y_coords": "34.0167033"
    },
    {
        "destination_intersection": "I-10 and main st Cabazon, Ca, California",
        "x_coords": "-116.806466",
        "y_coords": "33.9212912"
    },
    {
        "destination_intersection": "I-10 and mecca Cactus City, California",
        "x_coords": "-115.9641112",
        "y_coords": "33.6776035"
    },
    {
        "destination_intersection": "I-10 and milliken ave Fontana, California",
        "x_coords": "-117.5582023",
        "y_coords": "34.0674249"
    },
    {
        "destination_intersection": "I-10 and monterey ave Rancho Mirage, California",
        "x_coords": "-116.3881637",
        "y_coords": "33.8063591"
    },
    {
        "destination_intersection": "I-10 and mountain ave Bloomington, California",
        "x_coords": "-117.6701471",
        "y_coords": "34.0873092"
    },
    {
        "destination_intersection": "I-10 and mountain ave Upland, California",
        "x_coords": "-117.6701588",
        "y_coords": "34.0872818"
    },
    {
        "destination_intersection": "I-10 and mt vernon ave Colton, California",
        "x_coords": "-117.3136971",
        "y_coords": "34.0648006"
    },
    {
        "destination_intersection": "I-10 and palm dr 66575 2Nd St, Desert Hot Springs, Ca 92240, California",
        "x_coords": "-116.5037388",
        "y_coords": "33.8795722"
    },
    {
        "destination_intersection": "I-10 and palm dr Desert Hot Springs, California",
        "x_coords": "-116.5036673",
        "y_coords": "33.8795409"
    },
    {
        "destination_intersection": "I-10 and rancho ave Colton, California",
        "x_coords": "-117.3358446",
        "y_coords": "34.0674456"
    },
    {
        "destination_intersection": "I-10 and riverside ave Rialto, California",
        "x_coords": "-117.3703085",
        "y_coords": "34.0688419"
    },
    {
        "destination_intersection": "I-10 and riverside ave Bloomington, California",
        "x_coords": "-117.3702547",
        "y_coords": "34.0688531"
    },
    {
        "destination_intersection": "I-10 and riverside ave Riverside, California",
        "x_coords": "-117.3879081",
        "y_coords": "33.9475436"
    },
    {
        "destination_intersection": "I-10 and sierra ave Bloomington, California",
        "x_coords": "-117.4356641",
        "y_coords": "34.0673262"
    },
    {
        "destination_intersection": "I-10 and singleton rd Cherry Valley, California",
        "x_coords": "-117.0501687",
        "y_coords": "33.9790422"
    },
    {
        "destination_intersection": "I-10 and soto st Los Angeles, California",
        "x_coords": "-118.2026497",
        "y_coords": "34.0548466"
    },
    {
        "destination_intersection": "I-10 and SR-20 Santa Monica, California",
        "x_coords": "-118.4737098",
        "y_coords": "34.0232409"
    },
    {
        "destination_intersection": "I-10 and SR-83 145 N 8 Ave, Upland, Ca 91786, California",
        "x_coords": "-117.6510616",
        "y_coords": "34.0872099"
    },
    {
        "destination_intersection": "I-10 and sunset ave Baldwin Park, California",
        "x_coords": "-117.9361555",
        "y_coords": "34.072395"
    },
    {
        "destination_intersection": "I-10 and sunset ave Banning, California",
        "x_coords": "-116.911818",
        "y_coords": "33.9245844"
    },
    {
        "destination_intersection": "I-10 and tippecanoe ave San Bernardino, Ca, California",
        "x_coords": "-117.2611981",
        "y_coords": "34.0644341"
    },
    {
        "destination_intersection": "I-10 and tippecanoe ave San Bernardino, California",
        "x_coords": "-117.2611984",
        "y_coords": "34.0644344"
    },
    {
        "destination_intersection": "I-10 and vineyard ave Ontario, California",
        "x_coords": "-117.6110988",
        "y_coords": "34.0706798"
    },
    {
        "destination_intersection": "I-10 and washington st Palm Desert, California",
        "x_coords": "-116.3013546",
        "y_coords": "33.7604296"
    },
    {
        "destination_intersection": "I-10 and wileys well rd Blythe, California",
        "x_coords": "-114.9016314",
        "y_coords": "33.6079671"
    },
    {
        "destination_intersection": "I-10 and wileys well rd Buttonwillow, California",
        "x_coords": "-114.9016484",
        "y_coords": "33.6079509"
    },
    {
        "destination_intersection": "I-10 and wileys well rd Colorado Sub Station - Powerline Rd Blyth, California",
        "x_coords": "-114.9016376",
        "y_coords": "33.6079631"
    },
    {
        "destination_intersection": "I-105 and bellflower blvd Downey, Ca, California",
        "x_coords": "-118.1253983",
        "y_coords": "33.9130304"
    },
    {
        "destination_intersection": "I-105 and crenshaw blvd 1 Rocket Rd Hawthorne Ca, California",
        "x_coords": "-118.326514",
        "y_coords": "33.9252445"
    },
    {
        "destination_intersection": "I-105 and crenshaw blvd Spacex (Hawthorne), California",
        "x_coords": "-118.326544",
        "y_coords": "33.9251237"
    },
    {
        "destination_intersection": "I-105 and crenshaw blvd Hawthorne, Ca, California",
        "x_coords": "-118.3266076",
        "y_coords": "33.9251609"
    },
    {
        "destination_intersection": "I-105 and crenshaw blvd Hawthorne, California",
        "x_coords": "-118.3265143",
        "y_coords": "33.9252139"
    },
    {
        "destination_intersection": "I-105 and crenshaw blvd Space X (Hawthorne), California",
        "x_coords": "-118.3263355",
        "y_coords": "33.925144"
    },
    {
        "destination_intersection": "I-105 and la cienega bl Los Angeles, California",
        "x_coords": "-118.3700177",
        "y_coords": "33.9299973"
    },
    {
        "destination_intersection": "I-105 and la cienega blvd 5721 W SR-99, Los Angeles, Ca, California",
        "x_coords": "-118.3700501",
        "y_coords": "33.929908"
    },
    {
        "destination_intersection": "I-105 and la cienega blvd Los Angeles, California",
        "x_coords": "-118.3700462",
        "y_coords": "33.9299587"
    },
    {
        "destination_intersection": "I-105 and la cienega blvd National Technical System (Los Angeles), California",
        "x_coords": "-118.3700676",
        "y_coords": "33.9300319"
    },
    {
        "destination_intersection": "I-105 and nash st El Segundo, California",
        "x_coords": "-118.3874855",
        "y_coords": "33.9310884"
    },
    {
        "destination_intersection": "I-105 and SR-1 El Segundo, California",
        "x_coords": "-118.3960853",
        "y_coords": "33.9314229"
    },
    {
        "destination_intersection": "I-105 and SR-99 Los Angeles, California",
        "x_coords": "-118.2918626",
        "y_coords": "33.9288626"
    },
    {
        "destination_intersection": "I-105 and SR-99 Playa Del Rey, California",
        "x_coords": "-118.2918197",
        "y_coords": "33.9287795"
    },
    {
        "destination_intersection": "I-105 and vermont ave Gardena, California",
        "x_coords": "-118.2917628",
        "y_coords": "33.92869"
    },
    {
        "destination_intersection": "I-105 and vermont ave Los Angeles, California",
        "x_coords": "-118.2917896",
        "y_coords": "33.9286249"
    },
    {
        "destination_intersection": "I-105 and wilmington ave Lynwood, California",
        "x_coords": "-118.239158",
        "y_coords": "33.9282786"
    },
    {
        "destination_intersection": "I-105 and wilmington ave Compton, California",
        "x_coords": "-118.2390511",
        "y_coords": "33.9283039"
    },
    {
        "destination_intersection": "I-15 and aero dr San Diego, California",
        "x_coords": "-117.1141026",
        "y_coords": "32.8103495"
    },
    {
        "destination_intersection": "I-15 and balboa ave 4200 Ruffin Rd, San Diego Ca. 92123, California",
        "x_coords": "-117.11649",
        "y_coords": "32.8229195"
    },
    {
        "destination_intersection": "I-15 and balboa ave San Diego, California",
        "x_coords": "-117.1165874",
        "y_coords": "32.8229064"
    },
    {
        "destination_intersection": "I-15 and baseline ave Rancho Cucamonga, California",
        "x_coords": "-117.5141015",
        "y_coords": "34.1212289"
    },
    {
        "destination_intersection": "I-15 and camino del norte Rancho Bernardo, California",
        "x_coords": "-117.0832973",
        "y_coords": "32.9976631"
    },
    {
        "destination_intersection": "I-15 and carmel mountain rd San Diego, California",
        "x_coords": "-117.2345409",
        "y_coords": "32.917801"
    },
    {
        "destination_intersection": "I-15 and carroll canyon rd San Diego, Ca, California",
        "x_coords": "-117.1162125",
        "y_coords": "32.9034001"
    },
    {
        "destination_intersection": "I-15 and centre city pkwy Escondido, California",
        "x_coords": "-117.0710495",
        "y_coords": "33.0822349"
    },
    {
        "destination_intersection": "I-15 and clinton keith rd Wildomar, California",
        "x_coords": "-117.2449561",
        "y_coords": "33.5958092"
    },
    {
        "destination_intersection": "I-15 and 'd' st Adelanto, California",
        "x_coords": "-117.3005561",
        "y_coords": "34.544911"
    },
    {
        "destination_intersection": "I-15 and deer springs rd 414 Smilax Rd, San Marcos, Ca 92078, California",
        "x_coords": "-117.1257569",
        "y_coords": "33.1973097"
    },
    {
        "destination_intersection": "I-15 and duncan canyon rd Fontana, California",
        "x_coords": "-117.4611762",
        "y_coords": "34.1650694"
    },
    {
        "destination_intersection": "I-15 and friars rd San Diego, California",
        "x_coords": "-117.1139037",
        "y_coords": "32.7883044"
    },
    {
        "destination_intersection": "I-15 and ft irwin rd Ft Irwin, California",
        "x_coords": "-116.9262574",
        "y_coords": "34.9017292"
    },
    {
        "destination_intersection": "I-15 and hidden valley parkway Corona, California",
        "x_coords": "-117.5597673",
        "y_coords": "33.8984722"
    },
    {
        "destination_intersection": "I-15 and hidden valley pkwy Corona, California",
        "x_coords": "-117.5596815",
        "y_coords": "33.8982585"
    },
    {
        "destination_intersection": "I-15 and indian truck trail Corona, California",
        "x_coords": "-117.4524298",
        "y_coords": "33.7487283"
    },
    {
        "destination_intersection": "I-15 and indian truck trail e/b off ramp * Corona, California",
        "x_coords": "-117.4523642",
        "y_coords": "33.7486447"
    },
    {
        "destination_intersection": "I-15 and jurupa st 4265 E Guasti Rd, Ontario, Ca, California",
        "x_coords": "-117.550006",
        "y_coords": "34.0480658"
    },
    {
        "destination_intersection": "I-15 and jurupa st Fontana, California",
        "x_coords": "-117.5499495",
        "y_coords": "34.0480589"
    },
    {
        "destination_intersection": "I-15 and l st Barstow, California",
        "x_coords": "-117.0497524",
        "y_coords": "34.884575"
    },
    {
        "destination_intersection": "I-15 and la mesa rd Victorville, California",
        "x_coords": "-117.3351833",
        "y_coords": "34.485183"
    },
    {
        "destination_intersection": "I-15 and limonite ave Ontario, California",
        "x_coords": "-117.5489029",
        "y_coords": "33.9747513"
    },
    {
        "destination_intersection": "I-15 and limonite ave Eastvale, California",
        "x_coords": "-117.5488398",
        "y_coords": "33.9745122"
    },
    {
        "destination_intersection": "I-15 and market st San Diego, California",
        "x_coords": "-117.1188883",
        "y_coords": "32.7115163"
    },
    {
        "destination_intersection": "I-15 and murrieta hot springs rd Murrieta, California",
        "x_coords": "-117.1921056",
        "y_coords": "33.5547272"
    },
    {
        "destination_intersection": "I-15 and Nevada Line",
        "x_coords": "-115.392266",
        "y_coords": "35.6048034"
    },
    {
        "destination_intersection": "I-15 and nipton rd Nevada, California",
        "x_coords": "-115.4487745",
        "y_coords": "35.4733664"
    },
    {
        "destination_intersection": "I-15 and ocean view blvd San Diego, California",
        "x_coords": "-117.1207605",
        "y_coords": "32.7012348"
    },
    {
        "destination_intersection": "I-15 and old highway Escondido, California",
        "x_coords": "-117.1495296",
        "y_coords": "33.2862661"
    },
    {
        "destination_intersection": "I-15 and pomerado rd 9220 Activity Rd, San Diego, Ca 92126, California",
        "x_coords": "-117.1173447",
        "y_coords": "32.8936268"
    },
    {
        "destination_intersection": "I-15 and pomerado rd Poway, California",
        "x_coords": "-117.0714901",
        "y_coords": "33.0558413"
    },
    {
        "destination_intersection": "I-15 and rancho bernardo rd Rancho Bernardo, Ca, California",
        "x_coords": "-117.0787762",
        "y_coords": "33.0231197"
    },
    {
        "destination_intersection": "I-15 and rancho bernardo rd San Diego, California",
        "x_coords": "-117.078778",
        "y_coords": "33.0231224"
    },
    {
        "destination_intersection": "I-15 and second st Norco, California",
        "x_coords": "-117.5585725",
        "y_coords": "33.9096549"
    },
    {
        "destination_intersection": "I-15 and sixth st 1954 Tandem Way Norco, Ca, California",
        "x_coords": "-117.5559851",
        "y_coords": "33.9378552"
    },
    {
        "destination_intersection": "I-15 and SR-18 Oro Grande, California",
        "x_coords": "-117.3006392",
        "y_coords": "34.5452523"
    },
    {
        "destination_intersection": "I-15 and SR-18 Victorville, California",
        "x_coords": "-117.3005492",
        "y_coords": "34.5450974"
    },
    {
        "destination_intersection": "I-15 and SR-4 10825 7Th St Rancho Cucamonga, Ca, California",
        "x_coords": "-117.5448541",
        "y_coords": "34.0771812"
    },
    {
        "destination_intersection": "I-15 and SR-4 Rancho Cucamonga, California",
        "x_coords": "-117.5448563",
        "y_coords": "34.0771735"
    },
    {
        "destination_intersection": "I-15 and SR-60 Riverside, California",
        "x_coords": "-117.5497395",
        "y_coords": "34.0211117"
    },
    {
        "destination_intersection": "I-15 and SR-60 Colton (2650 S La Cadena Dr), California",
        "x_coords": "-117.5499154",
        "y_coords": "34.0211929"
    },
    {
        "destination_intersection": "I-15 and SR-9 Escondido, California",
        "x_coords": "-117.0949768",
        "y_coords": "33.1084094"
    },
    {
        "destination_intersection": "I-15 and SR-91 Corona, California",
        "x_coords": "-117.5472561",
        "y_coords": "33.8816714"
    },
    {
        "destination_intersection": "I-15 and temescal cyn rd Corona, California",
        "x_coords": "-117.4872961",
        "y_coords": "33.7766883"
    },
    {
        "destination_intersection": "I-15 and valley parkway 21687 W Valley Parkway, Escondido, California",
        "x_coords": "-117.0991246",
        "y_coords": "33.1153479"
    },
    {
        "destination_intersection": "I-15 and wabash blvd San Diego, California",
        "x_coords": "-117.1178646",
        "y_coords": "32.7163761"
    },
    {
        "destination_intersection": "I-15 and west bernardo dr Escondido, California",
        "x_coords": "-117.0991118",
        "y_coords": "33.1153359"
    },
    {
        "destination_intersection": "I-15 and winchester rd Murrieta, California",
        "x_coords": "-117.1629534",
        "y_coords": "33.5224838"
    },
    {
        "destination_intersection": "I-15 and yates well rd Nipton, California",
        "x_coords": "-115.419702",
        "y_coords": "35.5408779"
    },
    {
        "destination_intersection": "I-15 and yermo Yermo, California",
        "x_coords": "-116.8380741",
        "y_coords": "34.9092487"
    },
    {
        "destination_intersection": "I-205 and macarthur dr Tracy, California",
        "x_coords": "-121.4147387",
        "y_coords": "37.7642675"
    },
    {
        "destination_intersection": "I-205 and mountain house pkwy Tracy, California",
        "x_coords": "-121.5316576",
        "y_coords": "37.7422035"
    },
    {
        "destination_intersection": "I-215 and alessandro blvd Riverside, California",
        "x_coords": "-117.2875326",
        "y_coords": "33.916806"
    },
    {
        "destination_intersection": "I-215 and alessandro blvd Moreno Valley, California",
        "x_coords": "-117.2875326",
        "y_coords": "33.9163074"
    },
    {
        "destination_intersection": "I-215 and barton rd Grand Terrace 22220 Barton Rd, California",
        "x_coords": "-117.325068",
        "y_coords": "34.0338321"
    },
    {
        "destination_intersection": "I-215 and blaine st Riverside, California",
        "x_coords": "-117.3428654",
        "y_coords": "33.9829714"
    },
    {
        "destination_intersection": "I-215 and clinton keith rd Murrieta, California",
        "x_coords": "-117.1744249",
        "y_coords": "33.5984109"
    },
    {
        "destination_intersection": "I-215 and ethanac rd Menifee, California",
        "x_coords": "-117.1894294",
        "y_coords": "33.7429545"
    },
    {
        "destination_intersection": "I-215 and ethanac rd Perris, California",
        "x_coords": "-117.1894401",
        "y_coords": "33.7429099"
    },
    {
        "destination_intersection": "I-215 and ethanac rd Romoland, California",
        "x_coords": "-117.1887428",
        "y_coords": "33.7430259"
    },
    {
        "destination_intersection": "I-215 and harley knox blvd Perris, California",
        "x_coords": "-117.2597002",
        "y_coords": "33.8625454"
    },
    {
        "destination_intersection": "I-215 and iowa ave 20 Iowa Ave, Riverside, Ca, 92507, California",
        "x_coords": "-117.3353329",
        "y_coords": "34.0236619"
    },
    {
        "destination_intersection": "I-215 and iowa ave 800 E La Cadena Dr, Riverside, Ca, 92507, California",
        "x_coords": "-117.3355469",
        "y_coords": "34.0239097"
    },
    {
        "destination_intersection": "I-215 and iowa ave Grand Terrace, California",
        "x_coords": "-117.335498",
        "y_coords": "34.0238588"
    },
    {
        "destination_intersection": "I-215 and murrieta hot springs rd Murrieta, Ca, California",
        "x_coords": "-117.1808811",
        "y_coords": "33.556148"
    },
    {
        "destination_intersection": "I-215 and newport rd Menifee, California",
        "x_coords": "-117.1714378",
        "y_coords": "33.684813"
    },
    {
        "destination_intersection": "I-215 and nuevo rd Nuevo, California",
        "x_coords": "-117.2327137",
        "y_coords": "33.8025015"
    },
    {
        "destination_intersection": "I-215 and palm ave San Bernardino, California",
        "x_coords": "-117.3614036",
        "y_coords": "34.1909263"
    },
    {
        "destination_intersection": "I-215 and placentia ave Perris, California",
        "x_coords": "-117.2261411",
        "y_coords": "33.7904597"
    },
    {
        "destination_intersection": "I-215 and ramona expwy e/b off ramp * Perris, California",
        "x_coords": "-117.2504861",
        "y_coords": "33.8438283"
    },
    {
        "destination_intersection": "I-215 and ramona expwy Perris, California",
        "x_coords": "-117.2508563",
        "y_coords": "33.843766"
    },
    {
        "destination_intersection": "I-215 and ramona expwy Riverside, California",
        "x_coords": "-117.2508294",
        "y_coords": "33.8438283"
    },
    {
        "destination_intersection": "I-215 and ramona expwy San Jacinto, California",
        "x_coords": "-117.2508958",
        "y_coords": "33.8437788"
    },
    {
        "destination_intersection": "I-215 and scott rd Menifee, California",
        "x_coords": "-117.1712087",
        "y_coords": "33.6417478"
    },
    {
        "destination_intersection": "I-215 and university parkway San Bernardino, California",
        "x_coords": "-117.3332737",
        "y_coords": "34.164392"
    },
    {
        "destination_intersection": "I-215 and van buren blvd Riverside, California",
        "x_coords": "-117.2699709",
        "y_coords": "33.8876494"
    },
    {
        "destination_intersection": "I-215 and washington st Colton, California",
        "x_coords": "-117.3075905",
        "y_coords": "34.0475679"
    },
    {
        "destination_intersection": "I-280 and alpine rd Stanford, California",
        "x_coords": "-122.1948242",
        "y_coords": "37.4087906"
    },
    {
        "destination_intersection": "I-280 and bird ave San Jose, California",
        "x_coords": "-121.898165",
        "y_coords": "37.3222572"
    },
    {
        "destination_intersection": "I-280 and foothill expressway Cupertino, California",
        "x_coords": "-122.0684104",
        "y_coords": "37.3343507"
    },
    {
        "destination_intersection": "I-280 and lawrence expressway Santa Clara, California",
        "x_coords": "-121.9953508",
        "y_coords": "37.3199093"
    },
    {
        "destination_intersection": "I-280 and sand hill rd Menlo Park, California",
        "x_coords": "-122.2194555",
        "y_coords": "37.4189357"
    },
    {
        "destination_intersection": "I-280 and saratoga ave San Jose (1960 Quito Rd), California",
        "x_coords": "-121.9746689",
        "y_coords": "37.3170683"
    },
    {
        "destination_intersection": "I-280 and saratoga ave San Jose, California",
        "x_coords": "-121.9749313",
        "y_coords": "37.3173013"
    },
    {
        "destination_intersection": "I-40 and amboy/kelso Amboy, California",
        "x_coords": "-115.7563233",
        "y_coords": "34.5553649"
    },
    {
        "destination_intersection": "I-40 and Arizona Line",
        "x_coords": "-114.4878107",
        "y_coords": "34.7171822"
    },
    {
        "destination_intersection": "I-40 and daggett Yermo, California",
        "x_coords": "-116.8871242",
        "y_coords": "34.902274"
    },
    {
        "destination_intersection": "I-40 and essex rd Essex, California",
        "x_coords": "-115.320485",
        "y_coords": "34.800946"
    },
    {
        "destination_intersection": "I-40 and goffs rd Needles, California",
        "x_coords": "-115.1902353",
        "y_coords": "34.8121817"
    },
    {
        "destination_intersection": "I-40 and kelbaker rd Amboy, California",
        "x_coords": "-115.6784579",
        "y_coords": "34.720997"
    },
    {
        "destination_intersection": "I-40 and ludlow Ludlow, California",
        "x_coords": "-116.163163",
        "y_coords": "34.7254931"
    },
    {
        "destination_intersection": "I-40 and marine corps Fort Irwin, Ca, California",
        "x_coords": "-116.7010334",
        "y_coords": "35.2447011"
    },
    {
        "destination_intersection": "I-40 and marine corps Ludlow, California",
        "x_coords": "-116.1631801",
        "y_coords": "34.7253687"
    },
    {
        "destination_intersection": "I-405 and artesia blvd Redondo Beach, California",
        "x_coords": "-118.3421001",
        "y_coords": "33.8727905"
    },
    {
        "destination_intersection": "I-405 and cherry ave 3276 Walnut Ave Signal Hill Ca, California",
        "x_coords": "-118.167636",
        "y_coords": "33.8135531"
    },
    {
        "destination_intersection": "I-405 and cherry ave Long Beach, California",
        "x_coords": "-118.1676236",
        "y_coords": "33.8135344"
    },
    {
        "destination_intersection": "I-405 and cherry ave Signal Hill, California",
        "x_coords": "-118.167619",
        "y_coords": "33.8135059"
    },
    {
        "destination_intersection": "I-405 and crenshaw blvd Torrance, California",
        "x_coords": "-118.3264699",
        "y_coords": "33.8645865"
    },
    {
        "destination_intersection": "I-405 and el segundo blvd El Segundo Blvd, California",
        "x_coords": "-118.3702776",
        "y_coords": "33.9161801"
    },
    {
        "destination_intersection": "I-405 and el segundo blvd El Segundo, California",
        "x_coords": "-118.370208",
        "y_coords": "33.9162045"
    },
    {
        "destination_intersection": "I-405 and harbor blvd Fountain Valley, California",
        "x_coords": "-117.9193906",
        "y_coords": "33.689105"
    },
    {
        "destination_intersection": "I-405 and lake forest dr Lake Forest, California",
        "x_coords": "-117.7205245",
        "y_coords": "33.6281025"
    },
    {
        "destination_intersection": "I-405 and lakewood blvd Long Beach, California",
        "x_coords": "-118.1426269",
        "y_coords": "33.8057654"
    },
    {
        "destination_intersection": "I-405 and long beach blvd Long Beach, California",
        "x_coords": "-118.1893999",
        "y_coords": "33.8174628"
    },
    {
        "destination_intersection": "I-405 and manchester blvd Los Angeles City, California",
        "x_coords": "-118.3694269",
        "y_coords": "33.9615917"
    },
    {
        "destination_intersection": "I-405 and national blvd Los Angeles, California",
        "x_coords": "-118.430062",
        "y_coords": "34.0268097"
    },
    {
        "destination_intersection": "I-405 and nordhoff st North Hills, California",
        "x_coords": "-118.4730631",
        "y_coords": "34.2355349"
    },
    {
        "destination_intersection": "I-405 and orange ave Tujunga, California",
        "x_coords": "-118.176274",
        "y_coords": "33.8143441"
    },
    {
        "destination_intersection": "I-405 and roscoe blvd Van Nuys, California",
        "x_coords": "-118.4732084",
        "y_coords": "34.2214095"
    },
    {
        "destination_intersection": "I-405 and santa monica blvd Los Angeles, California",
        "x_coords": "-118.4467134",
        "y_coords": "34.0469739"
    },
    {
        "destination_intersection": "I-405 and sepulveda blvd Encino, California",
        "x_coords": "-118.4769923",
        "y_coords": "34.0965551"
    },
    {
        "destination_intersection": "I-405 and spring st Long Beach, California",
        "x_coords": "-118.1616938",
        "y_coords": "33.8116623"
    },
    {
        "destination_intersection": "I-405 and sunset blvd Los Angeles, California",
        "x_coords": "-118.4666217",
        "y_coords": "34.0727977"
    },
    {
        "destination_intersection": "I-405 and western ave Torrance, California",
        "x_coords": "-118.3091716",
        "y_coords": "33.8598282"
    },
    {
        "destination_intersection": "I-405 and wilmington ave Carson, Ca 90810, California",
        "x_coords": "-118.2416566",
        "y_coords": "33.8258334"
    },
    {
        "destination_intersection": "I-405 and wilmington ave Wilmington, California",
        "x_coords": "-118.2416298",
        "y_coords": "33.8258489"
    },
    {
        "destination_intersection": "I-405 and wilmington ave 2040 E 220Th St, Carson, Ca 90810, California",
        "x_coords": "-118.2416781",
        "y_coords": "33.8258289"
    },
    {
        "destination_intersection": "I-405 and wilmington ave Carson, California",
        "x_coords": "-118.2416888",
        "y_coords": "33.8258467"
    },
    {
        "destination_intersection": "I-5 and `h` st Chula Vista, California",
        "x_coords": "-117.0958956",
        "y_coords": "32.629167"
    },
    {
        "destination_intersection": "I-5 and alameda ave 164 West Magnolia Blvd, Burbank, Ca, 91501, California",
        "x_coords": "-118.3042201",
        "y_coords": "34.1702673"
    },
    {
        "destination_intersection": "I-5 and alton parkway Foothill Ranch, California",
        "x_coords": "-117.7439144",
        "y_coords": "33.6556989"
    },
    {
        "destination_intersection": "I-5 and auction yard rd Cottonwood, California",
        "x_coords": "-122.2812161",
        "y_coords": "40.3268724"
    },
    {
        "destination_intersection": "I-5 and avenida califia San Clemente, California",
        "x_coords": "-117.6009352",
        "y_coords": "33.4086719"
    },
    {
        "destination_intersection": "I-5 and avenida pico San Juan Capistrano, California",
        "x_coords": "-117.6224827",
        "y_coords": "33.438787"
    },
    {
        "destination_intersection": "I-5 and basilone rd Colton, California",
        "x_coords": "-117.5831145",
        "y_coords": "33.3854348"
    },
    {
        "destination_intersection": "I-5 and bay marina wy National City, California",
        "x_coords": "-117.108054",
        "y_coords": "32.6599302"
    },
    {
        "destination_intersection": "I-5 and bowman rd Cottonwood, California",
        "x_coords": "-122.2841572",
        "y_coords": "40.3673559"
    },
    {
        "destination_intersection": "I-5 and burbank blvd Burbank, California",
        "x_coords": "-118.3181091",
        "y_coords": "34.1853391"
    },
    {
        "destination_intersection": "I-5 and camp pendleton Camp Pendleton, California",
        "x_coords": "-117.3872739",
        "y_coords": "33.2104368"
    },
    {
        "destination_intersection": "I-5 and charter way Stockton, California",
        "x_coords": "-121.2975938",
        "y_coords": "37.9366714"
    },
    {
        "destination_intersection": "I-5 and co rd Dunnigan, California",
        "x_coords": "-121.9785049",
        "y_coords": "38.8922724"
    },
    {
        "destination_intersection": "I-5 and co rd Sacramento, California",
        "x_coords": "-121.4969629",
        "y_coords": "38.4665937"
    },
    {
        "destination_intersection": "I-5 and cosumnes river blvd Sacramento, California",
        "x_coords": "-121.4970698",
        "y_coords": "38.466575"
    },
    {
        "destination_intersection": "I-5 and derrick ave Firebaugh, California",
        "x_coords": "-120.394156",
        "y_coords": "36.4356636"
    },
    {
        "destination_intersection": "I-5 and eight mile rd Stockton, California",
        "x_coords": "-121.3726466",
        "y_coords": "38.0578577"
    },
    {
        "destination_intersection": "I-5 and eighth street Stockton, California",
        "x_coords": "-121.2945038",
        "y_coords": "37.9266875"
    },
    {
        "destination_intersection": "I-5 and el toro rd Laguna Woods, California",
        "x_coords": "-117.7074756",
        "y_coords": "33.6146894"
    },
    {
        "destination_intersection": "I-5 and el toro rd Lake Forest, California",
        "x_coords": "-117.7076877",
        "y_coords": "33.6147843"
    },
    {
        "destination_intersection": "I-5 and encinitas blvd Encinittas, California",
        "x_coords": "-117.2872086",
        "y_coords": "33.048383"
    },
    {
        "destination_intersection": "I-5 and euclid st Anaheim, California",
        "x_coords": "-117.9415647",
        "y_coords": "33.8357516"
    },
    {
        "destination_intersection": "I-5 and fawndale rd Mountain Gate, California",
        "x_coords": "-122.3201603",
        "y_coords": "40.7309264"
    },
    {
        "destination_intersection": "I-5 and florin rd Sacramento, California",
        "x_coords": "-121.5165825",
        "y_coords": "38.4955288"
    },
    {
        "destination_intersection": "I-5 and frazier park Frazier Park, California",
        "x_coords": "-118.8820362",
        "y_coords": "34.8175168"
    },
    {
        "destination_intersection": "I-5 and french camp Stockton, California",
        "x_coords": "-121.2838262",
        "y_coords": "37.89705"
    },
    {
        "destination_intersection": "I-5 and front st San Diego, California",
        "x_coords": "-117.1650087",
        "y_coords": "32.7242891"
    },
    {
        "destination_intersection": "I-5 and fruitridge rd Sacramento, California",
        "x_coords": "-121.5213682",
        "y_coords": "38.525593"
    },
    {
        "destination_intersection": "I-5 and gas point rd Anderson, California",
        "x_coords": "-122.2852509",
        "y_coords": "40.3893195"
    },
    {
        "destination_intersection": "I-5 and genesee ave San Diego, California",
        "x_coords": "-117.2277887",
        "y_coords": "32.8871326"
    },
    {
        "destination_intersection": "I-5 and gilman rd Redding, California",
        "x_coords": "-122.3385208",
        "y_coords": "40.8478404"
    },
    {
        "destination_intersection": "I-5 and harbor dr San Diego, California",
        "x_coords": "-117.1123182",
        "y_coords": "32.669139"
    },
    {
        "destination_intersection": "I-5 and henley / hornbrook Hornbrook, California",
        "x_coords": "-122.5743744",
        "y_coords": "41.9196636"
    },
    {
        "destination_intersection": "I-5 and I-5 Acampo, California",
        "x_coords": "-121.262363",
        "y_coords": "38.1744438"
    },
    {
        "destination_intersection": "I-5 and I-80 Auburn, California",
        "x_coords": "-121.5171377",
        "y_coords": "38.6250414"
    },
    {
        "destination_intersection": "I-5 and I-80 Sacramento, California",
        "x_coords": "-121.5171669",
        "y_coords": "38.6249834"
    },
    {
        "destination_intersection": "I-5 and jamboree rd 14191 Myford Rd, Tustin, Ca, California",
        "x_coords": "-117.79512",
        "y_coords": "33.7195999"
    },
    {
        "destination_intersection": "I-5 and jayne ave Huron, California",
        "x_coords": "-120.1588951",
        "y_coords": "36.1377692"
    },
    {
        "destination_intersection": "I-5 and jayne ave Stratford, California",
        "x_coords": "-120.158869",
        "y_coords": "36.1377384"
    },
    {
        "destination_intersection": "I-5 and jeffrey rd Irvine, California",
        "x_coords": "-117.7684373",
        "y_coords": "33.6898947"
    },
    {
        "destination_intersection": "I-5 and katella ave Orange, California",
        "x_coords": "-117.902322",
        "y_coords": "33.8032634"
    },
    {
        "destination_intersection": "I-5 and lake hughes rd Castaic, California",
        "x_coords": "-118.6238173",
        "y_coords": "34.4958539"
    },
    {
        "destination_intersection": "I-5 and las pulgas rd San Diego, California",
        "x_coords": "-117.4645744",
        "y_coords": "33.2987345"
    },
    {
        "destination_intersection": "I-5 and las pulgas rd San Onofre, California",
        "x_coords": "-117.4645579",
        "y_coords": "33.2987403"
    },
    {
        "destination_intersection": "I-5 and laval road Wheeler Ridge, California",
        "x_coords": "-118.9422018",
        "y_coords": "34.9911584"
    },
    {
        "destination_intersection": "I-5 and magic mountain parkway Santa Clarita, California",
        "x_coords": "-118.5835025",
        "y_coords": "34.4230632"
    },
    {
        "destination_intersection": "I-5 and main st Los Angeles, Ca, California",
        "x_coords": "-118.2162765",
        "y_coords": "34.0659363"
    },
    {
        "destination_intersection": "I-5 and march lane Stockton, California",
        "x_coords": "-121.3434053",
        "y_coords": "37.9845524"
    },
    {
        "destination_intersection": "I-5 and mathews rd Stockton, California",
        "x_coords": "-121.2788959",
        "y_coords": "37.8820115"
    },
    {
        "destination_intersection": "I-5 and miles of cars way National City, California",
        "x_coords": "-117.1080449",
        "y_coords": "32.6599843"
    },
    {
        "destination_intersection": "I-5 and national ave 2205 Belt St. San Diego Ca. 92113, California",
        "x_coords": "-117.1351928",
        "y_coords": "32.6959877"
    },
    {
        "destination_intersection": "I-5 and national ave San Diego, California",
        "x_coords": "-117.1352228",
        "y_coords": "32.6959186"
    },
    {
        "destination_intersection": "I-5 and nees ave Firebaugh, California",
        "x_coords": "-120.7739672",
        "y_coords": "36.8503083"
    },
    {
        "destination_intersection": "I-5 and newhall ranch rd Santa Clarita, California",
        "x_coords": "-118.6057149",
        "y_coords": "34.4428946"
    },
    {
        "destination_intersection": "I-5 and oak st Stockton, California",
        "x_coords": "-121.3123027",
        "y_coords": "37.9547004"
    },
    {
        "destination_intersection": "I-5 and oceanside harbor dr Camp Pendleton, California",
        "x_coords": "-117.3872687",
        "y_coords": "33.2104951"
    },
    {
        "destination_intersection": "I-5 and old town ave San Diego, California",
        "x_coords": "-117.1933986",
        "y_coords": "32.7476629"
    },
    {
        "destination_intersection": "I-5 and Oregon Line",
        "x_coords": "-122.6154437",
        "y_coords": "42.0053409"
    },
    {
        "destination_intersection": "I-5 and oso parkway Mission Viejo, California",
        "x_coords": "-117.6717503",
        "y_coords": "33.5800824"
    },
    {
        "destination_intersection": "I-5 and palomar airport rd Carlsbad, California",
        "x_coords": "-117.322078",
        "y_coords": "33.1220228"
    },
    {
        "destination_intersection": "I-5 and pocket rd 5218 Martin Luther King Jr Blvd, Sacramento, California",
        "x_coords": "-121.5106106",
        "y_coords": "38.4812352"
    },
    {
        "destination_intersection": "I-5 and pocket rd North Highlands, California",
        "x_coords": "-121.5106077",
        "y_coords": "38.4812347"
    },
    {
        "destination_intersection": "I-5 and richards blvd Sacramento, California",
        "x_coords": "-121.5039483",
        "y_coords": "38.5970116"
    },
    {
        "destination_intersection": "I-5 and richards blvd San Francisco, California",
        "x_coords": "-121.5039796",
        "y_coords": "38.5970081"
    },
    {
        "destination_intersection": "I-5 and road Dunnigan, California",
        "x_coords": "-121.9739579",
        "y_coords": "38.8815673"
    },
    {
        "destination_intersection": "I-5 and rosecrans st San Diego, California",
        "x_coords": "-117.2048652",
        "y_coords": "32.7515131"
    },
    {
        "destination_intersection": "I-5 and rowlee rd Bakersfield, California",
        "x_coords": "-119.4748547",
        "y_coords": "35.4553678"
    },
    {
        "destination_intersection": "I-5 and rowlee rd Buttonwillow, California",
        "x_coords": "-119.4748945",
        "y_coords": "35.4553501"
    },
    {
        "destination_intersection": "I-5 and roxford st Sylmar, California",
        "x_coords": "-118.4795919",
        "y_coords": "34.303294"
    },
    {
        "destination_intersection": "I-5 and santa ana blvd Santa Ana, California",
        "x_coords": "-117.85252",
        "y_coords": "33.7542466"
    },
    {
        "destination_intersection": "I-5 and sea world dr San Diego, California",
        "x_coords": "-117.2066127",
        "y_coords": "32.7699262"
    },
    {
        "destination_intersection": "I-5 and sims rd Sims, California",
        "x_coords": "-122.3581722",
        "y_coords": "41.0718075"
    },
    {
        "destination_intersection": "I-5 and south ave Corning, California",
        "x_coords": "-122.2002129",
        "y_coords": "39.9064726"
    },
    {
        "destination_intersection": "I-5 and SR-126 Santa Clarita, California",
        "x_coords": "-118.6051195",
        "y_coords": "34.4429458"
    },
    {
        "destination_intersection": "I-5 and SR-14 Mojave, California",
        "x_coords": "-118.5084855",
        "y_coords": "34.3351789"
    },
    {
        "destination_intersection": "I-5 and SR-210 Duarte, California",
        "x_coords": "-118.488055",
        "y_coords": "34.313715"
    },
    {
        "destination_intersection": "I-5 and SR-7 Bakersfield, California",
        "x_coords": "-119.4567999",
        "y_coords": "35.4422214"
    },
    {
        "destination_intersection": "I-5 and SR-94 San Diego, California",
        "x_coords": "-117.1476234",
        "y_coords": "32.7131537"
    },
    {
        "destination_intersection": "I-5 and SR-99 Chico, California",
        "x_coords": "-122.1852829",
        "y_coords": "40.1856597"
    },
    {
        "destination_intersection": "I-5 and SR-99 Pleasant Grove, California",
        "x_coords": "-121.5410906",
        "y_coords": "38.6690371"
    },
    {
        "destination_intersection": "I-5 and templin highway Castaic, California",
        "x_coords": "-118.6879011",
        "y_coords": "34.5696472"
    },
    {
        "destination_intersection": "I-5 and templin highway Fontana, California",
        "x_coords": "-118.6879016",
        "y_coords": "34.5696405"
    },
    {
        "destination_intersection": "I-5 and templin highway Santa Clarita, California",
        "x_coords": "-118.6878393",
        "y_coords": "34.5697158"
    },
    {
        "destination_intersection": "I-5 and thornton Thornton, California",
        "x_coords": "-121.4308054",
        "y_coords": "38.22604"
    },
    {
        "destination_intersection": "I-5 and tracy Tracy, California",
        "x_coords": "-121.3284296",
        "y_coords": "37.7686144"
    },
    {
        "destination_intersection": "I-5 and twin view blvd Redding, California",
        "x_coords": "-122.3665908",
        "y_coords": "40.6229521"
    },
    {
        "destination_intersection": "I-5 and washington st San Diego, California",
        "x_coords": "-117.1825023",
        "y_coords": "32.7423508"
    },
    {
        "destination_intersection": "I-5 and west st Woodland, California",
        "x_coords": "-121.783888",
        "y_coords": "38.7030907"
    },
    {
        "destination_intersection": "I-5 and western ave Burbank, California",
        "x_coords": "-118.318258",
        "y_coords": "34.1853291"
    },
    {
        "destination_intersection": "I-5 and westley Westley, California",
        "x_coords": "-121.2676955",
        "y_coords": "37.540073"
    },
    {
        "destination_intersection": "I-505 and I-5 Oregon Line/5, California",
        "x_coords": "-122.6155149",
        "y_coords": "42.0053402"
    },
    {
        "destination_intersection": "I-505 and vaca valley parkway Vacaville, California",
        "x_coords": "-121.9498118",
        "y_coords": "38.3940055"
    },
    {
        "destination_intersection": "I-580 and corral hollow rd Tracy, California",
        "x_coords": "-121.4572663",
        "y_coords": "37.672195"
    },
    {
        "destination_intersection": "I-580 and el charro rd 52 El Charro Rd, Pleasanton, Ca, California",
        "x_coords": "-121.8494737",
        "y_coords": "37.7012447"
    },
    {
        "destination_intersection": "I-580 and el charro rd Pleasanton, California",
        "x_coords": "-121.8494376",
        "y_coords": "37.7012515"
    },
    {
        "destination_intersection": "I-580 and el charro rd Sunol, California",
        "x_coords": "-121.8495074",
        "y_coords": "37.7012749"
    },
    {
        "destination_intersection": "I-580 and hopyard rd Dublin, California",
        "x_coords": "-121.9069975",
        "y_coords": "37.7017168"
    },
    {
        "destination_intersection": "I-580 and north livermore ave 3000 Las Positas Blvd, Livermore, California",
        "x_coords": "-121.7735771",
        "y_coords": "37.7007513"
    },
    {
        "destination_intersection": "I-580 and santa rita rd 3300 Busch Rd, Pleasanton, Ca 94566, California",
        "x_coords": "-121.8720981",
        "y_coords": "37.7014588"
    },
    {
        "destination_intersection": "I-580 and santa rita rd 3300 Buch Road Pleasanton, California",
        "x_coords": "-121.8721007",
        "y_coords": "37.7014137"
    },
    {
        "destination_intersection": "I-580 and santa rita rd Pleasanton, California",
        "x_coords": "-121.8717391",
        "y_coords": "37.7011745"
    },
    {
        "destination_intersection": "I-580 and vasco rd Clayton, California",
        "x_coords": "-121.7238723",
        "y_coords": "37.7089578"
    },
    {
        "destination_intersection": "I-580 and vasco rd Livermore, Ca, California",
        "x_coords": "-121.7238588",
        "y_coords": "37.7090272"
    },
    {
        "destination_intersection": "I-580 and vasco rd Livermore, California",
        "x_coords": "-121.7238159",
        "y_coords": "37.7090655"
    },
    {
        "destination_intersection": "I-605 and carson st Long Beach, California",
        "x_coords": "-118.084062",
        "y_coords": "33.8313733"
    },
    {
        "destination_intersection": "I-605 and del amo blvd Compton, California",
        "x_coords": "-118.0922917",
        "y_coords": "33.8455259"
    },
    {
        "destination_intersection": "I-605 and florence ave Bell Gardens, California",
        "x_coords": "-118.0982853",
        "y_coords": "33.9378103"
    },
    {
        "destination_intersection": "I-605 and katella ave Seal Beach, California",
        "x_coords": "-118.08179",
        "y_coords": "33.8030641"
    },
    {
        "destination_intersection": "I-605 and lower azusa rd Irwindale, California",
        "x_coords": "-117.9917519",
        "y_coords": "34.0918114"
    },
    {
        "destination_intersection": "I-605 and peck rd Industry, California",
        "x_coords": "-118.0376017",
        "y_coords": "34.027278"
    },
    {
        "destination_intersection": "I-605 and rose hills rd City Of Industry, California",
        "x_coords": "-118.0523516",
        "y_coords": "34.0176135"
    },
    {
        "destination_intersection": "I-605 and rose hills rd Industry, Ca, California",
        "x_coords": "-118.0521799",
        "y_coords": "34.0175424"
    },
    {
        "destination_intersection": "I-605 and rose hills rd Industry, California",
        "x_coords": "-118.0522657",
        "y_coords": "34.0176847"
    },
    {
        "destination_intersection": "I-605 and rose hills rd Pico Rivera, California",
        "x_coords": "-118.0523516",
        "y_coords": "34.0176135"
    },
    {
        "destination_intersection": "I-605 and rose hills rd Whitter, California",
        "x_coords": "-118.0523471",
        "y_coords": "34.0177651"
    },
    {
        "destination_intersection": "I-605 and rose hills rd Whittier, Ca, California",
        "x_coords": "-118.052004",
        "y_coords": "34.0176231"
    },
    {
        "destination_intersection": "I-605 and slauson ave Montebello, California",
        "x_coords": "-118.0833933",
        "y_coords": "33.9672996"
    },
    {
        "destination_intersection": "I-605 and slauson ave Santa Fe Springs, California",
        "x_coords": "-118.0833493",
        "y_coords": "33.9671927"
    },
    {
        "destination_intersection": "I-605 and washington blvd Montebello, California",
        "x_coords": "-118.079964",
        "y_coords": "33.9743516"
    },
    {
        "destination_intersection": "I-680 and bayshore rd Benicia, California",
        "x_coords": "-122.1309134",
        "y_coords": "38.0644397"
    },
    {
        "destination_intersection": "I-680 and bernal ave Pleasanton, California",
        "x_coords": "-121.9020022",
        "y_coords": "37.6560519"
    },
    {
        "destination_intersection": "I-680 and calaveras blvd Milpitas, California",
        "x_coords": "-121.8886199",
        "y_coords": "37.4345129"
    },
    {
        "destination_intersection": "I-680 and I-80 Nv, California",
        "x_coords": "-122.1355675",
        "y_coords": "38.2182549"
    },
    {
        "destination_intersection": "I-680 and industrial park 401 West Channel Road, Benicia, Ca, California",
        "x_coords": "-122.1280522",
        "y_coords": "38.0680494"
    },
    {
        "destination_intersection": "I-80 and lake herman rd Benicia, California",
        "x_coords": "-122.111934",
        "y_coords": "38.085801"
    },
    {
        "destination_intersection": "I-80 and landess ave Santa Clara, California",
        "x_coords": "-121.8804756",
        "y_coords": "37.4143968"
    },
    {
        "destination_intersection": "I-80 and marina vista Martinez, California",
        "x_coords": "-122.1119815",
        "y_coords": "38.0255013"
    },
    {
        "destination_intersection": "I-80 and marina vista w/b off ramp Martinez, California",
        "x_coords": "-122.1120807",
        "y_coords": "38.025514"
    },
    {
        "destination_intersection": "I-80 and pacheco blvd Martinez, California",
        "x_coords": "-122.0904935",
        "y_coords": "38.0066547"
    },
    {
        "destination_intersection": "I-80 and sheridan rd Sunol, California",
        "x_coords": "-121.8722777",
        "y_coords": "37.6049023"
    },
    {
        "destination_intersection": "I-80 and SR-84 Sunol, California",
        "x_coords": "-121.87078",
        "y_coords": "37.5928789"
    },
    {
        "destination_intersection": "I-710 and alondra blvd Long Beach, California",
        "x_coords": "-118.2081129",
        "y_coords": "33.7700516"
    },
    {
        "destination_intersection": "I-710 and atlantic blvd Vernon, Ca, California",
        "x_coords": "-118.176642",
        "y_coords": "33.9977632"
    },
    {
        "destination_intersection": "I-710 and bandini blvd 6111 Bandini Blvd, Commerce, Ca 90040, Usa, California",
        "x_coords": "-118.1766295",
        "y_coords": "33.9978246"
    },
    {
        "destination_intersection": "I-710 and bandini blvd Vernon, California",
        "x_coords": "-118.1765981",
        "y_coords": "33.9976485"
    },
    {
        "destination_intersection": "I-710 and del amo blvd Compton, California",
        "x_coords": "-118.2055005",
        "y_coords": "33.8466182"
    },
    {
        "destination_intersection": "I-710 and firestone blvd Los Angeles, California",
        "x_coords": "-118.1703349",
        "y_coords": "33.9496722"
    },
    {
        "destination_intersection": "I-710 and firestone blvd Commerce, Ca, California",
        "x_coords": "-118.1704637",
        "y_coords": "33.9496723"
    },
    {
        "destination_intersection": "I-710 and long beach blvd Long Beach, California",
        "x_coords": "-118.1993391",
        "y_coords": "33.8625062"
    },
    {
        "destination_intersection": "I-710 and pico ave 1320 Pier C St, Long Beach, Ca 90802, California",
        "x_coords": "-118.2080406",
        "y_coords": "33.7701432"
    },
    {
        "destination_intersection": "I-710 and pico ave Carson, California",
        "x_coords": "-118.2080755",
        "y_coords": "33.770121"
    },
    {
        "destination_intersection": "I-710 and pico ave Long Beach, Ca, California",
        "x_coords": "-118.2080621",
        "y_coords": "33.7700764"
    },
    {
        "destination_intersection": "I-710 and pico ave Long Beach, California",
        "x_coords": "-118.2080916",
        "y_coords": "33.7700875"
    },
    {
        "destination_intersection": "I-710 and pier t e/b off ramp Long Beach, California",
        "x_coords": "-118.2260474",
        "y_coords": "33.7629641"
    },
    {
        "destination_intersection": "I-710 and pier t Long Beach, California",
        "x_coords": "-118.2260895",
        "y_coords": "33.7629792"
    },
    {
        "destination_intersection": "I-710 and SR-91 Compton, California",
        "x_coords": "-118.1932723",
        "y_coords": "33.8746898"
    },
    {
        "destination_intersection": "I-710 and washington blvd Commerce, California",
        "x_coords": "-118.1729295",
        "y_coords": "34.0047569"
    },
    {
        "destination_intersection": "I-8 and Arizona Line",
        "x_coords": "-114.6178137",
        "y_coords": "32.7285541"
    },
    {
        "destination_intersection": "I-8 and greenfield dr El Cajon, California",
        "x_coords": "-116.9172186",
        "y_coords": "32.8117545"
    },
    {
        "destination_intersection": "I-8 and SR-2 El Cajon, California",
        "x_coords": "-116.9356432",
        "y_coords": "32.8030744"
    },
    {
        "destination_intersection": "I-8 and sunset cliffs blvd 997 G 2330 Shelter Island Dr #1, San Diego, , California",
        "x_coords": "-117.2359432",
        "y_coords": "32.754876"
    },
    {
        "destination_intersection": "I-8 and tavern rd Alpine, California",
        "x_coords": "-116.7795778",
        "y_coords": "32.8400159"
    },
    {
        "destination_intersection": "I-8 and west mission bay dr San Diego, California",
        "x_coords": "-117.2223854",
        "y_coords": "32.7568334"
    },
    {
        "destination_intersection": "I-80 and air base parkway Fairfield, California",
        "x_coords": "-122.0497253",
        "y_coords": "38.2733282"
    },
    {
        "destination_intersection": "I-80 and antelope rd Antelope, California",
        "x_coords": "-121.3093497",
        "y_coords": "38.7073879"
    },
    {
        "destination_intersection": "I-80 and cummings skyway Crockett, California",
        "x_coords": "-122.2379981",
        "y_coords": "38.0452101"
    },
    {
        "destination_intersection": "I-80 and el cerrito Richmond, California",
        "x_coords": "-122.3108027",
        "y_coords": "37.9008865"
    },
    {
        "destination_intersection": "I-80 and fremont st San Francisco, California",
        "x_coords": "-122.3918964",
        "y_coords": "37.7853507"
    },
    {
        "destination_intersection": "I-80 and gilman st Berkeley, California",
        "x_coords": "-122.3073069",
        "y_coords": "37.8781357"
    },
    {
        "destination_intersection": "I-80 and hirschdale rd Boca, California",
        "x_coords": "-120.0830475",
        "y_coords": "39.384001"
    },
    {
        "destination_intersection": "I-80 and manuel campos pkwy Fairfield, California",
        "x_coords": "-122.0340649",
        "y_coords": "38.2951872"
    },
    {
        "destination_intersection": "I-80 and midway rd Vacaville, California",
        "x_coords": "-121.8970831",
        "y_coords": "38.4161287"
    },
    {
        "destination_intersection": "I-80 and Nevada Line",
        "x_coords": "-120.0029996",
        "y_coords": "39.4680353"
    },
    {
        "destination_intersection": "I-80 and richmond parkway Richmond, California",
        "x_coords": "-122.3146378",
        "y_coords": "37.986782"
    },
    {
        "destination_intersection": "I-80 and riverside ave Roseville, California",
        "x_coords": "-121.2904235",
        "y_coords": "38.7245096"
    },
    {
        "destination_intersection": "I-80 and sierra college blvd Rocklin, California",
        "x_coords": "-121.2056626",
        "y_coords": "38.8029232"
    },
    {
        "destination_intersection": "I-80 and soda springs/norden Donner Pass, California",
        "x_coords": "-120.3924637",
        "y_coords": "39.3252948"
    },
    {
        "destination_intersection": "I-80 and soda springs/norden Soda Springs, California",
        "x_coords": "-120.3924788",
        "y_coords": "39.3253041"
    },
    {
        "destination_intersection": "I-80 and SR-12 Fairfield, California",
        "x_coords": "-122.0935655",
        "y_coords": "38.2391585"
    },
    {
        "destination_intersection": "I-80 and SR-37 Mare Island, California",
        "x_coords": "-122.2895161",
        "y_coords": "38.1174014"
    },
    {
        "destination_intersection": "I-80 and watt ave Sacramento, California",
        "x_coords": "-121.3831774",
        "y_coords": "38.6451666"
    },
    {
        "destination_intersection": "I-80 and west capitol ave Sacramento, California",
        "x_coords": "-121.5614941",
        "y_coords": "38.5784325"
    },
    {
        "destination_intersection": "I-80 and west capitol ave West Sacramento, California",
        "x_coords": "-121.5615751",
        "y_coords": "38.5783621"
    },
    {
        "destination_intersection": "I-80 and west el camino ave 2850 El Centro Rd, Sacramento, Ca, 95833, California",
        "x_coords": "-121.5354608",
        "y_coords": "38.6152157"
    },
    {
        "destination_intersection": "I-80 and winters st Mcclellan Park, California",
        "x_coords": "-121.4115316",
        "y_coords": "38.6374569"
    },
    {
        "destination_intersection": "I-805 and `h` st Chula Vista, California",
        "x_coords": "-117.0553991",
        "y_coords": "32.6405398"
    },
    {
        "destination_intersection": "I-805 and mira mesa blvd San Diego, California",
        "x_coords": "-117.2062322",
        "y_coords": "32.8903284"
    },
    {
        "destination_intersection": "I-805 and olympic pkwy Chula Vista, California",
        "x_coords": "-117.0406285",
        "y_coords": "32.5681779"
    },
    {
        "destination_intersection": "I-805 and plaza blvd National City, California",
        "x_coords": "-117.0846478",
        "y_coords": "32.6771961"
    },
    {
        "destination_intersection": "I-805 and telegraph cyn rd Chula Vista, California",
        "x_coords": "-117.0437249",
        "y_coords": "32.628982"
    },
    {
        "destination_intersection": "I-805 and university ave San Diego, California",
        "x_coords": "-117.1224556",
        "y_coords": "32.7486742"
    },
    {
        "destination_intersection": "I-880 and brokaw rd 2144 O'Toole Ave, San Jose, Ca 95131, California",
        "x_coords": "-121.9052357",
        "y_coords": "37.3822095"
    },
    {
        "destination_intersection": "I-880 and embarcadero Oakland, California",
        "x_coords": "-122.2560772",
        "y_coords": "37.7890885"
    },
    {
        "destination_intersection": "I-880 and gish rd San Jose, California",
        "x_coords": "-121.899838",
        "y_coords": "37.3680506"
    },
    {
        "destination_intersection": "I-880 and hegenberger road Oakland, California",
        "x_coords": "-122.1964253",
        "y_coords": "37.7406576"
    },
    {
        "destination_intersection": "I-880 and marina blvd 955 Marina Blvd, San Leandro, Ca, 94577, California",
        "x_coords": "-122.1665987",
        "y_coords": "37.7106532"
    },
    {
        "destination_intersection": "I-880 and marina blvd Marina Blvd, San Leandro, Ca, 94577, California",
        "x_coords": "-122.1667274",
        "y_coords": "37.7105514"
    },
    {
        "destination_intersection": "I-880 and marina blvd San Leandro, California",
        "x_coords": "-122.1667436",
        "y_coords": "37.7105641"
    },
    {
        "destination_intersection": "I-880 and market st Oakland, California",
        "x_coords": "-122.2832642",
        "y_coords": "37.8017155"
    },
    {
        "destination_intersection": "I-880 and montague expressway Milpitas, California",
        "x_coords": "-121.9090433",
        "y_coords": "37.4016063"
    },
    {
        "destination_intersection": "I-880 and mowry ave Newark, California",
        "x_coords": "-122.0022236",
        "y_coords": "37.5297782"
    },
    {
        "destination_intersection": "I-880 and oak st Oakland, California",
        "x_coords": "-122.2665171",
        "y_coords": "37.7951886"
    },
    {
        "destination_intersection": "I-880 and SR-66 5601 Oakport St ,Oakland, Ca, California",
        "x_coords": "-122.2082332",
        "y_coords": "37.7538666"
    },
    {
        "destination_intersection": "I-880 and SR-7 Oakland, California",
        "x_coords": "-122.304681",
        "y_coords": "37.8078278"
    },
    {
        "destination_intersection": "I-880 and SR-98 Oakland, California",
        "x_coords": "-122.1885187",
        "y_coords": "37.73171"
    },
    {
        "destination_intersection": "I-880 and SR-98 San Leandro (10700 Bigge St), California",
        "x_coords": "-122.1885828",
        "y_coords": "37.7316923"
    },
    {
        "destination_intersection": "I-880 and SR-98 San Leandro, California",
        "x_coords": "-122.18856",
        "y_coords": "37.7318175"
    },
    {
        "destination_intersection": "I-880 and SR-98 South San Francisco, California",
        "x_coords": "-122.1885728",
        "y_coords": "37.7317211"
    },
    {
        "destination_intersection": "I-880 and whipple rd Hwayward, California",
        "x_coords": "-122.0667041",
        "y_coords": "37.6067968"
    },
    {
        "destination_intersection": "I-880 and whipple rd 134 Neptune Ave, Hayward, California",
        "x_coords": "-122.0666842",
        "y_coords": "37.6068036"
    },
    {
        "destination_intersection": "I-880 and whipple rd Hayward, California",
        "x_coords": "-122.0667177",
        "y_coords": "37.606796"
    },
    {
        "destination_intersection": "I-880 and whipple rd Union City, California",
        "x_coords": "-122.0666098",
        "y_coords": "37.60678"
    },
    {
        "destination_intersection": "I-880 and winton ave Hayward, California",
        "x_coords": "-122.1014779",
        "y_coords": "37.6564185"
    },
    {
        "destination_intersection": "lake st Corona, California and I-15 Menifee, California",
        "x_coords": "-117.3935498",
        "y_coords": "33.7299952"
    },
    {
        "destination_intersection": "mojave/california city and barstow hwy Stockton, California",
        "x_coords": "-118.1487578",
        "y_coords": "35.0425924"
    },
    {
        "destination_intersection": "piers f and j 1320 Pier C St, Long Beach, Ca, 90813, California",
        "x_coords": "-118.200652",
        "y_coords": "33.7565224"
    },
    {
        "destination_intersection": "piers f and j Long Beach, California",
        "x_coords": "-118.2005201",
        "y_coords": "33.7567073"
    },
    {
        "destination_intersection": "piers f and j San Pedro, Ca 90731, California",
        "x_coords": "-118.2007563",
        "y_coords": "33.7566069"
    },
    {
        "destination_intersection": "roxford st Sylmar, California and I-5 Sun Valley, California",
        "x_coords": "-118.4796709",
        "y_coords": "34.303272"
    },
    {
        "destination_intersection": "spring st Long Beach, California and I-405 Long Beach, California",
        "x_coords": "-118.16165",
        "y_coords": "33.8116752"
    },
    {
        "destination_intersection": "SR-1 and `a` st Vandenberg Space Force Base, California",
        "x_coords": "-120.5205714",
        "y_coords": "34.7507815"
    },
    {
        "destination_intersection": "SR-1 and capitola / park ave Santa Cruz, California",
        "x_coords": "-121.9370337",
        "y_coords": "36.9839562"
    },
    {
        "destination_intersection": "SR-1 and dover dr Newport Beach/2200 West Coast Highway, California",
        "x_coords": "-117.9071933",
        "y_coords": "33.6162423"
    },
    {
        "destination_intersection": "SR-1 and hughes way 2080 S Hughes Way, El Segundo, Ca 90245, California",
        "x_coords": "-118.3958021",
        "y_coords": "33.907763"
    },
    {
        "destination_intersection": "SR-1 and imjin parkway Marina, California",
        "x_coords": "-121.813",
        "y_coords": "36.6694612"
    },
    {
        "destination_intersection": "SR-1 and las posas rd Oxnard, California",
        "x_coords": "-119.0853612",
        "y_coords": "34.1167625"
    },
    {
        "destination_intersection": "SR-1 and las posas rd Point Magu, California",
        "x_coords": "-119.0853693",
        "y_coords": "34.1167737"
    },
    {
        "destination_intersection": "SR-1 and marina Salinas, California",
        "x_coords": "-121.812895",
        "y_coords": "36.6694672"
    },
    {
        "destination_intersection": "SR-1 and rio del mar Soquel, California",
        "x_coords": "-121.8863298",
        "y_coords": "36.9750211"
    },
    {
        "destination_intersection": "SR-1 and santa fe ave 1320 Pier C St, Long Beach, Ca, California",
        "x_coords": "-118.2155832",
        "y_coords": "33.7898674"
    },
    {
        "destination_intersection": "SR-1 and SR-129 Watsonville, California",
        "x_coords": "-121.7742525",
        "y_coords": "36.8941559"
    },
    {
        "destination_intersection": "SR-1 and west 96th st 5908 Avion Dr, Los Angeles, California",
        "x_coords": "-118.3962542",
        "y_coords": "33.9493896"
    },
    {
        "destination_intersection": "SR-1 and wood rd Oxnard, Ca, California",
        "x_coords": "-119.0985455",
        "y_coords": "34.1332922"
    },
    {
        "destination_intersection": "SR-11 and enrico fermi dr 350 Alta Rd., San Diego Ca 92154, California",
        "x_coords": "-116.9278529",
        "y_coords": "32.5638418"
    },
    {
        "destination_intersection": "SR-11 and enrico fermi dr Otay Mesa, California",
        "x_coords": "-116.9278161",
        "y_coords": "32.5638533"
    },
    {
        "destination_intersection": "SR-11 and enrico fermi dr San Diego, California",
        "x_coords": "-116.9278121",
        "y_coords": "32.5638081"
    },
    {
        "destination_intersection": "SR-110 and anaheim st Wilmington, California",
        "x_coords": "-118.2802404",
        "y_coords": "33.7789682"
    },
    {
        "destination_intersection": "SR-110 and carson st Torrance, Ca, California",
        "x_coords": "-118.2873802",
        "y_coords": "33.8316095"
    },
    {
        "destination_intersection": "SR-110 and el segundo blvd Hawthorne, California",
        "x_coords": "-118.2859162",
        "y_coords": "33.9163869"
    },
    {
        "destination_intersection": "SR-110 and el segundo blvd 1521 W 134Th St., Gardena Ca 90249, California",
        "x_coords": "-118.2858304",
        "y_coords": "33.9164136"
    },
    {
        "destination_intersection": "SR-110 and florence ave National Sandblasting (Los Angeles), California",
        "x_coords": "-118.2808964",
        "y_coords": "33.9746582"
    },
    {
        "destination_intersection": "SR-110 and gaffey st San Pedro, Ca, California",
        "x_coords": "-118.292455",
        "y_coords": "33.7465496"
    },
    {
        "destination_intersection": "SR-110 and harry bridges blvd San Pedro, California",
        "x_coords": "-118.2799627",
        "y_coords": "33.7719055"
    },
    {
        "destination_intersection": "SR-110 and harry bridges blvd Wilmington, California",
        "x_coords": "-118.2799851",
        "y_coords": "33.7719082"
    },
    {
        "destination_intersection": "SR-110 and john s gibson blvd Compton, California",
        "x_coords": "-118.2798739",
        "y_coords": "33.7718359"
    },
    {
        "destination_intersection": "SR-110 and john s gibson blvd Los Angeles, California",
        "x_coords": "-118.2798954",
        "y_coords": "33.7718392"
    },
    {
        "destination_intersection": "SR-110 and redondo beach blvd Compton, California",
        "x_coords": "-118.2857003",
        "y_coords": "33.892626"
    },
    {
        "destination_intersection": "SR-110 and rosecrans ave Gardena, California",
        "x_coords": "-118.2865898",
        "y_coords": "33.9017612"
    },
    {
        "destination_intersection": "SR-110 and sepulveda blvd Wilmington, California",
        "x_coords": "-118.2876042",
        "y_coords": "33.8090394"
    },
    {
        "destination_intersection": "SR-110 and slauson ave Los Angeles City, California",
        "x_coords": "-118.2804865",
        "y_coords": "33.9891766"
    },
    {
        "destination_intersection": "SR-110 and slauson ave Vernon, California",
        "x_coords": "-118.2805261",
        "y_coords": "33.9891797"
    },
    {
        "destination_intersection": "SR-110 and slauson ave Certified Steel (Huntington Park), California",
        "x_coords": "-118.2804201",
        "y_coords": "33.9890608"
    },
    {
        "destination_intersection": "SR-110 and slauson ave Los Angeles, California",
        "x_coords": "-118.28051",
        "y_coords": "33.9891753"
    },
    {
        "destination_intersection": "SR-110 and SR-99 Los Angeles, California",
        "x_coords": "-118.2242129",
        "y_coords": "34.0816128"
    },
    {
        "destination_intersection": "SR-113 and churchill downs ave Williams, California",
        "x_coords": "-121.7652117",
        "y_coords": "38.6988169"
    },
    {
        "destination_intersection": "SR-113 and churchill downs ave Woodland, California",
        "x_coords": "-121.7652381",
        "y_coords": "38.6988164"
    },
    {
        "destination_intersection": "SR-113 and co rd Davis, California",
        "x_coords": "-121.7671718",
        "y_coords": "38.5901988"
    },
    {
        "destination_intersection": "SR-113 and co rd Woodland, California",
        "x_coords": "-121.7510669",
        "y_coords": "38.6769746"
    },
    {
        "destination_intersection": "SR-115 and evan hewes hwy Holtville, California",
        "x_coords": "-115.2858155",
        "y_coords": "32.7752371"
    },
    {
        "destination_intersection": "SR-115 and snyder rd 2653 E Wright Road, Holtville, California",
        "x_coords": "-115.3371064",
        "y_coords": "32.8110822"
    },
    {
        "destination_intersection": "SR-118 and balboa blvd Granada Hills, California",
        "x_coords": "-118.5024572",
        "y_coords": "34.27783"
    },
    {
        "destination_intersection": "SR-118 and hayvenhurst ave Granada Hills, California",
        "x_coords": "-118.493463",
        "y_coords": "34.2748215"
    },
    {
        "destination_intersection": "SR-118 and madera rd Simi Valley, California",
        "x_coords": "-118.7928193",
        "y_coords": "34.283802"
    },
    {
        "destination_intersection": "SR-118 and san fernando rd San Fernando, California",
        "x_coords": "-118.4289036",
        "y_coords": "34.2724548"
    },
    {
        "destination_intersection": "SR-118 and SR-27 Chatsworth, California",
        "x_coords": "-118.6047668",
        "y_coords": "34.2777243"
    },
    {
        "destination_intersection": "SR-118 and tampa ave Porter Ranch, California",
        "x_coords": "-118.5562257",
        "y_coords": "34.2723535"
    },
    {
        "destination_intersection": "SR-119 and orange ave Taft, California",
        "x_coords": "-119.4082928",
        "y_coords": "35.2074809"
    },
    {
        "destination_intersection": "SR-119 and taft highway Bakersfield, California",
        "x_coords": "-119.4483329",
        "y_coords": "35.1361354"
    },
    {
        "destination_intersection": "SR-12 and birds landing Rio Vista, California",
        "x_coords": "-121.8061462",
        "y_coords": "38.1836037"
    },
    {
        "destination_intersection": "SR-12 and commanche parkway Clements, California",
        "x_coords": "-121.0681086",
        "y_coords": "38.1976767"
    },
    {
        "destination_intersection": "SR-12 and currie rd 2360 Airport Rd, Rio Vista, Ca 94571, California",
        "x_coords": "-121.7782988",
        "y_coords": "38.183495"
    },
    {
        "destination_intersection": "SR-12 and south ham ln Ham Lane And Cardinal St., Lodi, Ca, California",
        "x_coords": "-121.2881431",
        "y_coords": "38.1154281"
    },
    {
        "destination_intersection": "SR-120 and airport way Manteca, California",
        "x_coords": "-121.2521957",
        "y_coords": "37.7832143"
    },
    {
        "destination_intersection": "SR-120 and SR-99 Ripon, California",
        "x_coords": "-121.1910759",
        "y_coords": "37.797225"
    },
    {
        "destination_intersection": "SR-132 and bird rd Vernalis, California",
        "x_coords": "-121.3611857",
        "y_coords": "37.6377793"
    },
    {
        "destination_intersection": "SR-134 and cahuenga blvd Los Angeles, California",
        "x_coords": "-118.3615113",
        "y_coords": "34.1532285"
    },
    {
        "destination_intersection": "SR-134 and san fernando rd Glendale, California",
        "x_coords": "-118.2754545",
        "y_coords": "34.1538227"
    },
    {
        "destination_intersection": "SR-138 and SR-110 Rosamond, California",
        "x_coords": "-118.3262027",
        "y_coords": "34.7761093"
    },
    {
        "destination_intersection": "SR-138 and SR-110 Rosemond, California",
        "x_coords": "-118.2248002",
        "y_coords": "34.0813333"
    },
    {
        "destination_intersection": "SR-138 and SR-133 Llano, California",
        "x_coords": "-117.8928544",
        "y_coords": "34.5064294"
    },
    {
        "destination_intersection": "SR-139 and or line Ca/Or Border, California",
        "x_coords": "-121.5189652",
        "y_coords": "41.997801"
    },
    {
        "destination_intersection": "SR-139 and tule lake Tule Lake, California",
        "x_coords": "-121.4645414",
        "y_coords": "41.9490471"
    },
    {
        "destination_intersection": "SR-14 and ave Rosamond, California",
        "x_coords": "-118.1717521",
        "y_coords": "34.8642512"
    },
    {
        "destination_intersection": "SR-14 and backus rd Rosamond, California",
        "x_coords": "-118.1611358",
        "y_coords": "34.9518988"
    },
    {
        "destination_intersection": "SR-14 and golden valley rd Santa Clarita, California",
        "x_coords": "-118.4725013",
        "y_coords": "34.3923228"
    },
    {
        "destination_intersection": "SR-14 and I-5 Castaic, California",
        "x_coords": "-118.5097511",
        "y_coords": "34.3360392"
    },
    {
        "destination_intersection": "SR-14 and red rover mine rd Acton, California",
        "x_coords": "-118.2188273",
        "y_coords": "34.4967993"
    },
    {
        "destination_intersection": "SR-14 and rosamond blvd Rosamond, California",
        "x_coords": "-118.1717916",
        "y_coords": "34.8642053"
    },
    {
        "destination_intersection": "SR-14 and rosamond Mojave, California",
        "x_coords": "-118.1716784",
        "y_coords": "34.8642329"
    },
    {
        "destination_intersection": "SR-14 and soledad cyn rd Acton, California",
        "x_coords": "-118.1366926",
        "y_coords": "34.4879987"
    },
    {
        "destination_intersection": "SR-14 and via princessa Santa Clarita, California",
        "x_coords": "-118.4566392",
        "y_coords": "34.4016749"
    },
    {
        "destination_intersection": "SR-145 and ave Madera, California",
        "x_coords": "-120.0604941",
        "y_coords": "36.9611937"
    },
    {
        "destination_intersection": "SR-152 and casa de fruta pkwy Hollister, California",
        "x_coords": "-121.3844342",
        "y_coords": "36.9883816"
    },
    {
        "destination_intersection": "SR-152 and I-10 Cupertino, California",
        "x_coords": "-121.5667198",
        "y_coords": "37.0221867"
    },
    {
        "destination_intersection": "SR-154 and foxen canyon rd 1728 Cottonwood St, Solvang, Ca 93463, California",
        "x_coords": "-120.1229461",
        "y_coords": "34.6679295"
    },
    {
        "destination_intersection": "SR-154 and SR-192 Santa Barbara, California",
        "x_coords": "-119.7623669",
        "y_coords": "34.4513236"
    },
    {
        "destination_intersection": "SR-160 and oakley/stockton 81 Big Break Rd, Oakley, Ca 94561, California",
        "x_coords": "-121.7555027",
        "y_coords": "37.99778"
    },
    {
        "destination_intersection": "SR-160 and wilbur ave Antioch, California",
        "x_coords": "-121.7521123",
        "y_coords": "38.0122201"
    },
    {
        "destination_intersection": "SR-162 and co rd Elk Creek, California",
        "x_coords": "-122.5303659",
        "y_coords": "39.6114389"
    },
    {
        "destination_intersection": "SR-163 and balboa ave San Diego, California",
        "x_coords": "-117.1467196",
        "y_coords": "32.8218165"
    },
    {
        "destination_intersection": "SR-163 and clairemont mesa blvd San Diego, California",
        "x_coords": "-117.1409805",
        "y_coords": "32.8323146"
    },
    {
        "destination_intersection": "SR-165 and I-5 Tracy, California",
        "x_coords": "-120.8405306",
        "y_coords": "36.9302761"
    },
    {
        "destination_intersection": "SR-168 and Clovis, California",
        "x_coords": "-119.7172437",
        "y_coords": "36.8230255"
    },
    {
        "destination_intersection": "SR-168 and fowler ave Clovis, California",
        "x_coords": "-119.6841129",
        "y_coords": "36.8408735"
    },
    {
        "destination_intersection": "SR-168 and herndon ave Clovis, California",
        "x_coords": "-119.7033873",
        "y_coords": "36.8373337"
    },
    {
        "destination_intersection": "SR-17 and scotts valley dr Saratoga, California",
        "x_coords": "-122.0011067",
        "y_coords": "37.0649588"
    },
    {
        "destination_intersection": "SR-18 and waterman canyon rd San Bernardino, California",
        "x_coords": "-117.2914019",
        "y_coords": "34.217171"
    },
    {
        "destination_intersection": "SR-180 and clovis ave Fresno, California",
        "x_coords": "-119.7002806",
        "y_coords": "36.7526979"
    },
    {
        "destination_intersection": "SR-190 and rd Porterville, California",
        "x_coords": "-119.0122115",
        "y_coords": "36.0510285"
    },
    {
        "destination_intersection": "SR-190 and road Pixley, California",
        "x_coords": "-119.3090526",
        "y_coords": "36.0512291"
    },
    {
        "destination_intersection": "SR-193 and taylor rd Newcastle, California",
        "x_coords": "-121.131748",
        "y_coords": "38.8775278"
    },
    {
        "destination_intersection": "SR-198 and plaza dr Visalia, California",
        "x_coords": "-119.3900086",
        "y_coords": "36.3272342"
    },
    {
        "destination_intersection": "SR-20 and bachelor valley 1275 W Hwy 20, Upper Lake, Ca 95485, California",
        "x_coords": "-122.9420288",
        "y_coords": "39.1732046"
    },
    {
        "destination_intersection": "SR-20 and george washington blvd 3056 Colusa Hwy Yuba City, Ca, California",
        "x_coords": "-121.6715751",
        "y_coords": "39.1413761"
    },
    {
        "destination_intersection": "SR-20 and george washington blvd Yuba City, California",
        "x_coords": "-121.6715779",
        "y_coords": "39.1413778"
    },
    {
        "destination_intersection": "SR-204 and union ave 1546 China Grade Loop, Bakersfield, Ca 93308, California",
        "x_coords": "-119.0030276",
        "y_coords": "35.3780691"
    },
    {
        "destination_intersection": "SR-210 and alder ave Rialto, California",
        "x_coords": "-117.4185032",
        "y_coords": "34.1363917"
    },
    {
        "destination_intersection": "SR-210 and baseline road Upland, California",
        "x_coords": "-117.6963999",
        "y_coords": "34.1216236"
    },
    {
        "destination_intersection": "SR-210 and cherry ave Fontana, California",
        "x_coords": "-117.4884496",
        "y_coords": "34.1363374"
    },
    {
        "destination_intersection": "SR-210 and citrus ave Banning, California",
        "x_coords": "-117.4535848",
        "y_coords": "34.1363862"
    },
    {
        "destination_intersection": "SR-210 and foothill blvd 9830 San Fernando Road Pacoima, Ca 91331, California",
        "x_coords": "-118.1850455",
        "y_coords": "34.1970818"
    },
    {
        "destination_intersection": "SR-210 and grand ave Glendora, California",
        "x_coords": "-117.8726863",
        "y_coords": "34.1206204"
    },
    {
        "destination_intersection": "SR-210 and huntington dr Duarte, California",
        "x_coords": "-118.0165477",
        "y_coords": "34.140142"
    },
    {
        "destination_intersection": "SR-210 and irwindale ave Azusa, California",
        "x_coords": "-117.9336383",
        "y_coords": "34.1298315"
    },
    {
        "destination_intersection": "SR-210 and irwindale ave Irwindale, California",
        "x_coords": "-117.9336584",
        "y_coords": "34.1298503"
    },
    {
        "destination_intersection": "SR-210 and maclay st 1191 Newton St, San Fernando, Ca, 91340, California",
        "x_coords": "-118.4192577",
        "y_coords": "34.2991206"
    },
    {
        "destination_intersection": "SR-210 and maclay st San Fernando, California",
        "x_coords": "-118.419601",
        "y_coords": "34.2991206"
    },
    {
        "destination_intersection": "SR-210 and pepper ave San Bernardino, California",
        "x_coords": "-117.3532498",
        "y_coords": "34.1343717"
    },
    {
        "destination_intersection": "SR-210 and state st San Bernardino, California",
        "x_coords": "-117.3354858",
        "y_coords": "34.1345355"
    },
    {
        "destination_intersection": "SR-210 and yarnell st Sylmar, California",
        "x_coords": "-118.479125",
        "y_coords": "34.3227882"
    },
    {
        "destination_intersection": "SR-22 and ducor) * Pixley, California",
        "x_coords": "-119.0534358",
        "y_coords": "35.8915689"
    },
    {
        "destination_intersection": "SR-22 and harbor blvd Garden Grove, Ca, California",
        "x_coords": "-117.9203834",
        "y_coords": "33.7677638"
    },
    {
        "destination_intersection": "SR-22 and harbor blvd Hungtington Beach, California",
        "x_coords": "-117.9204319",
        "y_coords": "33.7677647"
    },
    {
        "destination_intersection": "SR-22 and harbor blvd Santa Ana, California",
        "x_coords": "-117.9204121",
        "y_coords": "33.767718"
    },
    {
        "destination_intersection": "SR-22 and harbor blvd Garden Grove, California",
        "x_coords": "-117.9204581",
        "y_coords": "33.7677275"
    },
    {
        "destination_intersection": "SR-22 and SR-7 Long Beach, California",
        "x_coords": "-118.1060779",
        "y_coords": "33.7741294"
    },
    {
        "destination_intersection": "SR-221 and syar industries 50 Enterprise Ct, Napa, Ca, 94558, California",
        "x_coords": "-122.2711791",
        "y_coords": "38.2577369"
    },
    {
        "destination_intersection": "SR-223 and * Carson, California",
        "x_coords": "-118.2873578",
        "y_coords": "33.8244732"
    },
    {
        "destination_intersection": "SR-237 and great america parkway Santa Clara, California",
        "x_coords": "-121.9776045",
        "y_coords": "37.4155658"
    },
    {
        "destination_intersection": "SR-238 and washington ave 2584 Grant Ave, San Lorenzo, Ca 94580, California",
        "x_coords": "-122.1389057",
        "y_coords": "37.6911749"
    },
    {
        "destination_intersection": "SR-238 and washington ave San Leandro, California",
        "x_coords": "-122.1388351",
        "y_coords": "37.6911925"
    },
    {
        "destination_intersection": "SR-24 and orinda Orinda, California",
        "x_coords": "-122.1841625",
        "y_coords": "37.878814"
    },
    {
        "destination_intersection": "SR-241 and lake forrest dr Lakes Forest, California",
        "x_coords": "-117.660685",
        "y_coords": "33.6717986"
    },
    {
        "destination_intersection": "SR-247 and county dump rd Barstow, California",
        "x_coords": "-117.0198795",
        "y_coords": "34.8389338"
    },
    {
        "destination_intersection": "SR-25 and bolsa rd 1505 Bolsa Road Gilroy Ca, California",
        "x_coords": "-121.5230848",
        "y_coords": "36.9504636"
    },
    {
        "destination_intersection": "SR-25 and cienega rd Paicines, California",
        "x_coords": "-121.2769949",
        "y_coords": "36.7127796"
    },
    {
        "destination_intersection": "SR-269 and mt whitney ave 9790 W Mt Whitney Ave, Riverdale, Ca, California",
        "x_coords": "-120.1030401",
        "y_coords": "36.4295732"
    },
    {
        "destination_intersection": "SR-269 and mt whitney ave Mt Whitney/Five Point, California",
        "x_coords": "-120.1030239",
        "y_coords": "36.4295685"
    },
    {
        "destination_intersection": "SR-273 and alexander ave 5501 East St, Anderson, Ca, 96007, California",
        "x_coords": "-122.3149379",
        "y_coords": "40.4596717"
    },
    {
        "destination_intersection": "SR-273 and alexander ave Anderson, California",
        "x_coords": "-122.3149119",
        "y_coords": "40.4596986"
    },
    {
        "destination_intersection": "SR-29 and california dr Yountville, California",
        "x_coords": "-122.3591109",
        "y_coords": "38.3975361"
    },
    {
        "destination_intersection": "SR-29 and lakeport Upper Lake, California",
        "x_coords": "-122.9420562",
        "y_coords": "39.173169"
    },
    {
        "destination_intersection": "SR-29 and zinfandel ln 550 Community Hall Ln, St Helena, Ca 94574, California",
        "x_coords": "-122.4426043",
        "y_coords": "38.4826018"
    },
    {
        "destination_intersection": "SR-299 and helena Helena, California",
        "x_coords": "-123.1281182",
        "y_coords": "40.7696975"
    },
    {
        "destination_intersection": "SR-32 and chico 4444 Commerce Ln, Orland, Ca 95963, California",
        "x_coords": "-122.2089878",
        "y_coords": "39.7518242"
    },
    {
        "destination_intersection": "SR-33 and gonzaga rd 31426 Gonzaga Rd, Gustine, Ca, 95322, California",
        "x_coords": "-121.0163043",
        "y_coords": "37.0568326"
    },
    {
        "destination_intersection": "SR-33 and nees ave 985 12Th Street Firebaugh Ca, California",
        "x_coords": "-120.4563329",
        "y_coords": "36.8569412"
    },
    {
        "destination_intersection": "SR-33 and nees ave Firebaugh, California",
        "x_coords": "-120.456491",
        "y_coords": "36.8570393"
    },
    {
        "destination_intersection": "SR-33 and shell rd Ventura, Ca, California",
        "x_coords": "-119.2933313",
        "y_coords": "34.3169142"
    },
    {
        "destination_intersection": "SR-33 and shell rd Ventura, California",
        "x_coords": "-119.2932647",
        "y_coords": "34.3169857"
    },
    {
        "destination_intersection": "SR-36 and co rd Westwood, California",
        "x_coords": "-121.0186248",
        "y_coords": "40.3135267"
    },
    {
        "destination_intersection": "SR-37 and black point Novato, California",
        "x_coords": "-122.5062767",
        "y_coords": "38.114987"
    },
    {
        "destination_intersection": "SR-37 and mare island Mare Island, California",
        "x_coords": "-122.2895326",
        "y_coords": "38.1174175"
    },
    {
        "destination_intersection": "SR-4 and el dorado st Stockton, California",
        "x_coords": "-121.2860493",
        "y_coords": "37.9389648"
    },
    {
        "destination_intersection": "SR-4 and hillcrest ave Antioch, California",
        "x_coords": "-121.7854382",
        "y_coords": "37.9961949"
    },
    {
        "destination_intersection": "SR-4 and holt brothers tractor Stockton, California",
        "x_coords": "-121.308023",
        "y_coords": "37.9339281"
    },
    {
        "destination_intersection": "SR-4 and laurel rd Oakley, California",
        "x_coords": "-121.7442205",
        "y_coords": "37.9788968"
    },
    {
        "destination_intersection": "SR-4 and loveridge rd Pittsburg, California",
        "x_coords": "-121.8671724",
        "y_coords": "38.0117982"
    },
    {
        "destination_intersection": "SR-4 and solano way Martinez, California",
        "x_coords": "-122.0531656",
        "y_coords": "38.0009626"
    },
    {
        "destination_intersection": "SR-41 and friant rd Clovis, California",
        "x_coords": "-119.7891832",
        "y_coords": "36.8544096"
    },
    {
        "destination_intersection": "SR-41 and jensen ave Fresno, California",
        "x_coords": "-119.7849251",
        "y_coords": "36.7068758"
    },
    {
        "destination_intersection": "SR-41 and kansas ave Tulare, California",
        "x_coords": "-119.8160915",
        "y_coords": "36.2114855"
    },
    {
        "destination_intersection": "SR-41 and kettleman hills waste management Kettleman City, California",
        "x_coords": "-119.9933764",
        "y_coords": "35.9539882"
    },
    {
        "destination_intersection": "SR-41 and mt whitney ave 9790 W Mt Whitney Ave, Riverdale, Ca, California",
        "x_coords": "-119.8011906",
        "y_coords": "36.4310572"
    },
    {
        "destination_intersection": "SR-41 and SR-180 Fresno, California",
        "x_coords": "-119.7758783",
        "y_coords": "36.7543344"
    },
    {
        "destination_intersection": "SR-43 and ave Earlimart, California",
        "x_coords": "-119.5437669",
        "y_coords": "36.0936769"
    },
    {
        "destination_intersection": "SR-43 and ave Alpaugh, California",
        "x_coords": "-119.405356",
        "y_coords": "35.8913765"
    },
    {
        "destination_intersection": "SR-43 and ave Corcoran, California",
        "x_coords": "-119.5434501",
        "y_coords": "36.0934311"
    },
    {
        "destination_intersection": "SR-43 and pickerell ave Corcoran, California",
        "x_coords": "-119.5542101",
        "y_coords": "36.1068598"
    },
    {
        "destination_intersection": "SR-43 and SR-7 Bakersfield, California",
        "x_coords": "-119.2518377",
        "y_coords": "35.4415646"
    },
    {
        "destination_intersection": "SR-43 and whitley ave Corcoran (1300 Letts Ave), California",
        "x_coords": "-119.5458406",
        "y_coords": "36.0980406"
    },
    {
        "destination_intersection": "SR-44 and airport rd 3300 Bolam Creek Rd Redding, Ca 96002, California",
        "x_coords": "-122.3005645",
        "y_coords": "40.5610812"
    },
    {
        "destination_intersection": "SR-44 and eskimo hill turnout Lasson Park, California",
        "x_coords": "-121.5757972",
        "y_coords": "40.5624943"
    },
    {
        "destination_intersection": "SR-44 and victor ave Redding, California",
        "x_coords": "-122.3387887",
        "y_coords": "40.5797175"
    },
    {
        "destination_intersection": "SR-46 and river grove dr 180 Mcmillan Canyon Rd, Shandon, Ca, 93461, California",
        "x_coords": "-120.4997098",
        "y_coords": "35.6587337"
    },
    {
        "destination_intersection": "SR-47 and ferry st San Pedro, California",
        "x_coords": "-118.2629248",
        "y_coords": "33.7488538"
    },
    {
        "destination_intersection": "SR-49 and woodbridge rd Auburn, California",
        "x_coords": "-121.095178",
        "y_coords": "39.0386155"
    },
    {
        "destination_intersection": "SR-51 and exposition blvd Sacramento, California",
        "x_coords": "-121.4435424",
        "y_coords": "38.5969255"
    },
    {
        "destination_intersection": "SR-51 and watt ave Sacramento, California",
        "x_coords": "-121.3831896",
        "y_coords": "38.6390473"
    },
    {
        "destination_intersection": "SR-52 and cuyamaca st El Cajon, California",
        "x_coords": "-116.9838201",
        "y_coords": "32.8329743"
    },
    {
        "destination_intersection": "SR-54 and briarwood rd 4364 Bonita Road, Bonita, Ca 91902, California",
        "x_coords": "-117.0301288",
        "y_coords": "32.6815149"
    },
    {
        "destination_intersection": "SR-54 and I-5 Chula Vista, California",
        "x_coords": "-117.1041604",
        "y_coords": "32.650381"
    },
    {
        "destination_intersection": "SR-55 and chapman ave Orange, California",
        "x_coords": "-117.8309449",
        "y_coords": "33.7878422"
    },
    {
        "destination_intersection": "SR-55 and edinger ave Santa Ana, California",
        "x_coords": "-117.8349207",
        "y_coords": "33.7265846"
    },
    {
        "destination_intersection": "SR-55 and newport blvd Newport Beach - 101 Shipyard Way, California",
        "x_coords": "-117.9294945",
        "y_coords": "33.6207115"
    },
    {
        "destination_intersection": "SR-57 and ball rd Anaheim, California",
        "x_coords": "-117.8762925",
        "y_coords": "33.8180918"
    },
    {
        "destination_intersection": "SR-57 and bristol st Santa Ana, California",
        "x_coords": "-117.8843839",
        "y_coords": "33.7779911"
    },
    {
        "destination_intersection": "SR-57 and covina blvd San Dimas, California",
        "x_coords": "-117.8194341",
        "y_coords": "34.0993916"
    },
    {
        "destination_intersection": "SR-57 and katella ave Orange, California",
        "x_coords": "-117.880223",
        "y_coords": "33.8060887"
    },
    {
        "destination_intersection": "SR-57 and orangethorpe ave Placentia, California",
        "x_coords": "-117.8778394",
        "y_coords": "33.8610579"
    },
    {
        "destination_intersection": "SR-57 and SR-90 Brea, California",
        "x_coords": "-117.8835729",
        "y_coords": "33.9107527"
    },
    {
        "destination_intersection": "SR-57 and temple ave Walnut, California",
        "x_coords": "-117.8008784",
        "y_coords": "34.0434509"
    },
    {
        "destination_intersection": "SR-58 and borax rd Boron, California",
        "x_coords": "-117.7032815",
        "y_coords": "35.0067574"
    },
    {
        "destination_intersection": "SR-58 and boron Boron, California",
        "x_coords": "-117.6498233",
        "y_coords": "35.0102777"
    },
    {
        "destination_intersection": "SR-58 and coffee rd Bakersfield, California",
        "x_coords": "-119.09207",
        "y_coords": "35.3670298"
    },
    {
        "destination_intersection": "SR-58 and comanche dr Bakersfield, California",
        "x_coords": "-118.8432102",
        "y_coords": "35.3383315"
    },
    {
        "destination_intersection": "SR-58 and edwards afb Edwards, California",
        "x_coords": "-117.8811301",
        "y_coords": "35.0095775"
    },
    {
        "destination_intersection": "SR-58 and gibson st Bakersfield, California",
        "x_coords": "-119.0520854",
        "y_coords": "35.3831744"
    },
    {
        "destination_intersection": "SR-58 and hinkley rd Hinkley, California",
        "x_coords": "-117.1891096",
        "y_coords": "34.915035"
    },
    {
        "destination_intersection": "SR-58 and mohawk st Bakersfield, California",
        "x_coords": "-119.0659405",
        "y_coords": "35.3714908"
    },
    {
        "destination_intersection": "SR-58 and oswell st Bakersfield, California",
        "x_coords": "-118.9498049",
        "y_coords": "35.3522479"
    },
    {
        "destination_intersection": "SR-59 and SR-17 Merced, California",
        "x_coords": "-120.4876293",
        "y_coords": "37.2983524"
    },
    {
        "destination_intersection": "SR-59 and youd rd Winton, California",
        "x_coords": "-120.5052475",
        "y_coords": "37.3190771"
    },
    {
        "destination_intersection": "SR-60 and atlantic blvd Los Angeles, California",
        "x_coords": "-118.1499967",
        "y_coords": "34.0352121"
    },
    {
        "destination_intersection": "SR-60 and country village rd 13470 Philadelphia Ave Fontana, Ca 92337, California",
        "x_coords": "-117.5065959",
        "y_coords": "34.0187519"
    },
    {
        "destination_intersection": "SR-60 and grove ave Ontario, California",
        "x_coords": "-117.6284125",
        "y_coords": "34.0304717"
    },
    {
        "destination_intersection": "SR-60 and hamner ave Ontario, California",
        "x_coords": "-117.5581443",
        "y_coords": "34.0238691"
    },
    {
        "destination_intersection": "SR-60 and I-10 Rancho Mirage, California",
        "x_coords": "-118.2177036",
        "y_coords": "34.030818"
    },
    {
        "destination_intersection": "SR-60 and main st Colton (2650 La Cadena), California",
        "x_coords": "-117.3647721",
        "y_coords": "33.9964282"
    },
    {
        "destination_intersection": "SR-60 and main st Riverside (800 E La Cadena Dr), California",
        "x_coords": "-117.3647508",
        "y_coords": "33.9964993"
    },
    {
        "destination_intersection": "SR-60 and mountain ave Ontario, California",
        "x_coords": "-117.6676798",
        "y_coords": "34.0302693"
    },
    {
        "destination_intersection": "SR-60 and nogales st La Mirada, Ca, California",
        "x_coords": "-117.8886527",
        "y_coords": "33.994158"
    },
    {
        "destination_intersection": "SR-60 and pedley rd Riverside, California",
        "x_coords": "-117.4801647",
        "y_coords": "34.0192357"
    },
    {
        "destination_intersection": "SR-60 and reservoir st Pomona, California",
        "x_coords": "-117.7314779",
        "y_coords": "34.0249858"
    },
    {
        "destination_intersection": "SR-60 and rubidoux blvd Bloomington, California",
        "x_coords": "-117.4002875",
        "y_coords": "34.0045714"
    },
    {
        "destination_intersection": "SR-60 and rubidoux blvd Riverside, California",
        "x_coords": "-117.4003331",
        "y_coords": "34.004571"
    },
    {
        "destination_intersection": "SR-60 and south hamner ave Ontario, California",
        "x_coords": "-117.5581442",
        "y_coords": "34.0240824"
    },
    {
        "destination_intersection": "SR-60 and theodore st Moreno Valley, California",
        "x_coords": "-117.1393489",
        "y_coords": "33.9392493"
    },
    {
        "destination_intersection": "SR-60 and valley way Irwindale, California",
        "x_coords": "-117.96435",
        "y_coords": "34.0068516"
    },
    {
        "destination_intersection": "SR-60 and van buren blvd Mira Loma, California",
        "x_coords": "-117.5278953",
        "y_coords": "34.0175131"
    },
    {
        "destination_intersection": "SR-60 and vineyard ave Ontario, California",
        "x_coords": "-117.6109801",
        "y_coords": "34.0305026"
    },
    {
        "destination_intersection": "SR-62 and Arizona Line",
        "x_coords": "-114.2993208",
        "y_coords": "34.1596529"
    },
    {
        "destination_intersection": "SR-62 and yucca mesa rd Landers, California",
        "x_coords": "-116.3690523",
        "y_coords": "34.266126"
    },
    {
        "destination_intersection": "SR-65 and ave 2832 Richgrove Dr, Delano, Ca, California",
        "x_coords": "-119.0534417",
        "y_coords": "35.8800937"
    },
    {
        "destination_intersection": "SR-65 and forty mile rd Marysville, California",
        "x_coords": "-121.5410179",
        "y_coords": "39.0902002"
    },
    {
        "destination_intersection": "SR-65 and henderson ave 770 N Plano St, Porterville, Ca, 93257, California",
        "x_coords": "-119.0394462",
        "y_coords": "36.0801885"
    },
    {
        "destination_intersection": "SR-65 and henderson ave Porterville, California",
        "x_coords": "-119.0393116",
        "y_coords": "36.0804408"
    },
    {
        "destination_intersection": "SR-65 and merle haggard dr 2200 Pegasus Dr, Bakersfield, Ca, 93308, California",
        "x_coords": "-119.0746856",
        "y_coords": "35.4413734"
    },
    {
        "destination_intersection": "SR-65 and pleasant grove blvd Roseville, California",
        "x_coords": "-121.2840643",
        "y_coords": "38.7864641"
    },
    {
        "destination_intersection": "SR-65 and whitney ranch pkwy Rocklin, California",
        "x_coords": "-121.2999392",
        "y_coords": "38.8315292"
    },
    {
        "destination_intersection": "SR-67 and bradley ave 1148 Bert Acosta St, El Cajon, Ca 92020, California",
        "x_coords": "-116.960595",
        "y_coords": "32.8191126"
    },
    {
        "destination_intersection": "SR-67 and mapleview st Lakeside (10168 Channel Rd), California",
        "x_coords": "-116.9229699",
        "y_coords": "32.8628605"
    },
    {
        "destination_intersection": "SR-67 and riverford rd 10108 Riverford Rd, Lakeside, Ca, 92040, California",
        "x_coords": "-116.947118",
        "y_coords": "32.8549306"
    },
    {
        "destination_intersection": "SR-67 and slaughter house cyn rd 12570 Hwy 67 Lakeside, Ca. 92040, California",
        "x_coords": "-116.9427511",
        "y_coords": "32.9066808"
    },
    {
        "destination_intersection": "SR-67 and slaughter house cyn rd 12570 Hwy 67, Lakeside, Ca 92040, California",
        "x_coords": "-116.9427111",
        "y_coords": "32.9067234"
    },
    {
        "destination_intersection": "SR-7 and SR-65 Bakersfield, California",
        "x_coords": "-119.0746652",
        "y_coords": "35.4414296"
    },
    {
        "destination_intersection": "SR-7 and united states/mexico border I-7 Us/Mexico Border, California",
        "x_coords": "-115.3878704",
        "y_coords": "32.6736504"
    },
    {
        "destination_intersection": "SR-70 and big bend rd Oroville, California",
        "x_coords": "-121.50305",
        "y_coords": "39.725479"
    },
    {
        "destination_intersection": "SR-70 and bucks lake /main st Quincy, California",
        "x_coords": "-120.9488983",
        "y_coords": "39.936792"
    },
    {
        "destination_intersection": "SR-70 and bucks powerhouse rd Belden, California",
        "x_coords": "-121.2494512",
        "y_coords": "40.0069779"
    },
    {
        "destination_intersection": "SR-70 and mcgowan parkway 3736 Rancho Rd Marysville Ca 95901, California",
        "x_coords": "-121.541379",
        "y_coords": "39.077304"
    },
    {
        "destination_intersection": "SR-70 and plumas lake blvd Olivehurst, California",
        "x_coords": "-121.5468762",
        "y_coords": "39.0222268"
    },
    {
        "destination_intersection": "SR-70 and twain rd Twain, California",
        "x_coords": "-121.0714503",
        "y_coords": "40.0206029"
    },
    {
        "destination_intersection": "SR-71 and grand ave Chino, California",
        "x_coords": "-117.7239069",
        "y_coords": "34.0007007"
    },
    {
        "destination_intersection": "SR-71 and mission blvd Pomona, California",
        "x_coords": "-117.7812785",
        "y_coords": "34.0534673"
    },
    {
        "destination_intersection": "SR-71 and SR-142 Chino, California",
        "x_coords": "-117.7089818",
        "y_coords": "33.9825048"
    },
    {
        "destination_intersection": "SR-73 and jamboree rd Irvine, California",
        "x_coords": "-117.8649513",
        "y_coords": "33.6553919"
    },
    {
        "destination_intersection": "SR-73 and jamboree rd Newport Beach, California",
        "x_coords": "-117.8650058",
        "y_coords": "33.6553912"
    },
    {
        "destination_intersection": "SR-74 and central ave Lake Elsinore, California",
        "x_coords": "-117.3377414",
        "y_coords": "33.6918868"
    },
    {
        "destination_intersection": "SR-74 and palomar rd 28711 Mclaughlin Rd, Romoland, Ca 92585, California",
        "x_coords": "-117.1629965",
        "y_coords": "33.7430236"
    },
    {
        "destination_intersection": "SR-76 and college blvd Camp Pendleton @ 1211 Vandegrift Bvd, California",
        "x_coords": "-117.3001976",
        "y_coords": "33.2507609"
    },
    {
        "destination_intersection": "SR-78 and gfoc mine access rd Brawley, California",
        "x_coords": "-114.9899607",
        "y_coords": "33.0171663"
    },
    {
        "destination_intersection": "SR-78 and rancho santa fe rd San Marcos, Ca, California",
        "x_coords": "-117.1982532",
        "y_coords": "33.1501332"
    },
    {
        "destination_intersection": "SR-78 and SR-22 Blythe, California",
        "x_coords": "-114.6565695",
        "y_coords": "33.5444206"
    },
    {
        "destination_intersection": "SR-78 and sycamore ave 2485 Ash St, Vista, Ca, California",
        "x_coords": "-117.2146397",
        "y_coords": "33.1670124"
    },
    {
        "destination_intersection": "SR-78 and sycamore ave Vista, California",
        "x_coords": "-117.2145774",
        "y_coords": "33.1669219"
    },
    {
        "destination_intersection": "SR-78 and twin oaks valley rd San Marcos, California",
        "x_coords": "-117.1636809",
        "y_coords": "33.1370424"
    },
    {
        "destination_intersection": "SR-79 and holland rd 33011 Holland Rd Winchester Ca, California",
        "x_coords": "-117.0848776",
        "y_coords": "33.6709161"
    },
    {
        "destination_intersection": "SR-79 and patterson rd Hemet, California",
        "x_coords": "-117.0849207",
        "y_coords": "33.696033"
    },
    {
        "destination_intersection": "SR-84 and el camino real north Redwood City, California",
        "x_coords": "-122.221815",
        "y_coords": "37.4764311"
    },
    {
        "destination_intersection": "SR-86 and dillon rd Indio, California",
        "x_coords": "-116.1813235",
        "y_coords": "33.7078353"
    },
    {
        "destination_intersection": "SR-86 and grimes rd 3393 Ca-86 Imperial Ca 92251, California",
        "x_coords": "-115.5700904",
        "y_coords": "32.8981563"
    },
    {
        "destination_intersection": "SR-86 and I-8 Brawley, California",
        "x_coords": "-115.5696995",
        "y_coords": "32.7736466"
    },
    {
        "destination_intersection": "SR-86 and lydick loop 3275 Ca-86, Imperial, California",
        "x_coords": "-115.5699502",
        "y_coords": "32.8874282"
    },
    {
        "destination_intersection": "SR-905 and britannia blvd Otay Mesa, Ca, California",
        "x_coords": "-116.9798046",
        "y_coords": "32.5642534"
    },
    {
        "destination_intersection": "SR-905 and custom house rd San Diego, California",
        "x_coords": "-116.9399269",
        "y_coords": "32.5553443"
    },
    {
        "destination_intersection": "SR-905 and la media rd Otay Mesa, California",
        "x_coords": "-116.962433",
        "y_coords": "32.5642051"
    },
    {
        "destination_intersection": "SR-905 and la media rd Otey Mesa, California",
        "x_coords": "-116.9624329",
        "y_coords": "32.5641599"
    },
    {
        "destination_intersection": "SR-905 and la media rd San Diego, California",
        "x_coords": "-116.9622506",
        "y_coords": "32.5639338"
    },
    {
        "destination_intersection": "SR-905 and siempre viva rd San Diego, California",
        "x_coords": "-116.9398397",
        "y_coords": "32.5552681"
    },
    {
        "destination_intersection": "SR-91 and 19 Long Beach, California",
        "x_coords": "-118.1424854",
        "y_coords": "33.8764619"
    },
    {
        "destination_intersection": "SR-91 and bloomfield ave La Mirada, California",
        "x_coords": "-118.0639343",
        "y_coords": "33.8753743"
    },
    {
        "destination_intersection": "SR-91 and carmenita rd La Mirada, Ca, California",
        "x_coords": "-118.0463202",
        "y_coords": "33.8639811"
    },
    {
        "destination_intersection": "SR-91 and cherry ave Long Beach, California",
        "x_coords": "-118.1686985",
        "y_coords": "33.8760655"
    },
    {
        "destination_intersection": "SR-91 and green river rd Corona, California",
        "x_coords": "-117.6577535",
        "y_coords": "33.878569"
    },
    {
        "destination_intersection": "SR-91 and I-105 Compton, California",
        "x_coords": "-118.1799715",
        "y_coords": "33.9129307"
    },
    {
        "destination_intersection": "SR-91 and long beach blvd Compton, California",
        "x_coords": "-118.2022884",
        "y_coords": "33.8726484"
    },
    {
        "destination_intersection": "SR-91 and magnolia ave Anaheim, California",
        "x_coords": "-117.9761879",
        "y_coords": "33.8573135"
    },
    {
        "destination_intersection": "SR-91 and magnolia ave Riverside, California",
        "x_coords": "-117.4875612",
        "y_coords": "33.8970773"
    },
    {
        "destination_intersection": "SR-91 and main st 268 E Gardena Blvd., Gardena Ca 90247, California",
        "x_coords": "-118.275944",
        "y_coords": "33.8727445"
    },
    {
        "destination_intersection": "SR-91 and maple st Corona, California",
        "x_coords": "-117.6036965",
        "y_coords": "33.8801309"
    },
    {
        "destination_intersection": "SR-91 and mckinley st Corona, California",
        "x_coords": "-117.5179026",
        "y_coords": "33.8863358"
    },
    {
        "destination_intersection": "SR-91 and paramount blvd 6242 Paramount Blvd, Long Beach, Ca, 90805, California",
        "x_coords": "-118.1599687",
        "y_coords": "33.8761012"
    },
    {
        "destination_intersection": "SR-91 and paramount blvd Long Beach, California",
        "x_coords": "-118.1599769",
        "y_coords": "33.8761032"
    },
    {
        "destination_intersection": "SR-91 and santa fe ave Compton, California",
        "x_coords": "-118.2142751",
        "y_coords": "33.8733256"
    },
    {
        "destination_intersection": "SR-91 and serfas club dr Corona, California",
        "x_coords": "-117.6131851",
        "y_coords": "33.8807408"
    },
    {
        "destination_intersection": "SR-91 and SR-39 Buena Park, California",
        "x_coords": "-117.9980212",
        "y_coords": "33.8560863"
    },
    {
        "destination_intersection": "SR-91 and tustin ave Anaheim, California",
        "x_coords": "-117.8380319",
        "y_coords": "33.8509203"
    },
    {
        "destination_intersection": "SR-91 and tustin ave Placentia, California",
        "x_coords": "-117.8379031",
        "y_coords": "33.8511698"
    },
    {
        "destination_intersection": "SR-91 and west 6th st Corona, California",
        "x_coords": "-117.6037267",
        "y_coords": "33.8794054"
    },
    {
        "destination_intersection": "SR-91 and wilmington ave Compton, California",
        "x_coords": "-118.2360106",
        "y_coords": "33.873613"
    },
    {
        "destination_intersection": "SR-92 and eden landing rd 3643 Depot Rd Hayward Ca, California",
        "x_coords": "-122.1209419",
        "y_coords": "37.6281089"
    },
    {
        "destination_intersection": "SR-92 and eden landing rd Hayward, California",
        "x_coords": "-122.1233237",
        "y_coords": "37.6274121"
    },
    {
        "destination_intersection": "SR-92 and hesperian blvd Hayward, California",
        "x_coords": "-122.0999825",
        "y_coords": "37.6362933"
    },
    {
        "destination_intersection": "SR-92 and mariners island blvd San Mateo, California",
        "x_coords": "-122.2834377",
        "y_coords": "37.5573324"
    },
    {
        "destination_intersection": "SR-94 and euclid ave San Diego, California",
        "x_coords": "-117.0849517",
        "y_coords": "32.7180364"
    },
    {
        "destination_intersection": "SR-94 and lemon grove ave La Mesa, California",
        "x_coords": "-117.0311709",
        "y_coords": "32.746355"
    },
    {
        "destination_intersection": "SR-94 and SR-25 San Diego, California",
        "x_coords": "-117.1402533",
        "y_coords": "32.7131672"
    },
    {
        "destination_intersection": "SR-99 and adams ave Reedley, California",
        "x_coords": "-119.6929561",
        "y_coords": "36.6344198"
    },
    {
        "destination_intersection": "SR-99 and american ave Fresno, California",
        "x_coords": "-119.7248891",
        "y_coords": "36.6635432"
    },
    {
        "destination_intersection": "SR-99 and arch rd 4800 Logistics Dr, Stockton, Ca, California",
        "x_coords": "-121.2229224",
        "y_coords": "37.9047874"
    },
    {
        "destination_intersection": "SR-99 and arch rd Stockton, California",
        "x_coords": "-121.222901",
        "y_coords": "37.9048318"
    },
    {
        "destination_intersection": "SR-99 and atwater blvd Atwater, California",
        "x_coords": "-120.594823",
        "y_coords": "37.3401582"
    },
    {
        "destination_intersection": "SR-99 and austin rd Ripon, California",
        "x_coords": "-121.1790911",
        "y_coords": "37.7746182"
    },
    {
        "destination_intersection": "SR-99 and ave Firebaugh, California",
        "x_coords": "-120.7739866",
        "y_coords": "36.850315"
    },
    {
        "destination_intersection": "SR-99 and betty dr Visalia, California",
        "x_coords": "-119.4266062",
        "y_coords": "36.3509264"
    },
    {
        "destination_intersection": "SR-99 and buck owens blvd Bakersfield, California",
        "x_coords": "-119.043213",
        "y_coords": "35.3860281"
    },
    {
        "destination_intersection": "SR-99 and california ave Bakersfield, California",
        "x_coords": "-119.0029223",
        "y_coords": "35.3683946"
    },
    {
        "destination_intersection": "SR-99 and carpenter rd Modesto, Ca, California",
        "x_coords": "-121.0312806",
        "y_coords": "37.666001"
    },
    {
        "destination_intersection": "SR-99 and cherokee rd Stockton, California",
        "x_coords": "-121.2527359",
        "y_coords": "37.9921298"
    },
    {
        "destination_intersection": "SR-99 and chestnut ave Fresno, California",
        "x_coords": "-119.7367112",
        "y_coords": "36.674041"
    },
    {
        "destination_intersection": "SR-99 and clinton ave Fresno, California",
        "x_coords": "-119.8387113",
        "y_coords": "36.7718853"
    },
    {
        "destination_intersection": "SR-99 and county line Delano, California",
        "x_coords": "-119.2508377",
        "y_coords": "35.7903937"
    },
    {
        "destination_intersection": "SR-99 and crystal way Galt, California",
        "x_coords": "-121.2936813",
        "y_coords": "38.2543557"
    },
    {
        "destination_intersection": "SR-99 and esplanade 13407 Garner Lane, Chico, California",
        "x_coords": "-121.8988519",
        "y_coords": "39.7933894"
    },
    {
        "destination_intersection": "SR-99 and florin rd Sacramento, California",
        "x_coords": "-121.446027",
        "y_coords": "38.4961186"
    },
    {
        "destination_intersection": "SR-99 and franklin rd Merced, California",
        "x_coords": "-120.5411504",
        "y_coords": "37.3208602"
    },
    {
        "destination_intersection": "SR-99 and french camp Stockton, California",
        "x_coords": "-121.2187004",
        "y_coords": "37.8599273"
    },
    {
        "destination_intersection": "SR-99 and fresno st Fresno, California",
        "x_coords": "-119.7991918",
        "y_coords": "36.7310195"
    },
    {
        "destination_intersection": "SR-99 and golden state blvd Fresno, California",
        "x_coords": "-119.7880473",
        "y_coords": "36.7239859"
    },
    {
        "destination_intersection": "SR-99 and jensen ave Fresno, California",
        "x_coords": "-119.7748155",
        "y_coords": "36.7068224"
    },
    {
        "destination_intersection": "SR-99 and manning ave Selma, California",
        "x_coords": "-119.6602742",
        "y_coords": "36.604165"
    },
    {
        "destination_intersection": "SR-99 and mariposa rd Stockton, Ca ( 3755 E Munford Ave), California",
        "x_coords": "-121.2340252",
        "y_coords": "37.9337973"
    },
    {
        "destination_intersection": "SR-99 and mariposa rd Stockton, California",
        "x_coords": "-121.2340407",
        "y_coords": "37.9338281"
    },
    {
        "destination_intersection": "SR-99 and olive dr Bakersfield, California",
        "x_coords": "-119.0551317",
        "y_coords": "35.4121136"
    },
    {
        "destination_intersection": "SR-99 and panama ln Bakersfield, California",
        "x_coords": "-119.0305051",
        "y_coords": "35.2958771"
    },
    {
        "destination_intersection": "SR-99 and riego rd Pleasant Grove, California",
        "x_coords": "-121.5403209",
        "y_coords": "38.7500999"
    },
    {
        "destination_intersection": "SR-99 and shaw ave Fresno-4248 N Weber Ave, California",
        "x_coords": "-119.8867841",
        "y_coords": "36.808045"
    },
    {
        "destination_intersection": "SR-99 and SR-12 Sacramento, California",
        "x_coords": "-121.4737537",
        "y_coords": "38.5410968"
    },
    {
        "destination_intersection": "SR-99 and SR-165 Turlock, California",
        "x_coords": "-120.849024",
        "y_coords": "37.4738272"
    },
    {
        "destination_intersection": "SR-99 and SR-58 Edwards, California",
        "x_coords": "-119.0395362",
        "y_coords": "35.3526219"
    },
    {
        "destination_intersection": "SR-99 and SR-7 31101 Coberly Rd, Shafter, Ca 93263, California",
        "x_coords": "-119.0841822",
        "y_coords": "35.4413547"
    },
    {
        "destination_intersection": "SR-99 and SR-7 Bakersfield, California",
        "x_coords": "-119.0396125",
        "y_coords": "35.3524188"
    },
    {
        "destination_intersection": "SR-99 and stockton blvd 6101 Midway St, Sacramento, California",
        "x_coords": "-121.4167962",
        "y_coords": "38.4685456"
    },
    {
        "destination_intersection": "turlock rd and SR-17 12523 N Ca-59, Merced, California",
        "x_coords": "-120.5012293",
        "y_coords": "37.4932756"
    },
    {
        "destination_intersection": "US-101 and airport blvd 10 Harris Place Salinas, Ca, California",
        "x_coords": "-121.6227935",
        "y_coords": "36.6607793"
    },
    {
        "destination_intersection": "US-101 and airport blvd Salinas, California",
        "x_coords": "-121.6227506",
        "y_coords": "36.6608137"
    },
    {
        "destination_intersection": "US-101 and bayshore blvd Brisbane, California",
        "x_coords": "-122.3985541",
        "y_coords": "37.7159498"
    },
    {
        "destination_intersection": "US-101 and bayshore blvd San Francisco, California",
        "x_coords": "-122.3988545",
        "y_coords": "37.7163226"
    },
    {
        "destination_intersection": "US-101 and bernal rd San Jose, California",
        "x_coords": "-121.772719",
        "y_coords": "37.2440501"
    },
    {
        "destination_intersection": "US-101 and capitol expwy w/b off ramp San Jose, California",
        "x_coords": "-121.8167221",
        "y_coords": "37.3031083"
    },
    {
        "destination_intersection": "US-101 and cochrane rd Morgan Hill, California",
        "x_coords": "-121.6525275",
        "y_coords": "37.1529515"
    },
    {
        "destination_intersection": "US-101 and dry creek rd Healdsburg, California",
        "x_coords": "-122.8781561",
        "y_coords": "38.6262299"
    },
    {
        "destination_intersection": "US-101 and garden st 208 N Cesar Chavez, Santa Barbara, Ca 93103, California",
        "x_coords": "-119.6898229",
        "y_coords": "34.4182135"
    },
    {
        "destination_intersection": "US-101 and grand ave South San Francisco, California",
        "x_coords": "-122.4067962",
        "y_coords": "37.65533"
    },
    {
        "destination_intersection": "US-101 and hearn ave Santa Rosa, California",
        "x_coords": "-122.7160693",
        "y_coords": "38.4146106"
    },
    {
        "destination_intersection": "US-101 and holly st San Carlos, California",
        "x_coords": "-122.2562684",
        "y_coords": "37.5138929"
    },
    {
        "destination_intersection": "US-101 and I-405 Long Beach, California",
        "x_coords": "-118.2847008",
        "y_coords": "33.8568989"
    },
    {
        "destination_intersection": "US-101 and lawrence expressway Sunnyvale, California",
        "x_coords": "-121.9959947",
        "y_coords": "37.3909993"
    },
    {
        "destination_intersection": "US-101 and lewis rd Camarillo, California",
        "x_coords": "-119.0340798",
        "y_coords": "34.2167657"
    },
    {
        "destination_intersection": "US-101 and los osos valley rd San Luis Obispo, California",
        "x_coords": "-120.6822215",
        "y_coords": "35.2446106"
    },
    {
        "destination_intersection": "US-101 and lynn rd Thousand Oaks, California",
        "x_coords": "-118.8915726",
        "y_coords": "34.1834929"
    },
    {
        "destination_intersection": "US-101 and main st Ventura, California",
        "x_coords": "-119.3059455",
        "y_coords": "34.2852034"
    },
    {
        "destination_intersection": "US-101 and millbrae ave Millbrae, California",
        "x_coords": "-122.3874193",
        "y_coords": "37.5984789"
    },
    {
        "destination_intersection": "US-101 and moffett field Mountain View, California",
        "x_coords": "-122.0663499",
        "y_coords": "37.407631"
    },
    {
        "destination_intersection": "US-101 and monterey peninsula 1300 Abbott St, Salinas, Ca, California",
        "x_coords": "-121.5859869",
        "y_coords": "36.6264145"
    },
    {
        "destination_intersection": "US-101 and new hunter creek rd Klamath, California",
        "x_coords": "-124.0626425",
        "y_coords": "41.5614906"
    },
    {
        "destination_intersection": "US-101 and north access rd 900 N Access Rd, San Francisco, Ca 94128, USA, California",
        "x_coords": "-122.4036142",
        "y_coords": "37.6358793"
    },
    {
        "destination_intersection": "US-101 and old redwood hwy Santa Rosa, California",
        "x_coords": "-122.8082138",
        "y_coords": "38.5469738"
    },
    {
        "destination_intersection": "US-101 and oyster point blvd South San Francisco, California",
        "x_coords": "-122.3993619",
        "y_coords": "37.663357"
    },
    {
        "destination_intersection": "US-101 and petaluma blvd Santa Rosa, California",
        "x_coords": "-122.6074975",
        "y_coords": "38.2207817"
    },
    {
        "destination_intersection": "US-101 and reseda blvd Reseda, California",
        "x_coords": "-118.5360266",
        "y_coords": "34.1732528"
    },
    {
        "destination_intersection": "US-101 and rose ave 1930 Lockwood St, Oxnard, Ca, 93036, California",
        "x_coords": "-119.1587546",
        "y_coords": "34.2251429"
    },
    {
        "destination_intersection": "US-101 and rose ave Oxnard, California",
        "x_coords": "-119.1587616",
        "y_coords": "34.2250028"
    },
    {
        "destination_intersection": "US-101 and san bruno ave 800 S Airport Blvd, San Francisco, Ca, California",
        "x_coords": "-122.4027198",
        "y_coords": "37.6312903"
    },
    {
        "destination_intersection": "US-101 and san bruno ave San Francisco, Ca, California",
        "x_coords": "-122.4026996",
        "y_coords": "37.6312997"
    },
    {
        "destination_intersection": "US-101 and san bruno ave San Francisco, California",
        "x_coords": "-122.4077555",
        "y_coords": "37.7357765"
    },
    {
        "destination_intersection": "US-101 and san martin ave San Martin, California",
        "x_coords": "-121.5994601",
        "y_coords": "37.0887374"
    },
    {
        "destination_intersection": "US-101 and santa rosa ave Santa Rosa, California",
        "x_coords": "-122.7135471",
        "y_coords": "38.421183"
    },
    {
        "destination_intersection": "US-101 and seaward ave Oxnard, California",
        "x_coords": "-119.2692158",
        "y_coords": "34.264995"
    },
    {
        "destination_intersection": "US-101 and shiloh rd Windsor, California",
        "x_coords": "-122.7900886",
        "y_coords": "38.5257808"
    },
    {
        "destination_intersection": "US-101 and SR-12 Fortuna Ca, California",
        "x_coords": "-124.1548172",
        "y_coords": "40.5874399"
    },
    {
        "destination_intersection": "US-101 and SR-232 100 Port Hueneme Rd, Port Hueneme, California",
        "x_coords": "-119.1732648",
        "y_coords": "34.2318579"
    },
    {
        "destination_intersection": "US-101 and SR-232 Port Hueneme, California",
        "x_coords": "-119.1732464",
        "y_coords": "34.231859"
    },
    {
        "destination_intersection": "US-101 and todd rd Santa Rosa, California",
        "x_coords": "-122.7162651",
        "y_coords": "38.3870075"
    },
    {
        "destination_intersection": "US-101 and trimble rd 2130 De La Cruz Blvd, Santa Clara, Ca 95050, California",
        "x_coords": "-121.9417814",
        "y_coords": "37.3767666"
    },
    {
        "destination_intersection": "US-101 and trimble rd Santa Clara, California",
        "x_coords": "-121.94173",
        "y_coords": "37.376541"
    },
    {
        "destination_intersection": "US-101 and tully rd San Jose, California",
        "x_coords": "-121.8312885",
        "y_coords": "37.3181725"
    },
    {
        "destination_intersection": "US-101 and tully road San Jose, California",
        "x_coords": "-121.831543",
        "y_coords": "37.3184288"
    },
    {
        "destination_intersection": "US-101 and US-101 Greenfield, California",
        "x_coords": "-121.2560515",
        "y_coords": "36.3405545"
    },
    {
        "destination_intersection": "US-101 and US-101 San Francisco City, California",
        "x_coords": "-122.4692308",
        "y_coords": "37.8029302"
    },
    {
        "destination_intersection": "US-101 and valley rd Piercy, California",
        "x_coords": "-123.7950916",
        "y_coords": "39.9659427"
    },
    {
        "destination_intersection": "US-101 and victoria ave Oxnard, California",
        "x_coords": "-119.2110173",
        "y_coords": "34.2525003"
    },
    {
        "destination_intersection": "US-101 and woodside rd Redwood City, California",
        "x_coords": "-122.2128795",
        "y_coords": "37.4888443"
    },
    {
        "destination_intersection": "US-395 and air expressway 18200 Phantom W, Victorville, Ca 92394, California",
        "x_coords": "-117.4095765",
        "y_coords": "34.5719811"
    },
    {
        "destination_intersection": "US-395 and farmington rd Az Line Hwy 10, California",
        "x_coords": "-114.5310075",
        "y_coords": "33.6047422"
    },
    {
        "destination_intersection": "US-395 and farmington rd Byron, California",
        "x_coords": "-121.6410217",
        "y_coords": "37.8904863"
    },
    {
        "destination_intersection": "US-395 and nv line Nv Line Us-395 Border, California",
        "x_coords": "-120.0070829",
        "y_coords": "39.6795194"
    },
    {
        "destination_intersection": "US-395 and nv line Nv Line, California",
        "x_coords": "-120.0061014",
        "y_coords": "39.6838903"
    },
    {
        "destination_intersection": "US-395 and SR-3 Hesperia, California",
        "x_coords": "-117.3999983",
        "y_coords": "34.4267374"
    },   
    {
        "destination_intersection": "US-395 and US-6 Nv State Line, California",
        "x_coords": "-118.4359206",
        "y_coords": "37.9007438"
    },
    {
        "destination_intersection": "US-50 and e bidwell st Rancho Cordova, California",
        "x_coords": "-121.114902",
        "y_coords": "38.6432199"
    },
    {
        "destination_intersection": "US-50 and harbor blvd West Sacramento, California",
        "x_coords": "-121.5486244",
        "y_coords": "38.5749478"
    },
    {
        "destination_intersection": "US-50 and hazel ave Sacramento, California",
        "x_coords": "-121.2169889",
        "y_coords": "38.6307947"
    },
    {
        "destination_intersection": "US-50 and latrobe rd El Dorado Hills, California",
        "x_coords": "-121.070945",
        "y_coords": "38.6523749"
    },
    {
        "destination_intersection": "US-50 and ponderosa rd Shingle Springs, California",
        "x_coords": "-120.9375951",
        "y_coords": "38.6625846"
    },
    {
        "destination_intersection": "US-50 and power inn rd Sacramento, California",
        "x_coords": "-121.4091435",
        "y_coords": "38.5494385"
    },
    {
        "destination_intersection": "US-50 and SR-16 Sacramento, California",
        "x_coords": "-121.4916141",
        "y_coords": "38.5633056"
    },
    {
        "destination_intersection": "US-50 and SR-65 Sacramento, California",
        "x_coords": "-121.4277665",
        "y_coords": "38.5506428"
    },
    {
        "destination_intersection": "US-50 and sunrise blvd Rancho Cordova, California",
        "x_coords": "-121.2704757",
        "y_coords": "38.6090307"
    },
    {
        "destination_intersection": "US-50 and sunrise blvd Sacramento, California",
        "x_coords": "-121.2704693",
        "y_coords": "38.6089921"
    },
    {
        "destination_intersection": "US-50 and watt ave 8594 Fruitridge Rd, Sacramento, Ca, California",
        "x_coords": "-121.3794416",
        "y_coords": "38.5587472"
    },
    {
        "destination_intersection": "US-50 and watt ave North Highlands, California",
        "x_coords": "-121.379404",
        "y_coords": "38.558696"
    },
    {
        "destination_intersection": "US-50 and watt ave Sacramento, California",
        "x_coords": "-121.3794064",
        "y_coords": "38.5587018"
    },
    {
        "destination_intersection": "US-50 and West Sacramento, California",
        "x_coords": "-121.5164582",
        "y_coords": "38.5719439"
    },
    {
        "destination_intersection": "US-6 and nv line Ca/Nv Line, California",
        "x_coords": "-118.436053",
        "y_coords": "37.9006402"
    },
    {
        "destination_intersection": "US-6 and nv line Nevada Line, California",
        "x_coords": "-118.4356914",
        "y_coords": "37.9008981"
    },
    {
        "destination_intersection": "US-6 and nv line Nv Line 6, California",
        "x_coords": "-118.4358932",
        "y_coords": "37.9007829"
    },
    {
        "destination_intersection": "US-6 and nv line Nv Line, California",
        "x_coords": "-118.4360461",
        "y_coords": "37.9006661"
    },
    {
        "destination_intersection": "US-6 and nv line Nv State Line, California",
        "x_coords": "-118.4355557",
        "y_coords": "37.9010263"
    },
    {
        "destination_intersection": "US-6 and nv line State Line, California",
        "x_coords": "-118.4356613",
        "y_coords": "37.9009122"
    },
    {
        "destination_intersection": "US-95 and havasu lake rd 13145 Havasu Lake Rd. Needles, Ca., California",
        "x_coords": "-114.6451912",
        "y_coords": "34.5577884"
    },
    {
        "destination_intersection": "US-95 and nv line Ca/Nv State Line, California",
        "x_coords": "-114.6108829",
        "y_coords": "34.8370064"
    },
    {
        "destination_intersection": "US-95 and nv line Nv Line, California",
        "x_coords": "-114.6108346",
        "y_coords": "34.8370367"
    },
    {
        "destination_intersection": "US-95 and SR-2 Blythe, California",
        "x_coords": "-114.5726544",
        "y_coords": "33.6907779"
    },
    {
        "destination_intersection": "US-97 and lake shastina Weed, California",
        "x_coords": "-122.3516034",
        "y_coords": "41.4726173"
    },
    {
        "destination_intersection": "US-97 and or line Ca/Or Line On 97, California",
        "x_coords": "-121.8893875",
        "y_coords": "42.0030092"
    },
    {
        "destination_intersection": "US-97 and or line Or Line 97, California",
        "x_coords": "-121.8903294",
        "y_coords": "42.0019217"
    },
    {
        "destination_intersection": "US-97 and or line Oregon @ Hwy 97, California",
        "x_coords": "-121.8893891",
        "y_coords": "42.0029698"
    },
    {
        "destination_intersection": "walnut st and SR-7 Earlimart, California",
        "x_coords": "-119.284043",
        "y_coords": "35.967657"
    }
]

In [27]:
type(destination_locations)

list